In [1]:
import requests
import json
import numpy as np
import pandas as pd
from unicodedata import normalize
from sqlalchemy import create_engine
import os
import psycopg2
import io
import matplotlib.pyplot as plt
import warnings
import folium
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from datetime import date
from datetime import datetime
from IPython.display import Image
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
#Conexión con la base de datos
host = 'team73test.c4xr0dadx0gz.us-east-2.rds.amazonaws.com'
port = 5432
user = 'postgres'
password = 'ds4a_t73'
database = 'postgres'
connDB = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
conn = connDB.raw_connection()
cur = conn.cursor()
%load_ext sql
%sql postgresql://{user}:{password}@{host}/{database}

'Connected: postgres@postgres'

# Load RNDC

In [3]:
df_RNDC = pd.read_sql(''' SELECT * FROM carguebogota.rndc ''', conn)

In [4]:
df_rndc = df_RNDC.copy()

In [5]:
df_rndc['hour'] = pd.to_datetime(df_rndc['hour'], format='%H:%M:%S').dt.hour

In [6]:
df_rndc.head()

,weekday,year,month,date,hour,operation_code,city_cod,city,paid_vale,a_remittances,cod_product,product,plaque,setting,corridor,driver,corridor_2
0,Sabado,2019,4,2019-04-13,14,True,25148000,CAPARRAPI CUNDINAMARCA,0.0,1,2501,SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y...,313236,2,CALLE 80,140758.0,CALLE 13
1,Martes,2019,4,2019-04-16,15,True,25260000,EL ROSAL CUNDINAMARCA,0.0,3,1704,AZÚCARES Y ARTÍCULOS DE CONFITERÍA; ARTICULOS ...,1395406,2,CALLE 80,103268.0,CALLE 13
2,Lunes,2019,5,2019-05-20,8,True,25320000,GUADUAS CUNDINAMARCA,2500000.0,1,7224,FUNDICIÓN; HIERRO Y ACERO; LOS DEMAS ACEROS AL...,208251,3S3,CALLE 80,119366.0,CALLE 13
3,Jueves,2019,1,2019-01-31,21,True,66001000,PEREIRA RISARALDA,600.0,1,9980,VARIOS PRODUCTOS VARIOS,219609,2,AUTO SUR,185797.0,CALLE 13
4,Lunes,2019,5,2019-05-27,8,True,25320000,GUADUAS CUNDINAMARCA,2500000.0,1,7224,FUNDICIÓN; HIERRO Y ACERO; LOS DEMAS ACEROS AL...,208251,3S3,CALLE 80,119366.0,CALLE 13


In [7]:
setting_grouped = {'2':'C2P', 
                   '3S3':'TRACTOCAMIONES', 
                   '2S2':'TRACTOCAMIONES', 
                   '3S2':'TRACTOCAMIONES', 
                   '2S3':'TRACTOCAMIONES', 
                   'CA':'TRACTOCAMIONES', 
                   '3':'C2G', 
                   '4':'TRACTOCAMIONES', 
                   '3S4':'TRACTOCAMIONES', 
                   '2R2':'TRACTOCAMIONES',
                   '2S1':'TRACTOCAMIONES', 
                   '3R3':'TRACTOCAMIONES', 
                   '4R4':'TRACTOCAMIONES', 
                   '3S1':'TRACTOCAMIONES', 
                   '2B2':'TRACTOCAMIONES', 
                   '2B1':'TRACTOCAMIONES', 
                   '3B3':'TRACTOCAMIONES', 
                   '3S2S2':'TRACTOCAMIONES', 
                   '2R3':'TRACTOCAMIONES',
                   '3B2':'TRACTOCAMIONES', 
                   '3S3S2':'TRACTOCAMIONES', 
                   '3R2':'TRACTOCAMIONES'}

df_rndc['setting_grouped'] = df_rndc['setting'].map(setting_grouped)

# Categorías RNDC

In [8]:
#PRODUCT 

#Alimentos
alimentos = ['AZÚCARES Y ARTÍCULOS DE CONFITERÍA; ARTICULOS DE CONFITERIA SIN CACAO (INCLUIDO EL CHOCOLATE BLANCO)',
 'AZÚCARES Y ARTÍCULOS DE CONFITERÍA; AZUCAR DE CA?A O DE REMOLACHA Y SACAROSA QUIMICAMENTE PURA; EN ESTADO SOLIDO',
 'AZÚCARES Y ARTÍCULOS DE CONFITERÍA; LOS DEMAS AZUCARES;  INCLUIDAS LA LACTOSA;  LAMALTOSA;  LA GLUCOSA Y LA FRUCTO',
 'AZÚCARES Y ARTÍCULOS DE CONFITERÍA; MELAZA DE LA EXTRACCION O DEL REFINADO DEL AZUCAR',
 'CACAO Y SUS PREPARACIONES; CACAO EN GRANO;  ENTERO O PARTIDO;  CRUDO O TOSTADO',
 'CACAO Y SUS PREPARACIONES; CACAO EN POLVO SIN AZUCARAR NI EDULCORAR DE OTRO MODO',
 'CACAO Y SUS PREPARACIONES; CASCARA;  PELICULAS Y DEMAS RESIDUOS DE CACAO',
 'CACAO Y SUS PREPARACIONES; CHOCOLATE Y DEMAS PREPARACIONES ALIMENTICIAS QUE CONTENGAN CACAO',
 'CACAO Y SUS PREPARACIONES; MANTECA;  GRASA Y ACEITE DE CACAO',
 'CACAO Y SUS PREPARACIONES; PASTA DE CACAO;  INCLUSO DESGRASADA',
 'CACAO\xa0Y\xa0SUS\xa0PREPARACIONES;\xa0MANTECA;\xa0\xa0GRASA\xa0Y\xa0ACEITE\xa0DE\xa0CACAO',
 'CAFÉ; TÉ; YERBA MATE Y ESPECIAS; CAFE;  INCLUSO TOSTADO O DESCAFEINADO; CASCARA O CASCARILLA DE CAFE; SUCEDANEO',
 'CAFÉ; TÉ; YERBA MATE Y ESPECIAS; CANELA Y FLORES DEL CANELERO',
 'CAFÉ; TÉ; YERBA MATE Y ESPECIAS; CLAVO(FRUTOS CLAVILLOS Y PEDUNCULOS)',
 'CAFÉ; TÉ; YERBA MATE Y ESPECIAS; JENGIBRE; AZAFRAN; CURCUMA; TOMILLO; HOJAS DE LAUREL; CURRY Y DEMAS ESPECIAS',
 'CAFÉ; TÉ; YERBA MATE Y ESPECIAS; NUEZ MOSCADA; MACIS; AMOMOS Y CARDAMOMOS',
 'CAFÉ; TÉ; YERBA MATE Y ESPECIAS; PIMIENTA SECOS; TRITURADOS O PULVERIZADOS',
 'CAFÉ; TÉ; YERBA MATE Y ESPECIAS; SEMILLAS DE ANIS; BADIANA; HINOJO; CILANTRO; COMINO O ALCARAVEA; BAYAS DE ENEB',
 'CAFÉ; TÉ; YERBA MATE Y ESPECIAS; TE;  INCLUSO AROMATIZADO',
 'CAFÉ; TÉ; YERBA MATE Y ESPECIAS; VAINILLA',
 'CAFÉ;\xa0TÉ;\xa0YERBA\xa0MATE\xa0Y\xa0ESPECIAS;\xa0CLAVO(FRUTOS\xa0CLAVILLOS\xa0Y\xa0PEDUNCULOS)',
 'CAFÉ;\xa0TÉ;\xa0YERBA\xa0MATE\xa0Y\xa0ESPECIAS;\xa0PIMIENTA\xa0SECOS;\xa0TRITURADOS\xa0O\xa0PULVERIZADOS',
 'CAFÉ;\xa0TÉ;\xa0YERBA\xa0MATE\xa0Y\xa0ESPECIAS;\xa0TE;\xa0\xa0INCLUSO\xa0AROMATIZADO',
 'CARNES Y DESPOJOS COMESTIBLES CARNE DE ANIMALES DE LA ESPECIE BOVINA; CONGELADA',
 'CARNES Y DESPOJOS COMESTIBLES CARNE DE ANIMALES DE LA ESPECIE BOVINA; FRESCA O REFRIGERADA',
 'CARNES Y DESPOJOS COMESTIBLES CARNE DE ANIMALES DE LA ESPECIE PORCINA; FRESCA; REFRIGERADA O CONGELADA',
 'CARNES Y DESPOJOS COMESTIBLES CARNE DE ANIMALES DE LAS ESPECIES CABALLAR; ASNAL O MULAR FRESCA; REFRIGERADA',
 'CARNES Y DESPOJOS COMESTIBLES CARNE DE ANIMALES DE LAS ESPECIES OVINA O CAPRINA; FRESCA; REFRIGERADA O CONGE',
 'CARNES Y DESPOJOS COMESTIBLES CARNE Y DESPOJOS COMESTIBLES DE AVES DE LA PARTIDA 0 1;0 5; FRESCOS; REFRIGERD',
 'CARNES Y DESPOJOS COMESTIBLES CARNE Y DESPOJOS COMESTIBLES; SALADOS O EN SALMUERA; SECOS O AHUMADOS;  HARIN',
 'CARNES Y DESPOJOS COMESTIBLES DESPOJOS COMESTIBLES DE ANIMALES DE LAS ESPECIES BOVINA; PORCINA; OVINA; CAPRI',
 'CARNES Y DESPOJOS COMESTIBLES LAS DEMAS CARNES Y DESPOJOS COMESTIBLES; FRESCOS; REFRIGERADOS O CONGELADOS',
 'CARNES Y DESPOJOS COMESTIBLES TOCINO SIN PARTES MAGRAS Y GRASA DE CERDO O DE AVE SIN FUNDIR NI EXTRAER DE OT',
 'CARNES\xa0Y\xa0DESPOJOS\xa0COMESTIBLES\xa0CARNE\xa0DE\xa0ANIMALES\xa0DE\xa0LAS\xa0ESPECIES\xa0CABALLAR;\xa0ASNAL\xa0O\xa0MULAR\xa0FRESCA;\xa0REFRIGERADA',
 'CARNES\xa0Y\xa0DESPOJOS\xa0COMESTIBLES\xa0LAS\xa0DEMAS\xa0CARNES\xa0Y\xa0DESPOJOS\xa0COMESTIBLES;\xa0FRESCOS;\xa0REFRIGERADOS\xa0O\xa0CONGELADOS',
 'CARNES\xa0Y\xa0DESPOJOS\xa0COMESTIBLES\xa0TOCINO\xa0SIN\xa0PARTES\xa0MAGRAS\xa0Y\xa0GRASA\xa0DE\xa0CERDO\xa0O\xa0DE\xa0AVE\xa0SIN\xa0FUNDIR\xa0NI\xa0EXTRAER\xa0DE\xa0OT',
 'CEREALES; ALFORFON;  MIJO Y ALPISTE; LOS DEMAS CEREALES',
 'CEREALES; AVENA',
 'CEREALES; CEBADA',
 'CEREALES; CENTENO',
 'CEREALES; MAIZ',
 'CEREALES; SORGO DE GRANO',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; AGRIOS (CITRICOS) FRESCOS O SECOS',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; ALBARICOQUES(DAMASCOS; CHABACANOS); CEREZAS; MELOCOTONES(DURAZNOS); (INCLUIDO',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; BANANAS O PLATANOS FRESCOS O SECOS',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; COCOS; NUECES DEL BRAZIL Y NUECES DE MARA?ON;  FRESCOS O SECOS;  INCLUSO CON C',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; CORTEZAS DE AGRIOS(CITRICOS); MELONES; SANDIAS FRESCAS;  CONGELADAS; SECAS O P',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; DATILES; HIGOS; PI?AS TROPICALES8ANANAS); AGUACATES(PALTAS); GUAYABAS; MANGOS',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; FRUTAS Y OTROS FRUTOS;  SIN COCER O COCIDOS EN AGUA O VAPOR; CONGELADOS; INCLU',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; FRUTAS Y OTROS FRUTOS; CONSERVADOS PROVISIONALMENTE CON GAS SULFUROSO O CON AG',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; FRUTOS Y OTROS FRUTOS SECOS;  EXCEPTO LOS DE LAS PARTIDAS 0 8;0 1 A 0 8;0 6',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; LAS DEMAS FRUTAS U OTROS FRUTOS FRESCOS',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; LOS DEMAS FRUTOS DE CASCARA FRESCOS O SECOS; INCLUSO SIN CASCARA O MONDADOS',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; MANZANAS; PERAS;  Y MENBRILLOS FRESCOS',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; MELONES; SANDIAS Y PAPAYAS FRESCOS',
 'FRUTOS COMESTIBLES; CORTEZAS DE AGRIOS O DE MELONES; UVAS FRESCAS O SECAS; INCLUIDAS LAS PASAS',
 'FRUTOS\xa0COMESTIBLES;\xa0CORTEZAS\xa0DE\xa0AGRIOS\xa0O\xa0DE\xa0MELONES;\xa0CORTEZAS\xa0DE\xa0AGRIOS(CITRICOS);\xa0MELONES;\xa0SANDIAS\xa0FRESCAS;\xa0\xa0CONGELADAS;\xa0SECAS\xa0O\xa0P',
 'FRUTOS\xa0COMESTIBLES;\xa0CORTEZAS\xa0DE\xa0AGRIOS\xa0O\xa0DE\xa0MELONES;\xa0DATILES;\xa0HIGOS;\xa0PI?AS\xa0TROPICALES8ANANAS);\xa0AGUACATES(PALTAS);\xa0GUAYABAS;\xa0MANGOS',
 'GOMAS; RESINAS Y DEMÁS JUGOS Y EXTRACTOS VEGETALES; GOMA LACA;  GOMAS; RESINAS; GOMORRESINAS Y OLEORRESINAS (BALSAMOS) NATURALES',
 'GOMAS; RESINAS Y DEMÁS JUGOS Y EXTRACTOS VEGETALES; JUGOS Y EXTRACTOS VEGETALES; MATERIAS PECTICAS;  PECTINATOS Y PECTATOS; AGAR-A',
 'GOMAS; RESINAS Y DEMÁS JUGOS Y EXTRACTOS VEGETALES; JUGOS Y EXTRACTOS VEGETALES; MATERIAS PECTICAS;  PECTINATOS Y PECTATOS; AGAR-APRODUCTOS DE MOLINERÍA; MALTA; ALMIDÓN Y FÉCULA; INULINA; GLUTEN DE TRIGO; GLUTEN DE TRIGO;  INCLUSO SECO',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; CERAS VEGETALES (EXCEPTO LOS TRIGLICERIDOS);  CERA DE ABEJAS O DE OTROS INSECT',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; ESTEARINA SOLAR;  ACEITE DE MANTECA DE CERDO;  OLEOESTEARINA;  OLEOMARGARINA',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; GRASAS DE ANIMALES DE LAS ESPECIES BOVINA;  OVINA O CAPRINA;  EXCEPTO LAS DE L',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; LAS DEMAS GRASAS Y ACEITES VEGETALES FIJOS (INCLUIDO EL ACEITE DE JOJOBA);  Y',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; LOS DEMAS ACEITES OBTENIDOS EXCLUSIVAMENTE DE LA ACEITUNA;  Y SUS FRACCIONES',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; MARGARINA; MEZCLAS O PREPARACIONES ALIMENTICIAS DE GRASAS O DE ACEITES;  ANIMA',
 'GRASAS\xa0Y\xa0ACEITES\xa0ANIMALES\xa0O\xa0VEGETALES;\xa0PRODUCTOS\xa0DE\xa0SU\xa0DESDOBLAMIENTO;\xa0DEGRAS;\xa0RESIDUOS\xa0DEL\xa0TRATAMIENTO\xa0DE\xa0LAS\xa0GRASAS\xa0O\xa0DE\xa0LAS\xa0CERAS\xa0ANIMALES\xa0O\xa0VEGETAL',
 'GRASAS\xa0Y\xa0ACEITES\xa0ANIMALES\xa0O\xa0VEGETALES;\xa0PRODUCTOS\xa0DE\xa0SU\xa0DESDOBLAMIENTO;\xa0GLICEROL\xa0EN\xa0BRUTO;\xa0AGUAS\xa0Y\xa0LEJIAS\xa0GLICERINOSAS',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; ACEITE DE CACAHUETE (MANI) Y SUS FRACCIONES;  INCLUSO REFINADO;  PERO SIN MODI',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; ACEITE DE NABO(NABABINA); COLZA O MOSTAZA Y SUS FRACCIONES; INCLUSO REFINADOS',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; ACEITE DE OLIVA Y SUS FRACCIONES;  INCLUSO REFINADO;  PERO SIN MODIFICAR QUIMI',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; ACEITE DE PALMA Y SUS FRACCIONES;  INCLUSO REFINADO;  PERO SIN MODIFICAR QUIMI',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; ACEITE DE SOJA(SOYA) Y SUS FRACCIONES;  INCLUSO REFINADO;  PERO SIN MODIFICAR',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; ACEITES DE COCO (COPRA); DE ALMENDRA;  DE PALMA O BABASU; SUS FRACCIONES;  INC',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; ACEITES DE GIRASOL;  DE CARTAMO O DE ALGODON;  Y SUS FRACCIONES;  INCLUSO REFI',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; DEGRAS; RESIDUOS DEL TRATAMIENTO DE LAS GRASAS O DE LAS CERAS ANIMALES O VEGETAL',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; ESTEARINA SOLAR;  ACEITE DE MANTECA DE CERDO;  OLEOESTEARINA;  OLEOMARGARINA',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; GLICEROL EN BRUTO; AGUAS Y LEJIAS GLICERINOSAS',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; GRASA DE CERDO (INCLUIDA LA MANTECA DE CERDO); Y GRASA DE AVE;  EXCEPTO LAS D',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; GRASA DE LANA Y SUSTANCIAS GRASAS DERIVADAS;  INCLUIDA LA LANOLINA',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; GRASAS Y ACEITES;  ANIMALES O VEGETALES;  Y SUS FRACCIONES;  COCIDOS;  OXIDADO',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; GRASAS Y ACEITES;  ANIMALES O VEGETALES;  Y SUS FRACCIONES;  PARCIAL O TOTALME',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; GRASAS Y ACEITES;  DE PESCADO O DE MAMIFEROS MARINOS;  Y SUS FRACCIONES;  INCL',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; LAS DEMAS GRASAS Y ACEITES ANIMALES;  Y SUS FRACCIONES;  INCLUSO REFINADOS;  P',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; LAS DEMAS GRASAS Y ACEITES VEGETALES FIJOS (INCLUIDO EL ACEITE DE JOJOBA);  Y',
 'GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCTOS DE SU DESDOBLAMIENTO; LOS DEMAS ACEITES OBTENIDOS EXCLUSIVAMENTE DE LA ACEITUNA;  Y SUS FRACCIONESGOMAS; RESINAS Y DEMÁS JUGOS Y EXTRACTOS VEGETALES; JUGOS Y EXTRACTOS VEGETALES; MATERIAS PECTICAS;  PECTINATOS Y PECTATOS; AGAR-A',
 'GRASAS\xa0Y\xa0ACEITES\xa0ANIMALES\xa0O\xa0VEGETALES;\xa0PRODUCTOS\xa0DE\xa0SU\xa0DESDOBLAMIENTO;\xa0GRASA\xa0DE\xa0CERDO\xa0(INCLUIDA\xa0LA\xa0MANTECA\xa0DE\xa0CERDO);\xa0Y\xa0GRASA\xa0DE\xa0AVE;\xa0\xa0EXCEPTO\xa0LAS\xa0D',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; CEBOLLAS;  CHALOTES; AJOS; PUERROS Y DEMAS HORTALIZAS(INCLUSO SILVESTRES) ALIA',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; COLES INCLUIDOS LOS REPOLLOS; COLIFLORES; COLES RIZADAS; COLINABOS Y PRODUCTO',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; HORTALIZAS (INCLUOS SILVESTRES) CONSERVADAS PROVISIONALEMENTE(CON GAS SULFFURO',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; HORTALIZAS (INCLUSO SILVESTRES);  AUNQUE ESTEN COCIDAS EN AGUA O VAPOR; CONGEL',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; HORTALIZAS (INCLUSO SILVESTRES);  DE VAINA SECAS DESVAINADAS AUNQUE ESTEN MOND',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; HORTALIZAS(INCLUSO SILVESTRES) DE VAINA;  AUNQUE ESTEN DESVAINADAS;  FRESCAS',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; HORTALIZAS(INCLUSO SILVESTRES) SECAS; BIEN CORTADS EN TROZOS O EN RODAJAS O BI',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; LAS DEMAS HORTALIZAS (INCLUSO SILVESTRES) FRESCAS O REFRIGERADAS',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; LECHUGA(LACTUCA SATIVA) Y ACHICORIAS; COMPRENDIDAS LA ESCAROLA Y LA ENDIBIA FR',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; PATATAS (PAPAS) FRESCAS O REFRIGERADAS',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; PEPINOS Y PEPINILLOS FRESCOS O REFRIGERADOS',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; RAICES DE MANDIOCA (YUCA); ARRURRUZ O SALEP; AGUATURMAS(PATACAS); BATATAS (BON',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; TOMATES FRESCOS O REFRIGERADOS',
 'LEGUMBRES Y HORTALIZAS; PLANTAS; RAÍCES Y TUBÉRCULOS ALIMENTICIOS; ZANAHORIAS; NABOS; REMOLACHAS PARA ENSALADAS; SALDIFIES; APIONABOS; RABANOS Y',
 'LEGUMBRES\xa0Y\xa0HORTALIZAS;\xa0PLANTAS;\xa0RAÍCES\xa0Y\xa0TUBÉRCULOS\xa0ALIMENTICIOS;\xa0COLES\xa0INCLUIDOS\xa0LOS\xa0REPOLLOS;\xa0COLIFLORES;\xa0COLES\xa0RIZADAS;\xa0COLINABOS\xa0Y\xa0PRODUCTO',
 'LEGUMBRES\xa0Y\xa0HORTALIZAS;\xa0PLANTAS;\xa0RAÍCES\xa0Y\xa0TUBÉRCULOS\xa0ALIMENTICIOS;\xa0HORTALIZAS(INCLUSO\xa0SILVESTRES)\xa0DE\xa0VAINA;\xa0\xa0AUNQUE\xa0ESTEN\xa0DESVAINADAS;\xa0\xa0FRESCAS',
 'LEGUMBRES\xa0Y\xa0HORTALIZAS;\xa0PLANTAS;\xa0RAÍCES\xa0Y\xa0TUBÉRCULOS\xa0ALIMENTICIOS;\xa0HORTALIZAS\xa0(INCLUOS\xa0SILVESTRES)\xa0CONSERVADAS\xa0PROVISIONALEMENTE(CON\xa0GAS\xa0SULFFURO',
 'LEGUMBRES\xa0Y\xa0HORTALIZAS;\xa0PLANTAS;\xa0RAÍCES\xa0Y\xa0TUBÉRCULOS\xa0ALIMENTICIOS;\xa0ZANAHORIAS;\xa0NABOS;\xa0REMOLACHAS\xa0PARA\xa0ENSALADAS;\xa0SALDIFIES;\xa0APIONABOS;\xa0RABANOS\xa0Y',
 'LOS DEMÁS PRODUCTOS DE ORIGEN ANIMAL NO EXPRESADOS NI COMPRENDIDOS AMBAR GRIS; CASTOREO; ANGALIA Y ALMIZCLE; CANTARIDAS; BILIS; INCLUSO DESECADA',
 'LOS DEMÁS PRODUCTOS DE ORIGEN ANIMAL NO EXPRESADOS NI COMPRENDIDOS CERDAS DE CERDO O DE JABALI; PELO DE TEJON Y DEMAS PELOS DE CEPILLERIA; DESPER',
 'LOS DEMÁS PRODUCTOS DE ORIGEN ANIMAL NO EXPRESADOS NI COMPRENDIDOS CORAL Y MATERIAS SIMILARES EN BRUTO O SIMPLEMENTE PREPARADAS; PERO SIN OTRO TR',
 'LOS DEMÁS PRODUCTOS DE ORIGEN ANIMAL NO EXPRESADOS NI COMPRENDIDOS TRIPAS; VEJIGAS Y ESTOMAGOS DE ANIMALES; EXCEPTO LOS DE PESCADO; ENTEROS O EN',
 'MATERIAS ALBUMINOIDEAS; PRODUCTOS A BASE DE ALMIDÓN O DE FÉCULA ALBUMINAS (INCLUIDOS LOS CONCENTRADOS DE VARIAS PROTEINAS DE LACTOSUERO;  CON',
 'MATERIAS ALBUMINOIDEAS; PRODUCTOS A BASE DE ALMIDÓN O DE FÉCULA CASEINA;  CASEINATOS Y DEMAS DERIVADOS DE LA CASEINA; COLAS DE CASEINA',
 'MATERIAS ALBUMINOIDEAS; PRODUCTOS A BASE DE ALMIDÓN O DE FÉCULA DEXTRINA Y DEMAS ALMIDONES Y FECULAS MODIFICADOS (POR EJEMPLO ALMIDONES Y FECU',
 'MATERIAS ALBUMINOIDEAS; PRODUCTOS A BASE DE ALMIDÓN O DE FÉCULA ENZIMAS; PREPARACIONES ENZIMATICAS NO EXPRESADAS NI COMPRENDIDAS EN OTRAS PARTID',
 'MATERIAS ALBUMINOIDEAS; PRODUCTOS A BASE DE ALMIDÓN O DE FÉCULA COLAS Y DEMAS ADHESIVOS PREPARADOS;  NO EXPRESADOS NI COMPRENDIDOS EN OTRAS PA',
 'MATERIAS ALBUMINOIDEAS; PRODUCTOS A BASE DE ALMIDÓN O DE FÉCULA GELATINAS (AUNQUE SE PRESENTEN EN HOJAS CUADRADAS O RECTANGULARES;  INCLUSO TR',
 'MATERIAS TRENZABLES Y DEMÁS PRODUCTOS DE ORIGEN VEGETAL; NO EXPRESADOS MATERIAS VEGETALES DE LAS ESPECIES UTILIZADAS PRINCIPALMENTE EN LA FABRICACIO',
 'MATERIAS ALBUMINOIDEAS; PRODUCTOS A BASE DE ALMIDÓN O DE FÉCULA PEPTONAS Y SUS DERIVADOS; LAS DEMAS MATERIAS PROTEICAS Y SUS DERIVADOS;  NO EX',
 'MATERIAS TRENZABLES Y DEMÁS PRODUCTOS DE ORIGEN VEGETAL; NO EXPRESADOS MATERIAS VEGETALES DE LAS ESPECIES UTILIZADAS PRINCIPALMENTE EN CESTERIA O ESP',
 'MATERIAS TRENZABLES Y DEMÁS PRODUCTOS DE ORIGEN VEGETAL; NO EXPRESADOS MATERIAS VEGETALES DE LAS ESPECIES UTILIZADAS PRINCIPALMENTE PARA RELLENO (PO',
 'MATERIAS TRENZABLES Y DEMÁS PRODUCTOS DE ORIGEN VEGETAL; NO EXPRESADOS PRODUCTOS VEGETALES NO EXPRESADOS NI COMPRENDIDOS EN OTRAS PARTIDAS',
 'PESCADOS Y CRUSTÁCEOS; MOLUSCOS Y OTROS INVERTEBRADOS ACUÁTICOS CRUSTACEOS; INCLUSO PELADOS; VIVOS; FRESCOS; REFRIGERADOS; CONGELADOS; SECOS;',
 'PESCADOS Y CRUSTÁCEOS; MOLUSCOS Y OTROS INVERTEBRADOS ACUÁTICOS FILETES Y DEMAS CARNE DE PESCADO (INCLUSO PICADA); FRESCOS;  REFRIGERADOS O CO',
 'PESCADOS Y CRUSTÁCEOS; MOLUSCOS Y OTROS INVERTEBRADOS ACUÁTICOS MOLUSCOS; INCLUSO SEPARADOS DE SUS VALVAS; VIVOS; FRESCOS; REFRIGERADOS; CONGE',
 'PESCADOS Y CRUSTÁCEOS; MOLUSCOS Y OTROS INVERTEBRADOS ACUÁTICOS PECES O PESCADOS VIVOS',
 'PESCADOS Y CRUSTÁCEOS; MOLUSCOS Y OTROS INVERTEBRADOS ACUÁTICOS PESCADO CONGELADO;  EXCEPTO LOS FILETES Y DEMAS CARNE DE PESCADO DE LA PARTID',
 'PESCADOS Y CRUSTÁCEOS; MOLUSCOS Y OTROS INVERTEBRADOS ACUÁTICOS PESCADO FRESCO O REFRIGERADO; EXCEPTO LOS FILETES Y DEMAS CARNE DE PESCADO DE',
 'PESCADOS Y CRUSTÁCEOS; MOLUSCOS Y OTROS INVERTEBRADOS ACUÁTICOS PESCADO SECO; SALADO O EN SALMUERA; PESCADO AHUMADO INCLUSO COCIDO ANTES O DUR',
 'PESCADOS\xa0Y\xa0CRUSTÁCEOS;\xa0MOLUSCOS\xa0Y\xa0OTROS\xa0INVERTEBRADOS\xa0ACUÁTICOS\xa0PECES\xa0O\xa0PESCADOS\xa0VIVOS',
 'PLANTAS VIVAS Y PRODUCTOS DE LA FLORICULTURA; BULBOS; CEBOLLAS; TUBERCULOS; RAICES Y BULBOS TUBEROSOS; TURIONES O RIZOMAS;',
 'PLANTAS VIVAS Y PRODUCTOS DE LA FLORICULTURA; FLORES Y CAPULLOS; CORTADOS PARA RAMOS O ADORNOS; FRESCOS; SECOS; BLANQUEADOS',
 'PLANTAS VIVAS Y PRODUCTOS DE LA FLORICULTURA; FOLLAJES; HAJAS; RAMAS Y DEMAS PARTES DE PLANTAS SIN FLORES NI CAPULLOS;  Y HI',
 'PREPARACIONES A BASE DE CEREALES; HARINA; ALMIDÓN; FÉCULA O LECHE; EXTRACTO DE MALTA; PREPARACIONES ALIMENTICIASDE HARINA;  SEMOLA;  ALMIDON;  FE',
 'PREPARACIONES A BASE DE CEREALES; HARINA; ALMIDÓN; FÉCULA O LECHE; PASTAS ALIMENTICIAS;  INCLUSO COCIDAS O RELLENAS (DE CARNE U OTRAS SUSTANCIAS',
 'PREPARACIONES A BASE DE CEREALES; HARINA; ALMIDÓN; FÉCULA O LECHE; PRODUCTOS A BASE DE CEREALES OBTENIDOS PORINSUFLADO O TOSTADO ( COPOS DE MAIZ',
 'PREPARACIONES A BASE DE CEREALES; HARINA; ALMIDÓN; FÉCULA O LECHE; PRODUCTOS DE PANADERIA;  PASTELERIA O GALLETERIA;  INCLUSO CON CACAO; HOSTIAS',
 'PREPARACIONES A BASE DE CEREALES; HARINA; ALMIDÓN; FÉCULA O LECHE; TAPIOCA Y SUS SUCEDANEOS CON FECULA;  EN COPOS;  GRUMOS;  GRANOS PERLADOS;  CE',
 'PREPARACIONES ALIMENTICIAS DIVERSAS; EXTRACTOS;  ESENCIAS Y CONCENTRADOS DE CAFE;  TE O YERBA MATE Y PREPARACIONES',
 'PREPARACIONES ALIMENTICIAS DIVERSAS; HELADOS Y PRODUCTOS SIMILARES INCLUSO CON CACAO',
 'PREPARACIONES ALIMENTICIAS DIVERSAS; LEVADURAS (VIVAS O MUERTAS); LOS DEMAS MICROORGANISMOS MONOCELULARES MUERTOS (CO',
 'PREPARACIONES ALIMENTICIAS DIVERSAS; PREPARACIONES ALIMENTICIAS NO EXPRESADAS NI COMPRENDIDAS EN OTRAS PARTIDAS',
 'PREPARACIONES ALIMENTICIAS DIVERSAS; PREPARACIONES PARA SALSAS Y SALSAS PREPARADAS; CONDIMENTOS Y SAZONADORES;  COM',
 'PREPARACIONES ALIMENTICIAS DIVERSAS; PREPARACIONES PARA SOPAS;  POTAJES O CALDOS; SOPAS',
 'PREPARACIONES DE CARNE; DE PESCADO O DE CRUSTÁCEOS; DE MOLUSCOS O DE CRUSTACEOS;  MOLUSCOS Y DEMAS INVERTEBRADOS ACUATICOS;  PREPARADOS O CONSERVAD',
 'PREPARACIONES DE CARNE; DE PESCADO O DE CRUSTÁCEOS; DE MOLUSCOS O DE EMBUTIDOS Y PRODUCTOS SIMILARES;  DE CARNE;  DE DESPOJOS O DE SANGRE; PREPARAC',
 'PREPARACIONES DE CARNE; DE PESCADO O DE CRUSTÁCEOS; DE MOLUSCOS O DE EXTRACTOS Y JUGOS DE CARNE;  DE PESCADO O DE CRUSTACEOS;  DE MOLUSCOS O DE OTR',
 'PREPARACIONES DE CARNE; DE PESCADO O DE CRUSTÁCEOS; DE MOLUSCOS O DE LAS DEMAS PREPARACIONES Y CONSERVAS DE CARNE;  DE DESPOJOS O DE SANGRE',
 'PREPARACIONES DE CARNE; DE PESCADO O DE CRUSTÁCEOS; DE MOLUSCOS O DE PREPARACIONES Y CONSERVAS DE PESCADO; CAVIAR Y SUS SUCEDANEOS PREPARADOS CON HUE',
 'PREPARACIONES DE LEGUMBRES U HORTALIZAS; DE FRUTOS O DE OTRAS PARTES DE COMPOTAS;  JALEAS Y MERMELADAS;  PURES Y PASTAS DE FRUTOS;  OBTENIDOS POR COCC',
 'PREPARACIONES DE LEGUMBRES U HORTALIZAS; DE FRUTOS O DE OTRAS PARTES DE FRUTOS Y DEMAS PARTES COMESTIBLES DE PLANTAS;  PREPARADOS O CONSERVADOS DE OTR',
 'PREPARACIONES DE LEGUMBRES U HORTALIZAS; DE FRUTOS O DE OTRAS PARTES DE JUGOS DE FRUTAS (INCLUIDO EL MOSTO DE UVA) O DE LEGUMBRES U HORTALIZAS;  SIN F',
 'PREPARACIONES DE LEGUMBRES U HORTALIZAS; DE FRUTOS O DE OTRAS PARTES DE LAS DEMAS LEGUMBRES Y HORTALIZAS;  PREPARADAS O CONSERVADAS (EXCEPTO EN VINAGR',
 'PREPARACIONES DE LEGUMBRES U HORTALIZAS; DE FRUTOS O DE OTRAS PARTES DE LEGUMBRES Y HORTALIZAS;  FRUTAS Y OTROS FRUTOS Y SUS CORTEZAS Y DEMAS PARTES D',
 'PREPARACIONES DE LEGUMBRES U HORTALIZAS; DE FRUTOS O DE OTRAS PARTES DE LEGUMBRES;  HORTALIZAS;  FRUTOS Y DEMAS PARTES COMESTIBLES DE PLANTAS;  PREPAR',
 'PREPARACIONES DE LEGUMBRES U HORTALIZAS; DE FRUTOS O DE OTRAS PARTES DE SETAS Y DEMAS HONGOS;  Y TRUFAS PREPARADOS O CONSERVADOS(EXCEPTO EN VINAGRE O',
 'PREPARACIONES DE LEGUMBRES U HORTALIZAS; DE FRUTOS O DE OTRAS PARTES DE TOMATES PREPARADOS O CONSERVADOS (EXCEPTO EN VINAGRE O EN ACIDO ACETICO)',
 'PREPARACIONES\xa0ALIMENTICIAS\xa0DIVERSAS;\xa0PREPARACIONES\xa0PARA\xa0SOPAS;\xa0\xa0POTAJES\xa0O\xa0CALDOS;\xa0SOPAS',
 'PREPARACIONES\xa0A\xa0BASE\xa0DE\xa0CEREALES;\xa0HARINA;\xa0ALMIDÓN;\xa0FÉCULA\xa0O\xa0LECHE;\xa0TAPIOCA\xa0Y\xa0SUS\xa0SUCEDANEOS\xa0CON\xa0FECULA;\xa0\xa0EN\xa0COPOS;\xa0\xa0GRUMOS;\xa0\xa0GRANOS\xa0PERLADOS;\xa0\xa0CE',
 'PREPARACIONES\xa0DE\xa0LEGUMBRES\xa0U\xa0HORTALIZAS;\xa0DE\xa0FRUTOS\xa0O\xa0DE\xa0OTRAS\xa0PARTES\xa0DE\xa0JUGOS\xa0DE\xa0FRUTAS\xa0(INCLUIDO\xa0EL\xa0MOSTO\xa0DE\xa0UVA)\xa0O\xa0DE\xa0LEGUMBRES\xa0U\xa0HORTALIZAS;\xa0\xa0SIN\xa0F',
 'PREPARACIONES\xa0DE\xa0LEGUMBRES\xa0U\xa0HORTALIZAS;\xa0DE\xa0FRUTOS\xa0O\xa0DE\xa0OTRAS\xa0PARTES\xa0DE\xa0SETAS\xa0Y\xa0DEMAS\xa0HONGOS;\xa0\xa0Y\xa0TRUFAS\xa0PREPARADOS\xa0O\xa0CONSERVADOS(EXCEPTO\xa0EN\xa0VINAGRE\xa0O',
 'PRODUCTOS DE MOLINERÍA; MALTA; ALMIDÓN Y FÉCULA; INULINA; GLUTEN DE TRIGO; ALMIDON Y FECULA; INULINA',
 'PRODUCTOS DE MOLINERÍA; MALTA; ALMIDÓN Y FÉCULA; INULINA; GLUTEN DE TRIGO; GLUTEN DE TRIGO;  INCLUSO SECO',
 'PRODUCTOS DE MOLINERÍA; MALTA; ALMIDÓN Y FÉCULA; INULINA; GLUTEN DE TRIGO; GRA?ONES;  SEMOLA Y <PELLETS>;  DE CEREALES',
 'PRODUCTOS DE MOLINERÍA; MALTA; ALMIDÓN Y FÉCULA; INULINA; GLUTEN DE TRIGO; GRANOS DE CEREALES TRABAJADOS DE OTRA FORMA (POR EJEMPLO MONDADOS;  APLASTADOS',
 'PRODUCTOS DE MOLINERÍA; MALTA; ALMIDÓN Y FÉCULA; INULINA; GLUTEN DE TRIGO; HARINA DE CEREALES;  EXCEPTO DE TRIGO O DE MORCAJO O TRANQUILLON',
 'PRODUCTOS DE MOLINERÍA; MALTA; ALMIDÓN Y FÉCULA; INULINA; GLUTEN DE TRIGO; HARINA DE TRIGO O MORCAJO',
 'PRODUCTOS DE MOLINERÍA; MALTA; ALMIDÓN Y FÉCULA; INULINA; GLUTEN DE TRIGO; HARINA;  SEMOLA Y POLVO DE LAS LEGUMBRES SECAS DE LA PARTIDA 0713;  DE SAGU O',
 'PRODUCTOS DE MOLINERÍA; MALTA; ALMIDÓN Y FÉCULA; INULINA; GLUTEN DE TRIGO; HARINA;  SEMOLA;  POLVO;  COPOS;  GRANULOS Y <PELLETS> DE PATATAS (PAPAS)',
 'PRODUCTOS DE MOLINERÍA; MALTA; ALMIDÓN Y FÉCULA; INULINA; GLUTEN DE TRIGO; MALTA (DE CEBADA U OTROS CEREALES) INCLUSO TOSTADA',
 'RESIDUOS Y DESPERDICIOS DE LAS INDUSTRIAS ALIMENTARIAS; ALIMENTOS MATERIAS VEGETALES;  DESPERDICIOS;  RESIDUOS Y SUBPRODUCTOS VEGETALES;  INCLUS',
 'RESIDUOS Y DESPERDICIOS DE LAS INDUSTRIAS ALIMENTARIAS; ALIMENTOS RESIDUOS DE LA INDUSTRIA DEL ALMIDON Y RESIDUOS SIMILARES;  PULPA DE REMOLACHA',
 'RESIDUOS Y DESPERDICIOS DE LAS INDUSTRIAS ALIMENTARIAS; ALIMENTOS SALVADOS;  MOYUELOS Y DEMAS RESIDUOS DEL CERNIDO;  DE LA MOLIENDA O DE OTROS T',
 'RESIDUOS Y DESPERDICIOS DE LAS INDUSTRIAS ALIMENTARIAS; ALIMENTOS TORTAS Y DEMAS RESIDUOS SOLIDOS DE LA EXTRACCION DE GRASAS O ACEITES VEGETALES',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS ALGARROBAS;  ALGAS;  REMOLACHA AZUCARERA Y CA?A DE AZUCAR;  FRESCAS;  REFRIGER',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS CACAHUATES (MANI) SIN TOSTAR NI COCER DE OTRO MODO; INCLUSO SIN CASCARA O QUEB',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS CONOS DE LUPULO FRESCOS O SECOS;  INCLUSO QUEBRANTADOS;  MOLIDOS O EN <PELLETS',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS COPRA',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS HABAS(POROTOS; FRIJOLES) DE SOJA (SOYA) INCLUSO QUEBRANTADAS',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS HARINA DE SEMILLAS O DE FRUTOS OLEAGINOSOS;  EXCEPTO LA HARINA DE MOSTAZA',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS LAS DEMAS SEMILLAS Y FRUTOS OLEAGINOSOS;  INCLUSO QUEBRANTADOS',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS NABOS FORRAJEROS;  REMOLACHAS FORRAJERAS;  RAICES FORRAJERAS;  HENO;  ALFALFA',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS PAJA Y CASCABILLO DE CEREALES;  EN BRUTO;  INCLUSO PICADOS;  MOLIDOS;  PRENSAD',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS PLANTAS;  PARTES DE PLANTAS;  SEMILLAS Y FRUTOS DE LAS ESPECIES UTILIZADAS PRI',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS SEMILLA DE GIRASOL;  INCLUSO QUEBRANTADA',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS SEMILLA DE LINO;  INCLUSO QUEBRANTADA',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS SEMILLA DE NABO O DE COLZA;  INCLUSO QUEBRANTADA',
 'SEMILLAS Y FRUTOS OLEAGINOSOS; SEMILLAS Y FRUTOS DIVERSOS; PLANTAS SEMILLAS;  FRUTOS Y ESPORAS;  PARA SIEMBRA',
 'SEMILLAS\xa0Y\xa0FRUTOS\xa0OLEAGINOSOS;\xa0SEMILLAS\xa0Y\xa0FRUTOS\xa0DIVERSOS;\xa0PLANTAS\xa0NABOS\xa0FORRAJEROS;\xa0\xa0REMOLACHAS\xa0FORRAJERAS;\xa0\xa0RAICES\xa0FORRAJERAS;\xa0\xa0HENO;\xa0\xa0ALFALFA',
 'SEMILLAS\xa0Y\xa0FRUTOS\xa0OLEAGINOSOS;\xa0SEMILLAS\xa0Y\xa0FRUTOS\xa0DIVERSOS;\xa0PLANTAS\xa0PLANTAS;\xa0\xa0PARTES\xa0DE\xa0PLANTAS;\xa0\xa0SEMILLAS\xa0Y\xa0FRUTOS\xa0DE\xa0LAS\xa0ESPECIES\xa0UTILIZADAS\xa0PRI',
 'SEMILLAS\xa0Y\xa0FRUTOS\xa0OLEAGINOSOS;\xa0SEMILLAS\xa0Y\xa0FRUTOS\xa0DIVERSOS;\xa0PLANTAS\xa0SEMILLAS;\xa0\xa0FRUTOS\xa0Y\xa0ESPORAS;\xa0\xa0PARA\xa0SIEMBRA',
  'CEREALES; ARROZ',
 'CEREALES; TRIGO Y MORCAJO O TRANQUILLON',
 'MATERIAS ALBUMINOIDEAS; PRODUCTOS A BASE DE ALMIDÓN O DE FÉCULA COLAS Y DEMAS ADHESIVOS PREPARADOS;  NO EXPRESADOS NI COMPRENDIDOS EN OTRAS PA',
 'MATERIAS TRENZABLES Y DEMÁS PRODUCTOS DE ORIGEN VEGETAL; NO EXPRESADOS PRODUCTOS VEGETALES NO EXPRESADOS NI COMPRENDIDOS EN OTRAS PARTIDAS',
]

alimentos = sorted(set(alimentos))

In [9]:
# Bebidas
bebidas = ['BEBIDAS; LÍQUIDOS ALCOHÓLICOS Y VINAGRE; AGUA;  INCLUIDA EL AGUA MINERAL NATURAL O ARTIFICIAL Y LA GASIFICADA;  SIN AZU',
 'BEBIDAS; LÍQUIDOS ALCOHÓLICOS Y VINAGRE; AGUA;  INCLUIDA EL AGUA MINERAL Y LA GASIFICADA;  AZUCARADA;  EDULCORADA DE OT',
 'BEBIDAS; LÍQUIDOS ALCOHÓLICOS Y VINAGRE; ALCOHOL ETILICO SIN DESNATURALIZAR CON UN GRADO ALCOHOLICO VOLUMETRICO INFERIO',
 'BEBIDAS; LÍQUIDOS ALCOHÓLICOS Y VINAGRE; ALCOHOL ETILICO SIN DESNATURALIZAR CON UN GRADO ALCOHOLICO VOLUMETRICO SUPERIO',
 'BEBIDAS; LÍQUIDOS ALCOHÓLICOS Y VINAGRE; CERVEZA DE MALTA',
 'BEBIDAS; LÍQUIDOS ALCOHÓLICOS Y VINAGRE; LAS DEMAS BEBIDAS FERMENTADAS (POR EJEMPLO SIDRA;  PERADA O AGUAMIEL); MEZCLA',
 'BEBIDAS; LÍQUIDOS ALCOHÓLICOS Y VINAGRE; VERMUT Y DEMAS VINOS DE UVAS FRESCAS PREPARADOS CON PLANTAS O SUSTANCIAS AROMATI',
 'BEBIDAS; LÍQUIDOS ALCOHÓLICOS Y VINAGRE; VINAGRE COMESTIBLE Y SUCEDANEOS COMESTIBLES DEL VINAGRE OBTENIDOS CON ACIDO ACET',
 'BEBIDAS; LÍQUIDOS ALCOHÓLICOS Y VINAGRE; VINO DE UVAS;  INCLUSO ENCABEZADO; MOSTO DE UVA;  EXCEPTO EL DE LA PARTIDA 200',
 'LECHE Y PRODUCTOS LÁCTEOS; HUEVOS DE AVE; MIEL NATURAL; PRODUCTOS HUEVOS DE AVES CON CASCARA(CASCARON) FRESCOS;  CONSERVADOS O COCIDOS',
 'LECHE Y PRODUCTOS LÁCTEOS; HUEVOS DE AVE; MIEL NATURAL; PRODUCTOS HUEVOS DE AVES SIN CASCARA (CASCARON) Y YEMAS DE HUEVOS FRESCOS; SECOS;  COCID',
 'LECHE Y PRODUCTOS LÁCTEOS; HUEVOS DE AVE; MIEL NATURAL; PRODUCTOS LACTOSUERO INCLUSO CONCENTRADO O CON ADICION DE AZUCAR U OTRO EDULCORANTE;  PR',
 'LECHE Y PRODUCTOS LÁCTEOS; HUEVOS DE AVE; MIEL NATURAL; PRODUCTOS LECHE Y NATA (CREMA) SIN CONCENTRAR; SIN ADICION DE AZUCAR NI OTRO EDUCOLORANT',
 'LECHE Y PRODUCTOS LÁCTEOS; HUEVOS DE AVE; MIEL NATURAL; PRODUCTOS LECHE Y NATAS (CREMA) CONCENTRADAS O CON ADCION DE AZUCAR U OTRO EDULCORANTE',
 'LECHE Y PRODUCTOS LÁCTEOS; HUEVOS DE AVE; MIEL NATURAL; PRODUCTOS MANTEQUILLA (MANTECA) Y DEMAS MATERIAS GRASAS DE LA LECHE<:PASTAS LACTEAS PARA U',
 'LECHE Y PRODUCTOS LÁCTEOS; HUEVOS DE AVE; MIEL NATURAL; PRODUCTOS MIEL NATURAL',
 'LECHE Y PRODUCTOS LÁCTEOS; HUEVOS DE AVE; MIEL NATURAL; PRODUCTOS PRODUCTOS COMESTIBLES DE ORIGEN ANIMAL NO EXPRESADOS NI COMPRENDIDOS EN OTRA PAR',
 'LECHE Y PRODUCTOS LÁCTEOS; HUEVOS DE AVE; MIEL NATURAL; PRODUCTOS QUESOS Y REQUESON',
 'LECHE Y PRODUCTOS LÁCTEOS; HUEVOS DE AVE; MIEL NATURAL; PRODUCTOS SUERO DE MANTEQUILLA (DE MANTECA);  LECHE Y NATA (CREMA) CUAJADAS; YOGUR; KEFI']
bebidas = sorted(set(bebidas))

In [10]:

# Construccion
construccion = [
 'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y MASILLA;  CEMENTOS DE RESINA Y OTROS MASTIQUES; PLASTES DE RELLENO UTILIZADOS',
 'PRODUCTOS CERÁMICOS; ESTATUILLAS Y DEMAS ARTICULOS PARA ADORNO;  DE CERAMICA',
 'PRODUCTOS CERÁMICOS; FREGADEROS;  LAVABOS;  PEDESTALES DE LAVABO;  BA?ERAS;  BIDES;  INODOROS;  CIS',
 'PRODUCTOS CERÁMICOS; LADRILLOS DE CONSTRUCCION;  BOVEDILLAS;  CUBREVIGAS Y ARTICULOS SIMILARES;  D',
 'PRODUCTOS CERÁMICOS; LADRILLOS;  PLACAS;  BALDOSAS Y OTRAS PIEZAS CERAMICAS DE HARINAS SILICEAS FOS',
 'PRODUCTOS CERÁMICOS; LADRILLOS;  PLACAS;  BALDOSAS Y PIEZAS CERAMICAS ANALOGAS DE CONSTRUCCION;  RE',
 'PRODUCTOS CERÁMICOS; LAS DEMAS MANUFACTURAS DE CERAMICA',
 'PRODUCTOS CERÁMICOS; LOS DEMAS ARTICULOS CERAMICOS REFRACTARIOS (POR EJEMPLO RETORTAS;  CRISOLES;',
 'PRODUCTOS CERÁMICOS; PLACAS Y BALDOSAS;  DE CERAMICA; SIN BARNIZAR NI ESMALTAR PARA PAVIMENTACION',
 'PRODUCTOS CERÁMICOS; TEJAS;  ELEMENTOS DE CHIMENEA;  CONDUCTOS DE HUMO;  ORNAMENTOS ARQUITECTONICO',
 'PRODUCTOS CERÁMICOS; TUBOS;  CANALONES Y ACCESORIOS DE TUBERIA;  DE CERAMICA',
 'PRODUCTOS CERÁMICOS; VAJILLAS Y DEMAS ARTICULOS DE USO DOMESTICO;   HIGIENE O  TOCADOR;  DE PORCELA',
 'VEHÍCULOS Y MATERIAL PARA VÍAS FÉRREAS O SIMILARES Y SUS PARTES; APARATOS AUTOMOTORES Y TRANVIAS CON MOTOR;  EXCEPTO LOS DE LA PARTIDA 8604',
 'VEHÍCULOS Y MATERIAL PARA VÍAS FÉRREAS O SIMILARES Y SUS PARTES; APARATOS COCHES DE VIAJEROS;  FURGONES DE EQUIPAJES;  COCHES CORREO Y DEMAS COCHES ESPE',
 'VEHÍCULOS Y MATERIAL PARA VÍAS FÉRREAS O SIMILARES Y SUS PARTES; APARATOS CONTENEDORES (INCLUIDOS LOS CONTENEDORES CISTERNA Y LOS CONTENEDORES DEPOSITO) E',
 'VEHÍCULOS Y MATERIAL PARA VÍAS FÉRREAS O SIMILARES Y SUS PARTES; APARATOS LAS DEMAS LOCOMOTORAS Y LOCOTRACTORES; TENDERES',
 'VEHÍCULOS Y MATERIAL PARA VÍAS FÉRREAS O SIMILARES Y SUS PARTES; APARATOS LOCOMOTORAS Y LOCOTRACTORES;  ELECTRICOS (DE ENERGIA EXTERIOR O DE ACUMULADORE',
 'VEHÍCULOS Y MATERIAL PARA VÍAS FÉRREAS O SIMILARES Y SUS PARTES; APARATOS MATERIAL FIJO DE VIAS FERREAS O SIMILARES; APARATOS MECANICOS (INCLUSO ELECTRO',
 'VEHÍCULOS Y MATERIAL PARA VÍAS FÉRREAS O SIMILARES Y SUS PARTES; APARATOS PARTES DE VEHICULOS PARA VIAS FERREAS O SIMILARES',
 'VEHÍCULOS Y MATERIAL PARA VÍAS FÉRREAS O SIMILARES Y SUS PARTES; APARATOS VEHICULOS PARA EL MANTENIMIENTO O SERVICIO DE LAS VIAS FERREAS O SIMILARES;  I',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; ADOQUINES;  BALDOSAS;  LADRILLOS;  PLACAS;  TEJAS Y DEMAS ARTICULOS;  DE VIDRI',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; AMPOLLAS Y ENVOLTURAS TUBULARES;  ABIERTAS;  Y SUS PARTES;  DE VIDRIO;  SIN GU',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; DESPERDICIOS Y DESECHOS DE VIDRIO; VIDRIO EN MASA',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; LAS DEMAS MANUFACTURAS DE VIDRIO',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; VIDRIERAS AISLANTES DE PAREDES MULTIPLES',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; VIDRIO EN BOLAS (EXCEPTO LAS MICROESFERAS DE LA PARTIDA 7018);  BARRAS;  VARIL',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; AMPOLLAS DE VIDRIO PARA TERMOS O DEMAS RECIPIENTES ISOTERMICOS AISLADOS POR VACI',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; ARTICULOS DE VIDRIO PARA LABORATORIO;  HIGIENE O FARMACIA;  INCLUSO GRADUADOS',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; BOMBONAS;  (DAMAJUANAS) BOTELLAS;  FRASCOS;  BOCALES;  TARROS;  ENVASES TUBULA',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; CRISTALES PARA RELOJES Y CRISTALES ANALOGOS;  CRISTALES PARA GAFAS ( ANTEOJOS',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; CUENTAS DE VIDRIO;  IMITACIONES DE PERLAS DE PIEDRAS PRECIOSAS O SEMIPRECIOSA',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; ESPEJOS DE VIDRIO ENMARCADOS O NO;  INCLUIDOS LOS ESPEJOS RETROVISORES',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; FIBRA DE VIDRIO (INCLUIDA LA LANA DE VIDRIO) Y MANUFACTURAS DE ESTA MATERIA (P',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; OBJETOS DE VIDRIO PARA EL SERVICIO DE MESA;  DE COCINA;  DE TOCADOR;  DE OFICI',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; VIDRIO COLADO EN PLACAS;  HOJAS O PERFILES;  INCLUSO CON CAPA ABSORBENTE;  REF',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; VIDRIO DE LAS PARTIDAS 7003;  7004 O 7005;  CURVADO;  BISELADO;  GRABADO;  TAL',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; VIDRIO DE SEGURIDAD CONSTITUIDO POR VIDRIO TEMPLADO O CONTRACHAPADO',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; VIDRIO ESTIRADO O SOPLADO;  EN HOJAS;  INCLUSO CON CAPA ABSORBENTE REFLECTANT',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; VIDRIO FLOTADO Y VIDRIO DESBASTADO O PULIDO POR UNA DE LAS DOS CARAS;  EN PLAC',
 'VIDRIO Y MANUFACTURAS DE VIDRIO; VIDRIO PARA SE?ALIZACION Y ELEMENTOS DE OPTICA DE VIDRIO (EXCEPTO LOS DE LA PA',
 'VIDRIO\xa0Y\xa0MANUFACTURAS\xa0DE\xa0VIDRIO;\xa0AMPOLLAS\xa0Y\xa0ENVOLTURAS\xa0TUBULARES;\xa0\xa0ABIERTAS;\xa0\xa0Y\xa0SUS\xa0PARTES;\xa0\xa0DE\xa0VIDRIO;\xa0\xa0SIN\xa0GU',
  'PRODUCTOS CERÁMICOS; APARATOS Y ARTICULOS;  DE CERAMICA PARA USOS QUIMICOS O DEMAS USOS TECNICOS; A',
 'PRODUCTOS CERÁMICOS; LADRILLOS;  PLACAS;  BALDOSAS Y OTRAS PIEZAS CERAMICAS DE HARINAS SILICEAS FOS',
 'PRODUCTOS CERÁMICOS; PLACAS Y BALDOSAS;  DE CERAMICA BARNIZADAS O ESMALTADAS PARA PAVIMENTACION O R',
 'PRODUCTOS CERÁMICOS; VAJILLAS Y DEMAS ARTICULOS DE USO DOMESTICO;   HIGIENE O TOCADOR;  DE CERAMICA']

construccion = sorted(set(construccion))

In [11]:
# Combustibles
combustibles = ['COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; ACEITES CRUDOS DE PETROLEO O DE MINERALES BITUMINOSOS',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; LIGNITOS;  INCLUSO AGLOMERADOS;  CON EXCLUSION DEL AZABACHE',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; ACEITES DE PETROLEO O DE MINERALES BITUMINOSOS EXCEPTO LOS ACEITES CRUDOS; PREPARACIONES NO EXPRESADAS NI COMPRENDIDAS EN OTRA ',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; ALQUITRANES DE HULLA, DE LIGNITO O DE TURBA Y DEMAS ALQUITRANES MINERALES, INCLUIDOS LOS DESHIDRATADOS O DESCABEZADOS Y LOS REC',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; ALQUITRANES Y DEMAS PRODUCTOS DE LA DESTILACION DE LOS ALQUITRANES DE HULLA DE ALTA TEMPERATURA; PRODUCTOS ANALOGOS EN LOS QUE ',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; BETUNES Y ASFALTOS NATURALES; PIZARRAS Y ARENAS BITUMINOSAS; ASFALITITAS Y ROCAS ASFALTICAS.',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; COQUE DE PETROLEO, BETUM DE PETROLEO Y DEMAS RESIDUOS DE LOS ACEITES DE PETROLEO O DE MINERALES BITUMINOSOS.',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; COQUES Y SEMICOQUES DE HULLA, DE LIGNITO O DE TURBA, INCLUSO ALGOMERADOS; CARBON DE RETORTA.',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; ENERGIA ELECTRICA',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; GAS DE HULLA, GAS DE AGUA, GAS POBRE Y GASES SIMILARES, EXCEPTO EL GAS DE PETROLEO Y DEMAS HIDROCARBUROS GASEOSOS.',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; GAS DE PETROLEO Y DEMAS HIDROCARBUROS GASEOSOS',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; HULLAS: BRIQUETAS, OVOIDES Y COMBUSTIBLES SOLIDOS SIMILARES, OBTENIDOS DE LA HULLA (CARBON)',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; MEZCLAS BITUMINOSAS A BASE DE ASFALTO O DE BETUN NATURALES;  DE BETUN DE PETRO',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; TURBA;  INCLUIDA LA UTILIZADA PARA CAMA DE ANIMALES Y LA AGLOMERADA',
 'COMBUSTIBLES MINERALES; ACEITES MINERALES Y PRODUCTOS DE SU DESTILACIÓN; VASELINA; PARAFINA CERA DE PETROLEO MICROCRITALINA, SLACK WAX, OZOQUERITA, CERA DE LIGNITO, CERA DE TURBA Y DEMAS CERAS MINERAL',
 'EXPLOSIVOS 0403 - 1.4G - BENGALAS AÉREAS.',
 'GASES 1001 - 2.1 - ACETILENO DISUELTO',
 'GASES 1013 - 2.2 - DIÓXIDO DE CARBONO',
 'GASES 1058 - 2.2 - MEZCLAS DE GASES LICUADOS ininflamables con nitrógeno; dióxido de c',
 'GASES 1072 - 2.2 - OXÍGENO COMPRIMIDO',
 'GASES 1075 - 2.1 - GASES DE PETRÓLEO; LICUADOS',
 'GASES 1956 - 2.2 - GAS COMPRIMIDO; N.E.P.',
 'GASES 1964 - 2.1 - MEZCLA DE HIDROCARBUROS GASEOSOS COMPRIMIDOS; N.E.P.',
 'GASES 1978 - 2.1 - PROPANO',
 'GASES 2195 - 2.3 - HEXAFLUORURO DE TELURIO',
 'GASES 2417 - 2.3 - FLUORURO DE CARBONILO',
 'GASES 2418 - 2.3 - TETRAFLUORURO DE AZUFRE',
 'GASES 2419 - 2.1 - BROMOTRIFLUOROETILENO',
 'LÍQUIDOS INFLAMABLES 1202 - 3 - GASÓLEO o COMBUSTIBLE PARA MOTORES DIESEL o ACEITE MINERAL PARA CALDE',
 'LÍQUIDOS INFLAMABLES 1203 - 3 - COMBUSTIBLE PARA MOTORES o GASOLINA',
 'LÍQUIDOS INFLAMABLES 1212 - 3 - ISOBUTANOL (ALCOHOL ISOBUTÍLICO)',
 'LÍQUIDOS INFLAMABLES 1268 - 3 - DESTILADOS DE PETRÓLEO; N.E.P. o PRODUCTOS DE PETRÓLEO; N.E.P.',
 'LÍQUIDOS INFLAMABLES 1863 - 3 - COMBUSTIBLE PARA MOTORES DE TURBINA DE AVIACIÓN',
 'LÍQUIDOS INFLAMABLES 1866 - 3 - RESINA; SOLUCIONES DE; inflamables',
 'LÍQUIDOS INFLAMABLES 1987 - 3 - ALCOHOLES; N.E.P.',
 'LÍQUIDOS INFLAMABLES 1993 - 3 - LÍQUIDO INFLAMABLE; N.E.P.',
 'LÍQUIDOS INFLAMABLES 2266 - 3 - DIMETIL-N-PROPILAMINA',
 'LÍQUIDOS INFLAMABLES 2319 - 3 - HIDROCARBUROS TERPÉNICOS; N.E.P.',
 'LÍQUIDOS INFLAMABLES 2370 - 3 - 1-HEXENO',
 'LÍQUIDOS INFLAMABLES 2394 - 3 - PROPIONATO DE ISOBUTILO',
 'LÍQUIDOS INFLAMABLES 2461 - 3 - METILPENTADIENOLÍQUIDOS INFLAMABLES 1262 - 3 - OCTANOS',
 'LÍQUIDOS INFLAMABLES 2560 - 3 - 2-METIL-2-PENTANOL',
 'LÍQUIDOS INFLAMABLES 3295 - 3 - HIDROCARBUROS LÍQUIDOS; N.E.P.',
 'LÍQUIDOS INFLAMABLES 3475 - 3 - MEZCLA DE ETANOL Y GASOLINA o MEZCLA DE ETANOL Y COMBUSTIBLE PARA MOT',
 'LÍQUIDOS INFLAMABLES 3494 - 3 - PETRÓLEO BRUTO ÁCIDO; INFLAMABLE; TÓXICO',
 'MATERIAL RADIACTIVO 2915 - 7 - MATERIALES RADIACTIVOS; BULTOS DEL TIPO A; no en forma especial; no f',
 'PELIGROSO RESIDUOS RESULTANTES DE LAS OPERACIONES DE ELIMINACIÓN DE DESECHOS INDUSTRIALES;',
 'SUSTANCIAS CORROSIVAS 1777 - 8 - ÁCIDO FLUOROSULFÓNICO']

combustibles = sorted(set(combustibles))


In [12]:
# Manufacturas
manufacturas = ['MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; ALAMBRE DE PUAS DE HIERRO O ACERO; ALAMBRE (SIMPLE O DOBLE) Y FLEJE;   TORCIDO',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; PUNTAS;  CLAVOS;  CHINCHETAS(CHINCHES);  GRAPAS APUNTADAS;  GRAPAS ONDULADAS',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; TABLESTACAS DE HIERRO O DE ACERO;  INCLUSO PERFORADAS O HECHAS CON ELEMENTOS E',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; TELAS METALICAS (INCLUIDAS LAS CONTINUAS O SIN FIN)REDES Y REJAS DE ALAMBRE;',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS MANUFACTURAS DE PIEDRA O DEMAS MATERIAS MINERALES (INCLUIDAS LAS FIBRAS DE CAR',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE CUCHARAS;  TENEDORES;  CUCHARONES;  ESPUMADERAS;  PALAS PARA TARTA;  CUCHILLO',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE HERRAMIENTAS DE DOS O MAS DE LAS PARTIDAS 8202 A 8205;  ACONDICIONADAS EN JUEG',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE LAYAS;  PALAS;  AZADAS;  PICOS;  BINADERAS;  HORCAS DE LABRANZA;  RASTRILLOS',
  'ALUMINIO Y MANUFACTURAS DE ALUMINIO; DEPOSITOS;  CISTERNAS;  CUBAS Y RECIPIENTES SIMILARES PARA CUALQUIER MATERIA (',
 'CINC Y MANUFACTURAS DE CINC; DESPERDICIOS Y DESECHOS;  DE CINC',
 'CINC Y MANUFACTURAS DE CINC; TUBOS Y ACCESORIOS DE TUBERIA (POR EJEMPLO RACORES (EMPALMES);  CODOS O MANGUI',
 'COBRE Y MANUFACTURAS DE COBRE; APARATOS NO ELECTRICOS DE COCCION O DE CALEFACCION;  DE LOS TIPOS DOMESTICOS',
 'COBRE Y MANUFACTURAS DE COBRE; ARTICULOS DE USO DOMESTICO;  DE HIGIENE O DE TOCADOR;  Y SUS PARTES;  DE COBRE',
 'COBRE Y MANUFACTURAS DE COBRE; PUNTAS;  CLAVOS;  CHINCHETAS (CHINCHES);  GRAPAS APUNTADAS Y ARTICULOS SIMILAR',
 'COBRE Y MANUFACTURAS DE COBRE; TUBOS DE COBRE',
 'ESTAÑO Y MANUFACTURAS DE ESTAÑO; ESTA?O EN BRUTO',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE AISLADORES ELECTRICOS DE CUALQUIER MATERIA',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE APARATOS ELECTRICOS DE ALUMBRADO O DE SE?ALIZACION (CON EXCLUSION DE LOS ARTIC',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE CALENTADORES ELECTRICOS DE AGUA Y CALENTADORES ELECTRICOS DE INMERSION; APARAT',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE CIRCUITOS IMPRESOS',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE DESPERDICIOS Y DESECHOS DE PILAS;  BATERIAS DE PILAS O ACUMULADORES ELECTRICOS',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE DISCOS;  CINTAS Y DEMAS SOPORTES PARA GRABAR SONIDO O PARA GRABACIONES ANALOGA',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE ELECTRODOS Y ESCOBILLAS DE CARBON;  CARBON PARA LAMPARAS O PARA PILAS Y DEMAS',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE LAMPARAS ELECTRICAS PORTATILES QUE FUNCIONEN CON SU PROPIA FUENTE DE ENERGIA (',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE LAMPARAS;  TUBOS Y VALVULAS ELECTRONICOS DE CATODO CALIENTE;  DE CATODO FRIO',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE PARTES IDENTIFICABLES COMO DESTINADAS;  EXCLUSIVA O PRINCIPALMENTE;  A LOS APA',
 'NÍQUEL Y MANUFACTURAS DE NÍQUEL; BARRAS;  PERFILES Y ALAMBRE;  DE NIQUEL',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; ACCESORIOS DE TUBERIAS (POR EJEMPLO RACORES (EMPALMES);  CODOS O MANGUITOS);',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; ALAMBRE DE ALUMINIO',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; ALUMINIO EN BRUTO',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; ARTICULOS DE USO DOMESTICO;  DE HIGIENE O DE TOCADOR;  Y SUS PARTES;  DE ALUMI',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; BARRAS Y PERFILES;  DE ALUMINIO',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; CABLES;  TRENZAS Y SIMILARES;  DE ALUMINIO;  SIN AISLAR PARA ELECTRICIDAD;',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; CHAPAS Y TIRAS DE ALUMINIO;  DE ESPESOR SUPERIOR A 0; 2 MM',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; CONSTRUCCIONES Y PARTES DE CONSTRUCCIONES (POR EJEMPLO PUENTES Y SUS PARTES;',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; DEPOSITOS;  BARRILES;  TAMBORES;  BIDONES;  CAJAS;  BOTES Y RECIPIENTES SIMILA',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; DEPOSITOS;  CISTERNAS;  CUBAS Y RECIPIENTES SIMILARES PARA CUALQUIER MATERIA (',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; DESPERDICIOS Y DESECHOS;  DE ALUMINIO',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; HOJAS Y TIRAS DELGADAS;  DE ALUMINIO (INCLUSO IMPRESAS O FIJADAS SOBRE PAPEL',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; LAS DEMAS MANUFACTURAS DE ALUMINIO',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; POLVO Y ESCAMILLAS;  DE ALUMINIO',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; RECIPIENTES PARA GASES COMPRIMIDOS O LICUADOS;  DE ALUMINIO',
 'ALUMINIO Y MANUFACTURAS DE ALUMINIO; TUBOS DE ALUMINIO',
 'ALUMINIO\xa0Y\xa0MANUFACTURAS\xa0DE\xa0ALUMINIO;\xa0CHAPAS\xa0Y\xa0TIRAS\xa0DE\xa0ALUMINIO;\xa0\xa0DE\xa0ESPESOR\xa0SUPERIOR\xa0A\xa00;\xa02\xa0MM',
 'ARMAS Y MUNICIONES; Y SUS PARTES Y ACCESORIOS; ARMAS DE GUERRA;  EXCEPTO LOS REVOLVERES;  PISTOLAS Y ARMAS BLANCAS',
 'ARMAS Y MUNICIONES; Y SUS PARTES Y ACCESORIOS; BOMBAS;  GRANADAS;  TORPEDOS;  MINAS;  MISILES;  CARTUCHOS Y DEMAS MUNICIONES',
 'ARMAS Y MUNICIONES; Y SUS PARTES Y ACCESORIOS; LAS DEMAS ARMAS (POR EJEMPLO ARMAS LARGAS Y PISTOLAS DE MUELLE;  (RESORTES);',
 'ARMAS Y MUNICIONES; Y SUS PARTES Y ACCESORIOS; PARTES Y ACCESORIOS DE LOS ARTICULOS DE LAS PARTIDAS 9301 A 9304',
 'ARMAS Y MUNICIONES; Y SUS PARTES Y ACCESORIOS; SABLES;  ESPADAS;  BAYONETAS;  LANZAS Y DEMAS ARMAS BLANCAS;  SUS PARTES Y FUN',
 'ARTÍCULOS DE SOMBRERERÍA Y SUS PARTES; CASCOS PARA SOMBREROS;  TRENZADOS O FABRICADOS POR UNION DE TIRAS DE CUALQUIE',
 'ARTÍCULOS DE SOMBRERERÍA Y SUS PARTES; CASCOS SIN FORMA NI ACABADO;  PLATOS (DISCOS) Y CILINDROS;  AUNQUE ESTEN CORTA',
 'ARTÍCULOS DE SOMBRERERÍA Y SUS PARTES; DESUDADORES;  FORROS;  FUNDAS;  ARMADURAS;  VISERAS Y BARBOQUEJOS ( BARBIJOS )',
 'ARTÍCULOS DE SOMBRERERÍA Y SUS PARTES; LOS DEMAS SOMBREROS Y TOCADOS;  INCLUSO GUARNECIDOS',
 'ARTÍCULOS DE SOMBRERERÍA Y SUS PARTES; SOMBREROS Y DEMAS TOCADOS DE FIELTRO FABRICADOS CON CASCOS O PLATOS DE LA PART',
 'ARTÍCULOS DE SOMBRERERÍA Y SUS PARTES; SOMBREROS Y DEMAS TOCADOS;  DE PUNTO;  O CONFECCIONADOS CON ENCAJE;  FIELTRO',
 'ARTÍCULOS DE SOMBRERERÍA Y SUS PARTES; SOMBREROS Y DEMAS TOCADOS;  TRENZADOS O FABRICADOS POR UNION DE TIRAS DE CUALQ',
 'ARTÍCULOS\xa0DE\xa0SOMBRERERÍA\xa0Y\xa0SUS\xa0PARTES;\xa0DESUDADORES;\xa0\xa0FORROS;\xa0\xa0FUNDAS;\xa0\xa0ARMADURAS;\xa0\xa0VISERAS\xa0Y\xa0BARBOQUEJOS\xa0(\xa0BARBIJOS\xa0)',
 'CINC Y MANUFACTURAS DE CINC; BARRAS;  PERFILES Y ALAMBRE;  DE CINC',
 'CINC Y MANUFACTURAS DE CINC; CHAPAS;  HOJAS Y TIRAS;  DE CINC',
 'CINC Y MANUFACTURAS DE CINC; CINC EN BRUTO',
 'COBRE Y MANUFACTURAS DE COBRE; ACCESORIOS DE TUBERIA (POR EJEMPLO RACORES (EMPALMES);  CODOS O MANGUITOS);  D',
 'COBRE Y MANUFACTURAS DE COBRE; ALAMBRE DE COBRE',
 'COBRE Y MANUFACTURAS DE COBRE; ALEACIONES MADRE DE COBRE',
 'COBRE Y MANUFACTURAS DE COBRE; ARTICULOS DE USO DOMESTICO;  DE HIGIENE O DE TOCADOR;  Y SUS PARTES;  DE COBRE',
 'COBRE Y MANUFACTURAS DE COBRE; BARRAS Y PERFILES;  DE COBRE',
 'COBRE Y MANUFACTURAS DE COBRE; CABLES;  TRENZAS Y ARTICULOS SIMILARES;  DE COBRE;  SIN AISLAR PARA  ELECTRICI',
 'COBRE Y MANUFACTURAS DE COBRE; CHAPAS Y TIRAS;  DE COBRE;  DE ESPESOR SUPERIOR A 0; 15 MM',
 'COBRE Y MANUFACTURAS DE COBRE; COBRE REFINADO Y ALEACIONES DE COBRE;  EN BRUTO',
 'COBRE Y MANUFACTURAS DE COBRE; COBRE SIN REFINAR; ANODOS DE COBRE PARA REFINADO ELECTROLITICO',
 'COBRE Y MANUFACTURAS DE COBRE; HOJAS Y TIRAS DELGADAS;  DE COBRE (INCLUSO IMPRESAS O FIJADAS SOBRE PAPEL;  CA',
 'COBRE Y MANUFACTURAS DE COBRE; LAS DEMAS MANUFACTURAS DE COBRE',
 'COBRE Y MANUFACTURAS DE COBRE; MATAS DE COBRE; COBRE DE CEMENTACION (COBRE PRECIPITADO)',
 'COBRE Y MANUFACTURAS DE COBRE; TELAS METALICAS (INCLUIDAS LAS CONTINUAS O SIN FIN)REDES Y REJAS DE ALAMBRE D',
 'CORCHO Y SUS MANUFACTURAS; CORCHO AGLOMERADO (INCLUSO CON AGLUTINANTE) Y MANUFACTURAS DE CORCHO AGLOMERADO',
 'CORCHO Y SUS MANUFACTURAS; CORCHO NATURAL EN BRUTO O SIMPLEMENTE PREPARADO; DESPERDICIOS DE CORCHO; CORCH',
 'CORCHO Y SUS MANUFACTURAS; CORCHO NATURAL;  DESCORTEZADO O SIMPLEMENTE ESCUADRADO O EN CUBOS;  PLANCHAS',
 'CORCHO Y SUS MANUFACTURAS; MANUFACTURAS DE CORCHO NATURAL',
 'ESTAÑO Y MANUFACTURAS DE ESTAÑO; LAS DEMAS MANUFACTURAS DE ESTA?O',
 'ESTAÑO Y MANUFACTURAS DE ESTAÑO; BARRAS;  PERFILES Y ALAMBRE;  DE ESTA?O',
 'ESTAÑO Y MANUFACTURAS DE ESTAÑO; CHAPAS;  HOJAS Y TIRAS;  DE ESTA?O;  DE ESPESOR SUPERIOR A 0; 2 MM',
 'ESTAÑO Y MANUFACTURAS DE ESTAÑO; DESPERDICIOS Y DESECHOS;  DE ESTA?O',
 'ESTAÑO Y MANUFACTURAS DE ESTAÑO; HOJAS Y BANDAS DELGADAS;  DE ESTA?O (INCLUSO IMPRESAS O FIJADAS SOBRE PAPEL;',
 'ESTAÑO Y MANUFACTURAS DE ESTAÑO; TUBOS Y ACCESORIOS DE TUBERIA (POR EJEMPLO RACORES (EMPALMES);  CODOS O MANGUI',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; CABLES DE FILAMENTOS ARTIFICIALES',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; CABLES DE FILAMENTOS SINTETICOS',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; DESPERDICIOS DE FIBRAS SINTETICAS O ARTIFICIALES (INCLUIDAS LAS BORRAS;  LOS D',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; FIBRAS ARTIFICIALES DISCONTINUAS;  CARDADAS;  PEINADAS O TRANSFORMADAS DE OTR',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; FIBRAS ARTIFICIALES DISCONTINUAS;  SIN CARDAR;  PEINAR NI TRANSFORMAR DE OTRO',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; FIBRAS SINTETICAS DISCONTINUAS;  CARDADAS;  PEINADAS O TRANSFORMADAS DE OTRO M',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; FIBRAS SINTETICAS DISCONTINUAS;  SIN CARDAR;  PEINAR NI TRANSFORMAR DE OTRO MO',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; HILADOS DE FIBRAS ARTIFICIALES DISCONTINUAS (EXCEPTO EL HILO DE COSER);  SIN A',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; HILADOS DE FIBRAS ARTIFICIALES SINTETICAS O ARTIFICIALES;  DISCONTINUAS (EXCEP',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; HILADOS DE FIBRAS SINTETICAS DISCONTINUAS (EXCEPTO EL HILO DE COSER);  SIN ACO',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; HILO DE COSER DE FIBRAS SINTETICAS O ARTIFICIALES;  DISCONTINUAS;  INCLUSO ACO',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; LOS DEMAS TEJIDOS DE FIBRAS SINTETICAS DISCONTINUAS',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; TEJIDOS DE FIBRAS SINTETICAS DISCONTINUAS CON UN CONTENIDO DE ESTAS FIBRAS INF',
 'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; TEJIDOS DE FIBRAS SINTETICAS DISCONTINUAS CON UN CONTENIDO DE FIBRAS SINTETICAS',
 'FIBRAS\xa0SINTÉTICAS\xa0O\xa0ARTIFICIALES\xa0DISCONTINUAS;\xa0FIBRAS\xa0ARTIFICIALES\xa0DISCONTINUAS;\xa0\xa0SIN\xa0CARDAR;\xa0\xa0PEINAR\xa0NI\xa0TRANSFORMAR\xa0DE\xa0OTRO',
 'FIBRAS\xa0SINTÉTICAS\xa0O\xa0ARTIFICIALES\xa0DISCONTINUAS;\xa0FIBRAS\xa0SINTETICAS\xa0DISCONTINUAS;\xa0\xa0CARDADAS;\xa0\xa0PEINADAS\xa0O\xa0TRANSFORMADAS\xa0DE\xa0OTRO\xa0M',
 'FIBRAS\xa0SINTÉTICAS\xa0O\xa0ARTIFICIALES\xa0DISCONTINUAS;\xa0HILADOS\xa0DE\xa0FIBRAS\xa0SINTETICAS\xa0DISCONTINUAS\xa0(EXCEPTO\xa0EL\xa0HILO\xa0DE\xa0COSER);\xa0\xa0SIN\xa0ACO',
 'FIBRAS\xa0SINTÉTICAS\xa0O\xa0ARTIFICIALES\xa0DISCONTINUAS;\xa0LOS\xa0DEMAS\xa0TEJIDOS\xa0DE\xa0FIBRAS\xa0SINTETICAS\xa0DISCONTINUAS',
 'FILAMENTOS SINTÉTICOS O ARTIFICIALES; HILADOS DE FILAMENTOS ARTIFICIALES (EXCEPTO EL HILO DE COSER);  SIN ACONDICION',
 'FILAMENTOS SINTÉTICOS O ARTIFICIALES; HILADOS DE FILAMENTOS SINTETICOS (EXCEPTO EL HILO DE COSER) SIN ACONDICIONAR P',
 'FILAMENTOS SINTÉTICOS O ARTIFICIALES; HILADOS DE FILAMENTOS SINTETICOS O ARTIFICIALES (EXCEPTO EL HILO DE COSER);  A',
 'FILAMENTOS SINTÉTICOS O ARTIFICIALES; HILO DE COSER DE FILAMENTOS SINTETICOS O ARTIFICIALES;  INCLUSO ACONDICIONADO',
 'FILAMENTOS SINTÉTICOS O ARTIFICIALES; MONOFILAMENTOS SINTETICOS DE 67 DECITEX O MAS Y CUYA MAYOR DIMENSION DE LA SEC',
 'FILAMENTOS SINTÉTICOS O ARTIFICIALES; TEJIDOS DE HILADOS DE FILAMENTOS ARTIFICIALES;  INCLUIDOS LOS FABRICADOS CON P',
 'FILAMENTOS SINTÉTICOS O ARTIFICIALES; TEJIDOS DE HILADOS DE FILAMENTOS SINTETICOS;  INCLUIDOS LOS TEJIDOS FABRICADO',
 'FILAMENTOS\xa0SINTÉTICOS\xa0O\xa0ARTIFICIALES;\xa0HILADOS\xa0DE\xa0FILAMENTOS\xa0SINTETICOS\xa0O\xa0ARTIFICIALES\xa0(EXCEPTO\xa0EL\xa0HILO\xa0DE\xa0COSER);\xa0\xa0A',
 'FILAMENTOS\xa0SINTÉTICOS\xa0O\xa0ARTIFICIALES;\xa0TEJIDOS\xa0DE\xa0HILADOS\xa0DE\xa0FILAMENTOS\xa0SINTETICOS;\xa0\xa0INCLUIDOS\xa0LOS\xa0TEJIDOS\xa0FABRICADO',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE APARATOS MECANICOS;  ACCIONADOS A MANO;   DE PESO INFERIOR O IGUAL A 10 KG UTI',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE CUCHILLAS Y HOJAS CORTANTES;  PARA MAQUINAS O PARA APARATOS MECANICOS',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE CUCHILLOS;  CON HOJA CORTANTE O DENTADA;  INCLUIDAS LAS NAVAJAS DE PODAR;  Y S',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE HERRAMIENTAS DE DOS O MAS DE LAS PARTIDAS 8202 A 8205;  ACONDICIONADAS EN JUEG',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE HERRAMIENTAS DE MANO (INCLUIDOS LOS DIAMANTES DE VIDRIERO) NO EXPRESADAS NI CO',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE LIMAS;  ESCOFINAS;  ALICATES (INCLUSO CORTANTES);  TENAZAS;  PINZAS;  CIZALLA',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE LLAVES DE AJUSTE DE MANO (INCLUIDAS LAS LLAVES DINAMOMETRICAS); CUBOS DE AJUST',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE LOS DEMAS ARTICULOS DE CUCHILLERIA (POR EJEMPLO MAQUINAS DE CORTAR EL PELO O D',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE NAVAJAS Y MAQUINAS DE AFEITAR Y SUS HOJAS (INCLUIDOS LOS ESBOZOS EN FLEJE)',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE PLAQUITAS;  VARILLAS;  PUNTAS Y ARTICULOS SIMILARES PARA UTILES;  SIN MONTAR',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE SIERRAS DE MANO; HOJAS DE SIERRA DE CUALQUIER CLASE (INCLUSO LAS FRESAS-SIERRA Y',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE TIJERAS Y SUS HOJAS',
 'HERRAMIENTAS Y ÚTILES; ARTÍCULOS DE CUCHILLERÍA CUBIERTOS DE MESA; DE UTILES INTERCAMBIABLES PARA HERRAMIENTAS DE MANO;  INCLUSO MECANICAS;  O PARA',
 'HERRAMIENTAS\xa0Y\xa0ÚTILES;\xa0ARTÍCULOS\xa0DE\xa0CUCHILLERÍA\xa0CUBIERTOS\xa0DE\xa0MESA;\xa0DE\xa0LAYAS;\xa0\xa0PALAS;\xa0\xa0AZADAS;\xa0\xa0PICOS;\xa0\xa0BINADERAS;\xa0\xa0HORCAS\xa0DE\xa0LABRANZA;\xa0\xa0RASTRILLOS',
 'HERRAMIENTAS\xa0Y\xa0ÚTILES;\xa0ARTÍCULOS\xa0DE\xa0CUCHILLERÍA\xa0CUBIERTOS\xa0DE\xa0MESA;\xa0DE\xa0LOS\xa0DEMAS\xa0ARTICULOS\xa0DE\xa0CUCHILLERIA\xa0(POR\xa0EJEMPLO\xa0MAQUINAS\xa0DE\xa0CORTAR\xa0EL\xa0PELO\xa0O\xa0D',
 'HERRAMIENTAS\xa0Y\xa0ÚTILES;\xa0ARTÍCULOS\xa0DE\xa0CUCHILLERÍA\xa0CUBIERTOS\xa0DE\xa0MESA;\xa0DE\xa0TIJERAS\xa0Y\xa0SUS\xa0HOJAS',
 'INSTRUMENTOS DE MÚSICA; PARTES Y ACCESORIOS DE ESTOS INSTRUMENTOS; ACORDEONES E INSTRUMENTOS SIMILARES; ARMONICAS',
 'INSTRUMENTOS DE MÚSICA; PARTES Y ACCESORIOS DE ESTOS INSTRUMENTOS; CAJAS DE MUSICA;  ORQUESTRIONES;  ORGANILLOS;  PAJAROS CANTORES Y DEMAS INSTRU',
 'INSTRUMENTOS DE MÚSICA; PARTES Y ACCESORIOS DE ESTOS INSTRUMENTOS; INSTRUMENTOS MUSICALES DE PERCUSION (POR EJEMPLO TAMBORES;  CAJAS;  XILOFONOS',
 'INSTRUMENTOS DE MÚSICA; PARTES Y ACCESORIOS DE ESTOS INSTRUMENTOS; INSTRUMENTOS MUSICALES EN LOS QUE EL SONIDO SE PRODUZCA O TENGA QUE AMPLIFICAR',
 'INSTRUMENTOS DE MÚSICA; PARTES Y ACCESORIOS DE ESTOS INSTRUMENTOS; LOS DEMAS INSTRUMENTOS MUSICALES DE CUERDA (POR EJEMPLO GUITARRAS;  VIOLINES',
 'INSTRUMENTOS DE MÚSICA; PARTES Y ACCESORIOS DE ESTOS INSTRUMENTOS; LOS DEMAS INSTRUMENTOS MUSICALES DE VIENTO (POR EJEMPLO CLARINETES;  TROMPETAS',
 'INSTRUMENTOS DE MÚSICA; PARTES Y ACCESORIOS DE ESTOS INSTRUMENTOS; ORGANOS DE TUBOS Y TECLADO; ARMONIOS E INSTRUMENTOS SIMILARES DE TECLADO Y LENGU',
 'INSTRUMENTOS DE MÚSICA; PARTES Y ACCESORIOS DE ESTOS INSTRUMENTOS; PARTES (POR EJEMPLO MECANISMOS DE CAJAS DE MUSICA) Y ACCESORIOS DE INSTRUMENTO',
 'INSTRUMENTOS DE MÚSICA; PARTES Y ACCESORIOS DE ESTOS INSTRUMENTOS; PIANOS;  INCLUSO AUTOMATICOS; CLAVICORDIOS Y DEMAS INSTRUMENTOS DE CUERDA CON',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE APARATOS DE MECANOTERAPIA; APARATOS PARA MASAJES; APARATOS DE SICOTECNIA; APAR',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE APARATOS DE RAYOS X Y APARATOS QUE UTILICEN LAS RADIACIONES ALFA;  BETA O GAMM',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE APARATOS FOTOGRAFICOS; APARATOS Y DISPOSITIVOS;  INCLUIDOS LAS LAMPARAS Y TUBO',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE APARATOS Y MATERIAL PARA LABORATORIOS FOTOGRAFICOS O CINEMATOGRAFICOS (INCLUID',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE ARTICULOS Y APARATOS DE ORTOPEDIA;  INCLUIDAS LAS FAJAS Y BANDAS MEDICOQUIRURG',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE BALANZAS SENSIBLES A UN PESO INFERIOR O IGUAL A 5 CG;  INCLUSO CON PESAS',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE CAMARAS Y PROYECTORES CINEMATOGRAFICOS;  INCLUSO CON GRABADORES O REPRODUCTORE',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE CONTADORES DE GASES;  DE LIQUIDOS O DE ELECTRICIDAD;  INCLUIDOS LOS DE CALIBRA',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE DENSIMETROS;  AREOMETROS;  PESALIQUIDOS E INSTRUMENTOS FLOTANTES SIMILARES;  T',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE DISPOSITIVOS DE CRISTALES LIQUIDOS QUE NO CONSTITUYAN ARTICULOS COMPRENDIDOS M',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE FIBRAS OPTICAS Y HACES DE FIBRAS OPTICAS; CABLES DE FIBRAS OPTICAS;  EXCEPTO L',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE FOTOCOPIADORAS OPTICAS O POR CONTACTO Y TERMOCOPIADORAS',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE GAFAS CORRECTORAS;  PROTECTORAS U OTRAS;  Y ARTICULOS SIMILARES',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE GEMELOS Y PRISMATICOS;  ANTEOJOS DE LARGA VISTA (INCLUIDOS LOS ASTRONOMICOS)',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE INSTRUMENTOS DE DIBUJO;  TRAZADO O CALCULO (POR EJEMPLO MAQUINAS DE DIBUJAR;',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE INSTRUMENTOS Y APARATOS AUTOMATICOS PARA LA REGULACION Y EL CONTROL',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE INSTRUMENTOS Y APARATOS DE GEODESIA;  TOPOGRAFIA;  AGRIMENSURA;  NIVELACION;',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE INSTRUMENTOS Y APARATOS DE MEDICINA;  CIRUGIA;  ODONTOLOGIA O VETERINARIA;  IN',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE INSTRUMENTOS Y APARATOS PARA ANALISIS FISICOS O QUIMICOS (POLARIMETROS;  REFRA',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE INSTRUMENTOS Y APARATOS PARA LA MEDIDA O CONTROL DEL CAUDAL;  NIVEL;  PRESION',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE INSTRUMENTOS;  APARATOS Y MAQUINAS DE MEDIDA O CONTROL;  NO EXPRESADOS NI COMP',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE INSTRUMENTOS;  APARATOS Y MODELOS;  PROYECTADOS PARA DEMOSTRACIONES (POR EJEMP',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE LENTES;  PRISMAS;  ESPEJOS Y DEMAS ELEMENTOS DE OPTICA DE CUALQUIER MATERIA;',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE LOS DEMAS APARATOS RESPIRATORIOS Y MASCARAS ANTIGAS;  CON EXCLUSION DE LAS MAS',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE LOS DEMAS CONTADORES (POR EJEMPLO CUENTARREVOLUCIONES;  CONTADORES DE PRODUCCI',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE MAQUINAS Y APARATOS PARA ENSAYOS DE DUREZA;  TRACCION;  COMPRESION;  ELASTICID',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE MICROSCOPIOS OPTICOS;  INCLUIDOS LOS DE FOTOMICROGRAFIA O CINEFOTOMICROGRAFIA',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE MICROSCOPIOS;  EXCEPTO LOS OPTICOS;  Y DIFRACTOGRAFOS',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE MONTURAS DE GAFAS O DE ARTICULOS SIMILARES Y SUS PARTES',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE OSCILOSCOPIOS;  ANALIZADORES DE ESPECTRO Y DEMAS INSTRUMENTOS Y APARATOS PARA',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE PARTES Y ACCESORIOS;  NO EXPRESADOS NI COMPRENDIDOS EN OTRA PARTE DE ESTE CAPI',
 'INSTRUMENTOS Y APARATOS DE ÓPTICA; FOTOGRAFÍA O CINEMATOGRAFÍA; DE PROYECTORES DE IMAGEN FIJA; AMPLIADORAS O REDUCTORAS;  FOTOGRAFICAS',
 'INSTRUMENTOS\xa0DE\xa0MÚSICA;\xa0PARTES\xa0Y\xa0ACCESORIOS\xa0DE\xa0ESTOS\xa0INSTRUMENTOS;\xa0INSTRUMENTOS\xa0MUSICALES\xa0EN\xa0LOS\xa0QUE\xa0EL\xa0SONIDO\xa0SE\xa0PRODUZCA\xa0O\xa0TENGA\xa0QUE\xa0AMPLIFICAR',
 'INSTRUMENTOS\xa0DE\xa0MÚSICA;\xa0PARTES\xa0Y\xa0ACCESORIOS\xa0DE\xa0ESTOS\xa0INSTRUMENTOS;\xa0LOS\xa0DEMAS\xa0INSTRUMENTOS\xa0MUSICALES\xa0DE\xa0CUERDA\xa0(POR\xa0EJEMPLO\xa0GUITARRAS;\xa0\xa0VIOLINES',
 'INSTRUMENTOS\xa0DE\xa0MÚSICA;\xa0PARTES\xa0Y\xa0ACCESORIOS\xa0DE\xa0ESTOS\xa0INSTRUMENTOS;\xa0PARTES\xa0(POR\xa0EJEMPLO\xa0MECANISMOS\xa0DE\xa0CAJAS\xa0DE\xa0MUSICA)\xa0Y\xa0ACCESORIOS\xa0DE\xa0INSTRUMENTO',
 'INSTRUMENTOS\xa0Y\xa0APARATOS\xa0DE\xa0ÓPTICA;\xa0FOTOGRAFÍA\xa0O\xa0CINEMATOGRAFÍA;\xa0DE\xa0APARATOS\xa0DE\xa0MECANOTERAPIA;\xa0APARATOS\xa0PARA\xa0MASAJES;\xa0APARATOS\xa0DE\xa0SICOTECNIA;\xa0APAR',
 'INSTRUMENTOS\xa0Y\xa0APARATOS\xa0DE\xa0ÓPTICA;\xa0FOTOGRAFÍA\xa0O\xa0CINEMATOGRAFÍA;\xa0DE\xa0APARATOS\xa0DE\xa0RAYOS\xa0X\xa0Y\xa0APARATOS\xa0QUE\xa0UTILICEN\xa0LAS\xa0RADIACIONES\xa0ALFA;\xa0\xa0BETA\xa0O\xa0GAMM',
 'INSTRUMENTOS\xa0Y\xa0APARATOS\xa0DE\xa0ÓPTICA;\xa0FOTOGRAFÍA\xa0O\xa0CINEMATOGRAFÍA;\xa0DE\xa0APARATOS\xa0Y\xa0MATERIAL\xa0PARA\xa0LABORATORIOS\xa0FOTOGRAFICOS\xa0O\xa0CINEMATOGRAFICOS\xa0(INCLUID',
 'INSTRUMENTOS\xa0Y\xa0APARATOS\xa0DE\xa0ÓPTICA;\xa0FOTOGRAFÍA\xa0O\xa0CINEMATOGRAFÍA;\xa0DE\xa0CONTADORES\xa0DE\xa0GASES;\xa0\xa0DE\xa0LIQUIDOS\xa0O\xa0DE\xa0ELECTRICIDAD;\xa0\xa0INCLUIDOS\xa0LOS\xa0DE\xa0CALIBRA',
 'INSTRUMENTOS\xa0Y\xa0APARATOS\xa0DE\xa0ÓPTICA;\xa0FOTOGRAFÍA\xa0O\xa0CINEMATOGRAFÍA;\xa0DE\xa0INSTRUMENTOS\xa0Y\xa0APARATOS\xa0AUTOMATICOS\xa0PARA\xa0LA\xa0REGULACION\xa0Y\xa0EL\xa0CONTROL',
 'JUGUETES; JUEGOS Y ARTÍCULOS PARA RECREO O PARA DEPORTE; SUS PARTES Y ARTICULOS PARA FIESTAS;  CARNAVAL U OTRAS DIVERSIONES;  INCLUIDOS LOS DE MAGI',
 'JUGUETES; JUEGOS Y ARTÍCULOS PARA RECREO O PARA DEPORTE; SUS PARTES Y ARTICULOS PARA JUEGOS DE SOCIEDAD;  INCLUIDOS LOS JUEGOS CON MOTOR O MECANISMO',
 'JUGUETES; JUEGOS Y ARTÍCULOS PARA RECREO O PARA DEPORTE; SUS PARTES Y ARTICULOS Y MATERIAL PARA CULTURA FISICA;  GIMNASIA;  ATLETISMO Y DEMAS DEPORT',
 'JUGUETES; JUEGOS Y ARTÍCULOS PARA RECREO O PARA DEPORTE; SUS PARTES Y CA?AS DE PESCAR;  ANZUELOS Y DEMAS ARTICULOS PARA LA PESCA CON CA?A SALABARDOS',
 'JUGUETES; JUEGOS Y ARTÍCULOS PARA RECREO O PARA DEPORTE; SUS PARTES Y JUGUETES DE RUEDAS CONCEBIDOS PARA QUE SE MONTEN LOS NI?OS (POR EJEMPLO TRICIC',
 'JUGUETES; JUEGOS Y ARTÍCULOS PARA RECREO O PARA DEPORTE; SUS PARTES Y LOS DEMAS JUGUETES; MODELOS REDUCIDOS A ESCALA Y MODELOS SIMILARES PARA ENTRET',
 'JUGUETES; JUEGOS Y ARTÍCULOS PARA RECREO O PARA DEPORTE; SUS PARTES Y MU?ECAS Y MUN?ECOS QUE REPRESENTEN SOLAMENTE SERES HUMANOS',
 'JUGUETES; JUEGOS Y ARTÍCULOS PARA RECREO O PARA DEPORTE; SUS PARTES Y TIOVIVOS;  COLUMPIOS;  CASETAS DE TIRO Y DEMAS ATRACCIONES DE FERIA; CIRCOS;',
 'LOS DEMÁS METALES COMUNES; <<CERMETS>>; MANUFACTURAS DE ESTAS ANTIMONIO Y SUS MANUFACTURAS ;  INCLUIDOS LOS DESPERDICIOS Y DESECHOS',
 'LOS DEMÁS METALES COMUNES; <<CERMETS>>; MANUFACTURAS DE ESTAS CIRCONIO Y SUS MANUFACTURAS ;  INCLUIDOS LOS DESPERDICIOS Y DESECHOS',
 'LOS DEMÁS METALES COMUNES; <<CERMETS>>; MANUFACTURAS DE ESTAS MAGNESIO Y SUS MANUFACTURAS;  INCLUIDOS LOS DESPERDICIOS Y DESECHOS',
 'LOS DEMÁS METALES COMUNES; <<CERMETS>>; MANUFACTURAS DE ESTAS MANGANESO YSUS MANUFACTURAS;  INCLUIDOS LOS DESPERDICIOS Y DESECHOS',
 'LOS DEMÁS METALES COMUNES; <<CERMETS>>; MANUFACTURAS DE ESTAS MATAS DE COBALTO Y DEMAS PRODUCTOS INTERMEDIOS DE LA METALURGIA DEL COBALTO; C',
 'LOS DEMÁS METALES COMUNES; <<CERMETS>>; MANUFACTURAS DE ESTAS MOLIBDENO YSUS MANUFACTURAS ;  INCLUIDOS LOS DEPERDICIOS Y DESECHOS',
 'LOS DEMÁS METALES COMUNES; <<CERMETS>>; MANUFACTURAS DE ESTAS TANTALO Y SUS MANUFACTURAS;  INCLUIDOS LOS DESPERDICIOS Y DESECHOS',
 'LOS DEMÁS METALES COMUNES; <<CERMETS>>; MANUFACTURAS DE ESTAS TITANIO Y SUS MANUFACTURAS ;  INCLUIDOS LOS DESPERDICIOS Y DESECHOS',
 'LOS DEMÁS METALES COMUNES; <<CERMETS>>; MANUFACTURAS DE ESTAS [CERMETS]Y SUS MANUFACTURAS ;  INCLUIDOS LOS DESPERDICIOS Y DESECHOS',
 'LOS DEMÁS PRODUCTOS DE ORIGEN ANIMAL NO EXPRESADOS NI COMPRENDIDOS CABELLO EN BRUTO; ICLUSO LAVADO O DESGRASADO;  DESPERDICIOS DE CABELLO',
 'LOS DEMÁS PRODUCTOS DE ORIGEN ANIMAL NO EXPRESADOS NI COMPRENDIDOS ESPONJAS NATURALES DE ORIGEN ANIMAL',
 'LOS DEMÁS PRODUCTOS DE ORIGEN ANIMAL NO EXPRESADOS NI COMPRENDIDOS HUESOS Y NUCLEOS CORNEOS ENBRUTO DESGRASADOS; SIMPLEMENTE PREPARADOS (PERO SI',
 'LOS DEMÁS PRODUCTOS DE ORIGEN ANIMAL NO EXPRESADOS NI COMPRENDIDOS MARFIL; CONCHA(CARAPAZON) DE TORTUGA; BALLENAS DE MAMIFEROS MARINOS (INCLUIDA',
 'LOS DEMÁS PRODUCTOS DE ORIGEN ANIMAL NO EXPRESADOS NI COMPRENDIDOS PIELES Y DEMAS PARTES DE AVE CON PLUMAS O SU PLUMON; PLUMAS Y PARTES DE PLUMAS',
 'LOS DEMÁS PRODUCTOS DE ORIGEN ANIMAL NO EXPRESADOS NI COMPRENDIDOS PRODUCTOS DE ORIGEN ANIMAL NO EXPRESADOS NI COMPRENDIDOS EN OTRA PARTE;  ANIMA',
 'LOS DEMÁS METALES COMUNES; <<CERMETS>>; MANUFACTURAS DE ESTAS MANGANESO YSUS MANUFACTURAS;  INCLUIDOS LOS DESPERDICIOS Y DESECHOS',
 'LOS\xa0DEMÁS\xa0ARTÍCULOS\xa0TEXTILES\xa0CONFECCIONADOS;\xa0CONJUNTOS\xa0O\xa0SURTIDOS;\xa0MANTAS',
 'LOS\xa0DEMÁS\xa0PRODUCTOS\xa0DE\xa0ORIGEN\xa0ANIMAL\xa0NO\xa0EXPRESADOS\xa0NI\xa0COMPRENDIDOS\xa0AMBAR\xa0GRIS;\xa0CASTOREO;\xa0ANGALIA\xa0Y\xa0ALMIZCLE;\xa0CANTARIDAS;\xa0BILIS;\xa0INCLUSO\xa0DESECADA',
 'LOS\xa0DEMÁS\xa0PRODUCTOS\xa0DE\xa0ORIGEN\xa0ANIMAL\xa0NO\xa0EXPRESADOS\xa0NI\xa0COMPRENDIDOS\xa0CERDAS\xa0DE\xa0CERDO\xa0O\xa0DE\xa0JABALI;\xa0PELO\xa0DE\xa0TEJON\xa0Y\xa0DEMAS\xa0PELOS\xa0DE\xa0CEPILLERIA;\xa0DESPER',
 'LOS\xa0DEMÁS\xa0PRODUCTOS\xa0DE\xa0ORIGEN\xa0ANIMAL\xa0NO\xa0EXPRESADOS\xa0NI\xa0COMPRENDIDOS\xa0HUESOS\xa0Y\xa0NUCLEOS\xa0CORNEOS\xa0ENBRUTO\xa0DESGRASADOS;\xa0SIMPLEMENTE\xa0PREPARADOS\xa0(PERO\xa0SI',
 'LÍQUIDOS INFLAMABLES 1263 - 3 - PINTURA (incluye pintura; laca; esmalte; colorante; goma laca; barniz',
 'MANUFACTURAS DE CUERO; ARTÍCULOS DE GUARNICIONERÍA O DE TALABARTERÍA; ARTICULOS DE TALABARTERIA Y GUARNICIONERIA PARA TODOS LOS ANIMALES (INCLUIDOS',
 'MANUFACTURAS DE CUERO; ARTÍCULOS DE GUARNICIONERÍA O DE TALABARTERÍA; ARTICULOS PARA USOS TECNICOS DE CUERO NATURAL O DE CUERO ARTIFICIAL O REGENERADO',
 'MANUFACTURAS DE CUERO; ARTÍCULOS DE GUARNICIONERÍA O DE TALABARTERÍA; BAULES;  MALETAS;  MALETINES;  INCLUIDOS LOS DE ASEO Y PORTADOCUMENTOS;  CARTE',
 'MANUFACTURAS DE CUERO; ARTÍCULOS DE GUARNICIONERÍA O DE TALABARTERÍA; LAS DEMAS MANUFACTURAS DE CUERO NATURAL O DE CUERO ARTIFICIAL O REGENERADO',
 'MANUFACTURAS DE CUERO; ARTÍCULOS DE GUARNICIONERÍA O DE TALABARTERÍA; MANUFACTURAS DE TRIPA;  DE VEJIGAS O DE TENDONES',
 'MANUFACTURAS DE CUERO; ARTÍCULOS DE GUARNICIONERÍA O DE TALABARTERÍA; PRENDAS Y COMPLEMENTOS DE VESTIR;  DE CUERO NATURAL O DE CUERO ARTIFICIAL O RE',
 'MANUFACTURAS DE ESPARTERÍA O DE CESTERÍA; ARTICULOS DE CESTERIA OBTENIDOS DIRECTAMENTE EN SU FORMA CON MATERIAS TRENZABLES',
 'MANUFACTURAS DE ESPARTERÍA O DE CESTERÍA; TRENZAS Y ARTICULOS SIMILARES;  DE MATERIAS TRENZABLES; MATERIAS TRENZABLES;',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; ACCESORIOS DE TUBERIA (POR EJEMPLO EMPALMES (RACORES);  CODOS O MANGUITOS);  D',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; AGUJAS DE COSER;  DE TEJER;  PASACINTAS;  AGUJAS DE GANCHILLO (CROCHE);  PUNZO',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; ALAMBRE DE PUAS DE HIERRO O ACERO; ALAMBRE (SIMPLE O DOBLE) Y FLEJE;   TORCIDO',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; ANCLAS;  REZONES Y SUS PARTES;  DE FUNDICION;  HIERRO O ACERO',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; ARTICULOS DE HIGIENE O DE TOCADOR;  Y SUS PARTES;  DE FUNDICION;  DE HIERRO O',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; ARTICULOS DE USO DOMESTICO Y SUS PARTES;  DE FUNDICION;  DE HIERRO O DE ACERO',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; CABLES;  TRENZAS;  ESLINGAS Y ARTICULOS SIMILARES;  DE HIERRO O DE ACERO;  SI',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; CADENAS Y SUS PARTES;  DE FUNDICION;  DE HIERRO O DE ACERO',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; CONSTRUCCIONES Y PARTES DE CONSTRUCCIONES (POR EJEMPLO PUENTES Y SUS PARTES CO',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; DEPOSITOS;  BARRILES;  TAMBORES;  BIDONES; LATAS O BOTES;  CAJAS Y RECIPIENTE',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; DEPOSITOS;  CISTERNAS;  CUBAS Y RECIPIENTES SIMILARES PARA CUALQUIER MATERIA (',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; ELEMENTOS PARA VIAS FERREAS;  DE FUNDICION;   HIERRO O  ACERO: CARRILES;  (RIE',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; ESTUFAS;  CALDERAS CON HOGAR;  COCINAS (INCLUIDAS LAS QUE PUEDAN UTILIZARSE AC',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; LAS DEMAS MANUFACTURAS DE HIERRO O DE ACERO',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; LAS DEMAS MANUFACTURAS MOLDEADAS;  DE FUNDICION;  DE HIERRO O DE ACERO',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; LOS DEMAS TUBOS (POR EJEMPLO SOLDADOS O REMACHADOS);  DE SECCION CIRCULAR;  CO',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; LOS DEMAS TUBOS Y PERFILES HUECOS (POR EJEMPLO SOLDADOS;  REMACHADOS;  GRAPADO',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; MUELLES (RESORTES);  BALLESTAS Y SUS HOJAS;  DE HIERRO O DE ACERO',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; RADIADORES PARA LA CALEFACCION CENTRAL;  DE CALENTAMIENTO NO ELECTRICO;  Y SU',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; RECIPIENTES PARA GASES COMPRIMIDOS O LICUADOS;  DE FUNDICION;  DE HIERRO O DE',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; TORNILLOS;  PERNOS;  TUERCAS;  TIRAFONDOS;  ESCARPIAS ROSCADAS;  REMACHES;  PA',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; TUBOS Y PERFILES HUECOS;  DE FUNDICION',
 'MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACERO; TUBOS Y PERFILES HUECOS;  SIN SOLDADURA; ( SIN COSTURA) DE HIERRO O DE ACERO',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS ABRASIVOS NATURALES O ARTIFICIALES EN POLVO O EN GRANULOS CON SOPORTE DE MATER',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS ADOQUINES;  ENCINTADO ( BORDILLOS ) Y LOSAS PARA PAVIMENTOS;  DE PIEDRA NATURA',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS AMIANTO (ASBESTO) EN FIBRAS TRABAJADO; MEZCLAS A BASE DE AMIANTO O A BASE DE A',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS GUARNICIONES DE FRICCION (POR EJEMPLO HOJAS;  ROLLOS;  TIRAS;  SEGMENTOS;  DIS',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS LANA DE ESCORIA;  DE ROCA Y LANAS MINERALES SIMILARES;  VERMICULITA DILATADA',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS MANUFACTURAS DE AMIANTO-CEMENTO;  CELULOSA-CEMENTO O SIMILARES',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS MANUFACTURAS DE ASFALTO O DE PRODUCTOS SIMILARES (POR EJEMPLO PEZ DE PETROLEO',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS MANUFACTURAS DE CEMENTO;  HORMIGON O DE PIEDRA ARTIFICIAL;  INCLUSO ARMADAS',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS MANUFACTURAS DE YESO FRAGUABLE O DE PREPARACIONES A BASE DE YESO',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS MICA TRABAJADA Y MANUFACTURAS DE MICA;  INCLUIDA LA MICA AGLOMERADA O RECONSTI',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS MUELAS Y ARTICULOS SIMILARES;  SIN BASTIDOR;  PARA MOLER;  DESFIBRAR;  TRITURA',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS PANELES;  PLACAS;  LOSETAS;  BLOQUES Y ARTICULOS SIMILARES;  DE FIBRA VEGETAL',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS PIEDRAS DE TALLAS O DE CONSTRUCCION TRABAJADAS (EXCEPTO LA PIZARRA) Y SUS MANU',
 'MANUFACTURAS DE PIEDRA; YESO; CEMENTO; AMIANTO; MICA O MATERIAS PIZARRA NATURAL TRABAJADA Y MANUFACTURAS DE PIZARRA NATURAL O AGLOMERADA',
 'MANUFACTURAS DIVERSAS COMPRESAS Y TAMPONES HIGIÉNICOS, PAÑALES PARA BEBÉS Y ARTÍCULOS SIMILARES, DE CUALQUIER MATERIA.',
 'MANUFACTURAS DIVERSAS DE METALES COMUNES; ALAMBRES;  VARILLAS;  TUBOS;  PLACAS;  ELECTRODOS Y ARTICULOS SIMILARES;  DE M',
 'MANUFACTURAS DIVERSAS DE METALES COMUNES; CAJAS DE CAUDALES;  PUERTAS BLINDADAS Y COMPARTIMIENTOS PARA CAMARAS ACORAZADA',
 'MANUFACTURAS DIVERSAS DE METALES COMUNES; CAMPANAS;  CAMPANILLAS;  GONGOS Y ARTICULOS SIMILARES;  QUE NO SEAN ELECTRICOS',
 'MANUFACTURAS DIVERSAS DE METALES COMUNES; CANDADOS;  CERRADURAS Y CERROJOS (DE LLAVES;  DE COMBINACION O ELECTRICOS);  D',
 'MANUFACTURAS DIVERSAS DE METALES COMUNES; CIERRES;  MONTURAS CIERRE;  HEBILLAS;  HEBILLAS CIERRE;  CORCHETES;  GANCHOS',
 'MANUFACTURAS DIVERSAS DE METALES COMUNES; CLASIFICADORES;  FICHEROS;  CAJAS DE CLASIFICACION;  BANDEJAS DE CORRESPONDENC',
 'MANUFACTURAS DIVERSAS DE METALES COMUNES; GUARNICIONES;  HERRAJES Y ARTICULOS SIMILARES DE METALCOMUN;  PARA MUEBLES;  P',
 'MANUFACTURAS DIVERSAS DE METALES COMUNES; MECANISMOS DE ENCUADERNACION DE HOJAS INTERCAMBIABLES O PARA CLASIFICADORES; S',
 'MANUFACTURAS DIVERSAS DE METALES COMUNES; PLACAS INDICADORAS;  PLACAS ROTULO;  PLACAS DE DIRECCIONES Y PLACAS SIMILARES',
 'MANUFACTURAS DIVERSAS DE METALES COMUNES; TAPONES Y TAPAS (INCLUIDOS LOS TAPAS CORONA;  LAS TAPAS ROSCADAS Y LOS TAPONE',
 'MANUFACTURAS DIVERSAS DE METALES COMUNES; TUBOS FLEXIBLES DE METAL COMUN;  INCLUSO CON SUS ACCESORIOS',
 'MANUFACTURAS DIVERSAS; BOLIGRAFOS; ROTULADORES Y MARCADORES CON PUNTA DE FIELTRO U OTRA PUNTA POROSA',
 'MANUFACTURAS DIVERSAS; BOTONES Y BOTONES DE PRESION; FORMAS PARA BOTONES Y DEMAS PARTES DE BOTONES O DE',
 'MANUFACTURAS DIVERSAS; CIERRES DE CREMALLERA (CIERRES DE RELAMPAGO)  Y SUS PARTES;',
 'MANUFACTURAS DIVERSAS; CINTAS PARA MAQUINAS DE ESCRIBIR Y CINTAS SIMILARES;  ENTINTADAS O PREPARADAS',
 'MANUFACTURAS DIVERSAS; ENCENDEDORES Y MECHEROS;  INCLUSO MECANICOS O ELECTRICOS Y SUS PARTES;  EXCEPT',
 'MANUFACTURAS DIVERSAS; ESCOBAS;  CEPILLOS Y BROCHAS;  AUNQUE SEAN PARTES DE MAQUINAS;  DE APARATOS O',
 'MANUFACTURAS DIVERSAS; FECHADORES;  SELLOS;  NUMERADORES;  TIMBRADORES Y ARTICULOS SIMILARES (INCLUID',
 'MANUFACTURAS DIVERSAS; JUEGOS O SURTIDOS DE VIAJE PARA EL ASEO PERSONAL;  LA COSTURA;  O LA LIMPIEZA',
 'MANUFACTURAS DIVERSAS; LAPICES;  MINAS;  PASTELES;  CARBONCILLOS;  TIZAS PARA ESCRIBIR O DIBUJAR Y JA',
 'MANUFACTURAS DIVERSAS; MANIQUIES Y ARTICULOS SIMILARES; AUTOMATAS Y ESCENAS ANIMADAS PARA ESCAPARATES',
 'MANUFACTURAS DIVERSAS; MATERIAS VEGETALES O MINERALES PARA TALLAR;  TRABAJADAS;  Y MANUFACTURAS DE ES',
 'MANUFACTURAS DIVERSAS; PEINES;  PEINETAS;  PASADORES Y ARTICULOS SIMILARES; HORQUILLAS; RIZADORES;  B',
 'MANUFACTURAS DIVERSAS; PIPAS (INCLUIDAS LAS CAZOLETAS);  BOQUILLAS PARA CIGARROS (PUROS) O CIGARRILLO',
 'MANUFACTURAS DIVERSAS; PIZARRAS Y TABLEROS PARA ESCRIBIR O DIBUJAR;  INCLUSO CON ENMARCADOS',
 'MANUFACTURAS DIVERSAS; PULVERIZADORES DE TOCADOR;  SUS MONTURAS Y CABEZAS DE MONTURAS; BORLAS Y SIMIL',
 'MANUFACTURAS DIVERSAS; TAMICES;  CEDAZOS Y CRIBAS;  DE MANO',
 'MANUFACTURAS DIVERSAS; TERMOS Y DEMAS RECIPIENTES ISOTERMICOS;  MONTADOS Y AISLADOS POR VACIO;  ASI C',
 'MANUFACTURAS\xa0DE\xa0CUERO;\xa0ARTÍCULOS\xa0DE\xa0GUARNICIONERÍA\xa0O\xa0DE\xa0TALABARTERÍA;\xa0LAS\xa0DEMAS\xa0MANUFACTURAS\xa0DE\xa0CUERO\xa0NATURAL\xa0O\xa0DE\xa0CUERO\xa0ARTIFICIAL\xa0O\xa0REGENERADO',
 'MANUFACTURAS\xa0DE\xa0FUNDICIÓN;\xa0DE\xa0HIERRO\xa0O\xa0DE\xa0ACERO;\xa0ANCLAS;\xa0\xa0REZONES\xa0Y\xa0SUS\xa0PARTES;\xa0\xa0DE\xa0FUNDICION;\xa0\xa0HIERRO\xa0O\xa0ACERO',
 'MANUFACTURAS\xa0DE\xa0FUNDICIÓN;\xa0DE\xa0HIERRO\xa0O\xa0DE\xa0ACERO;\xa0CADENAS\xa0Y\xa0SUS\xa0PARTES;\xa0\xa0DE\xa0FUNDICION;\xa0\xa0DE\xa0HIERRO\xa0O\xa0DE\xa0ACERO',
 'MANUFACTURAS\xa0DE\xa0FUNDICIÓN;\xa0DE\xa0HIERRO\xa0O\xa0DE\xa0ACERO;\xa0DEPOSITOS;\xa0\xa0CISTERNAS;\xa0\xa0CUBAS\xa0Y\xa0RECIPIENTES\xa0SIMILARES\xa0PARA\xa0CUALQUIER\xa0MATERIA\xa0(',
 'MANUFACTURAS\xa0DE\xa0FUNDICIÓN;\xa0DE\xa0HIERRO\xa0O\xa0DE\xa0ACERO;\xa0TELAS\xa0METALICAS\xa0(INCLUIDAS\xa0LAS\xa0CONTINUAS\xa0O\xa0SIN\xa0FIN)REDES\xa0Y\xa0REJAS\xa0DE\xa0ALAMBRE;',
 'MANUFACTURAS\xa0DE\xa0PIEDRA;\xa0YESO;\xa0CEMENTO;\xa0AMIANTO;\xa0MICA\xa0O\xa0MATERIAS\xa0MICA\xa0TRABAJADA\xa0Y\xa0MANUFACTURAS\xa0DE\xa0MICA;\xa0\xa0INCLUIDA\xa0LA\xa0MICA\xa0AGLOMERADA\xa0O\xa0RECONSTI',
 'MANUFACTURAS\xa0DE\xa0PIEDRA;\xa0YESO;\xa0CEMENTO;\xa0AMIANTO;\xa0MICA\xa0O\xa0MATERIAS\xa0PIZARRA\xa0NATURAL\xa0TRABAJADA\xa0Y\xa0MANUFACTURAS\xa0DE\xa0PIZARRA\xa0NATURAL\xa0O\xa0AGLOMERADA',
 'MANUFACTURAS\xa0DIVERSAS;\xa0LAPICES;\xa0\xa0MINAS;\xa0\xa0PASTELES;\xa0\xa0CARBONCILLOS;\xa0\xa0TIZAS\xa0PARA\xa0ESCRIBIR\xa0O\xa0DIBUJAR\xa0Y\xa0JA',
 'MANUFACTURAS\xa0DIVERSAS;\xa0MANIQUIES\xa0Y\xa0ARTICULOS\xa0SIMILARES;\xa0AUTOMATAS\xa0Y\xa0ESCENAS\xa0ANIMADAS\xa0PARA\xa0ESCAPARATES',
 'MANUFACTURAS\xa0DIVERSAS;\xa0PEINES;\xa0\xa0PEINETAS;\xa0\xa0PASADORES\xa0Y\xa0ARTICULOS\xa0SIMILARES;\xa0HORQUILLAS;\xa0RIZADORES;\xa0\xa0B',
 'MANUFACTURAS\xa0DIVERSAS;\xa0TERMOS\xa0Y\xa0DEMAS\xa0RECIPIENTES\xa0ISOTERMICOS;\xa0\xa0MONTADOS\xa0Y\xa0AISLADOS\xa0POR\xa0VACIO;\xa0\xa0ASI\xa0C',
 'MANUFACTURAS\xa0DIVERSAS\xa0DE\xa0METALES\xa0COMUNES;\xa0CANDADOS;\xa0\xa0CERRADURAS\xa0Y\xa0CERROJOS\xa0(DE\xa0LLAVES;\xa0\xa0DE\xa0COMBINACION\xa0O\xa0ELECTRICOS);\xa0\xa0D',
 'MANUFACTURAS\xa0DIVERSAS\xa0DE\xa0METALES\xa0COMUNES;\xa0CIERRES;\xa0\xa0MONTURAS\xa0CIERRE;\xa0\xa0HEBILLAS;\xa0\xa0HEBILLAS\xa0CIERRE;\xa0\xa0CORCHETES;\xa0\xa0GANCHOS',
 'MANUFACTURAS\xa0DIVERSAS\xa0DE\xa0METALES\xa0COMUNES;\xa0PLACAS\xa0INDICADORAS;\xa0\xa0PLACAS\xa0ROTULO;\xa0\xa0PLACAS\xa0DE\xa0DIRECCIONES\xa0Y\xa0PLACAS\xa0SIMILARES',
 'MUEBLES; MOBILIARIO MEDICOQUIRÚRGICO; ARTÍCULOS DE CAMA Y SIMILARES; APARATOS DE ALUMBRADO (INCLUIDOS LOS PROYECTORES) Y SUS PARTES;  NO EXPRESADO',
 'MUEBLES; MOBILIARIO MEDICOQUIRÚRGICO; ARTÍCULOS DE CAMA Y SIMILARES; ASIENTOS (CON EXCLUSION DE LOS DE LA PARTIDA 9402);  INCLUSO LOS TRANSFORMABLE',
 'MUEBLES; MOBILIARIO MEDICOQUIRÚRGICO; ARTÍCULOS DE CAMA Y SIMILARES; CONSTRUCCIONES PREFABRICADAS',
 'MUEBLES; MOBILIARIO MEDICOQUIRÚRGICO; ARTÍCULOS DE CAMA Y SIMILARES; LOS DEMAS MUEBLES Y SUS PARTES',
 'MUEBLES; MOBILIARIO MEDICOQUIRÚRGICO; ARTÍCULOS DE CAMA Y SIMILARES; MOBILIARIO PARA LA MEDICINA;  CIRUGIA;  ODONTOLOGIA O VETERINARIA (POR EJEMPL',
 'MUEBLES; MOBILIARIO MEDICOQUIRÚRGICO; ARTÍCULOS DE CAMA Y SIMILARES; SOMIERES; ARTICULOS DE CAMA Y ARTICULOS SIMILARES (POR EJEMPLO COLCHONES;  CUB',
 'MUEBLES;\xa0MOBILIARIO\xa0MEDICOQUIRÚRGICO;\xa0ARTÍCULOS\xa0DE\xa0CAMA\xa0Y\xa0SIMILARES;\xa0CONSTRUCCIONES\xa0PREFABRICADAS',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE ACUMULADORES ELECTRICOS;  INCLUIDOS LOS SEPARADORES;  AUNQUE SEAN CUADRADOS O',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE AFEITADORAS;  MAQUINAS CORTADORAS DE PELO O DE ESQUILAR Y APARATOS DE DEPILAR',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE APARATOS DE GRABACION O DE REPRODUCCION DE IMAGEN Y SONIDO (VIDEOS);  INCLUSO',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE APARATOS DE RADIODETECCION Y RADIOSONDEO (RADAR);  DE RADIONAVEGACION Y DE RAD',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE APARATOS ELECTRICOS DE SE?ALIZACION (EXCEPTO LOS DE TRANSMISION DE MENSAJES)',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE APARATOS ELECTRICOS DE SE?ALIZACION ACUSTICA O VISUAL (POR EJEMPLO SONERIAS;',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE APARATOS ELECTRICOS DE TELEFONIA O TELEGRAFIA CON HILOS;  INCLUIDOS LOS TELEFO',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE APARATOS ELECTROMECANICOS CON MOTOR ELECTRICO INCORPORADO;  DE USO DOMESTICO',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE APARATOS EMISORES DE RADIOTELEFONIA;  RADIOTELEGRAFIA;  RADIODIFUSION O TELEVI',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE APARATOS PARA EL CORTE;  SECCIONAMIENTO;  PROTECCION;  DERIVACION;  EMPALME O',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE APARATOS RECEPTORES DE RADIOTELEFONIA;  RADIOTELEGRAFIA O RADIODIFUSION;  INCL',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE APARATOS RECEPTORES DE TELEVISION;  INCLUSO CON UN APARATO RECEPTOR DE RADIODI',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE APARATOS Y DISPOSITIVOS ELECTRICOS DE ENCENDIDO O DE ARRANQUE;  PARA MOTORES D',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE CIRCUITOS IMPRESOS',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE CIRCUITOS INTEGRADOS Y MICROESTRUCTURAS ELECTRONICAS',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE CONDENSADORES ELECTRICOS FIJOS;  VARIABLES O AJUSTABLES',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE CUADROS;  PANELES;  ARMARIOS Y DEMAS SOPORTES EQUIPADOS CON VARIOS APARATOS D',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE DIODOS;  TRANSISTORES Y DISPOSITIVOS SEMICONDUCTORES SIMILARES; DISPOSITIVOS S',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE ELECTROIMANES; IMANES PERMANENTES Y SIMILARES ; PLATOS Y DISPOSITIVOS MAGNETIC',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE GIRADISCOS;  TOCADISCOS;  REPRODUCTORES DE CASETES Y DEMAS REPRODUCTORES DE SO',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE GRUPOS ELECTROGENOS Y CONVERTIDORES ROTATIVOS ELECTRICOS',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE HERRAMIENTAS ELECTROMECANICAS CON MOTOR ELECTRICO INCORPORADO;  DE USO MANUAL',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE HILOS;  CABLES (INCLUIDOS LOS COAXIALES) Y DEMAS CONDUCTORES AISLADOS PARA ELE',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE HORNOS ELECTRICOS INDUSTRIALES O DE LABORATORIO;  INCLUIDOS LOS DE INDUCCION',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE LAMPARAS Y TUBOS ELECTRICOS DE INCANDESCENCIA O DE DESCARGA;  INCLUIDOS LOS FA',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE MAGNETOFONOS Y DEMAS APARATOS DE GRABACION DE SONIDO;  INCLUSO CON DISPOSITIV',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE MAQUINAS Y APARATOS ELECTRICOS CON UNA FUNCION PROPIA;  NO EXPRESADOS NI COMPR',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE MAQUINAS Y APARATOS PARA SOLDAR (AUNQUE PUEDAN CORTAR);  ELECTRICOS;  DE LASER',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE MICROFONOS Y SUS SOPORTES; ALTAVOCES;  INCLUSO MONTADOS EN SUS CAJAS; AURICULA',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE MOTORES Y GENERADORES;  ELECTRICOS;  CON EXCLUSION DE LOS GRUPOS ELECTROGENOS',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE PARTES IDENTIFICABLES COMO DESTINADAS;  EXCLUSIVA O PRINCIPALMENTE;  A LAS MAQ',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE PARTES Y ACCESORIOS IDENTIFICABLES COMO DESTINADOS;  EXCLUSIVA O PRINCIPALMENT',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE PIEZAS AISLANTES TOTALMENTE DE MATERIA AISLANTE O CON SIMPLES PIEZAS METALICA',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE PILAS Y BATERIAS DE PILAS;  ELECTRICAS',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE RESISTENCIAS ELECTRICAS;  EXCEPTO LAS DE CALENTAMIENTO (INCLUIDOS LOS REOSTATO',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE SOPORTES PREPARADOS PARA GRABAR SONIDO O PARA GRABACIONES ANALOGAS;  SIN GRABA',
 'MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS PARTES; APARATOS DE TRANSFORMADORES ELECTRICOS;  CONVERTIDORES ELECTRICOS ESTATICOS (POR EJEMPLO R',
 'MÁQUINA;\xa0APARATOS\xa0Y\xa0MATERIAL\xa0ELÉCTRICO\xa0Y\xa0SUS\xa0PARTES;\xa0APARATOS\xa0DE\xa0AFEITADORAS;\xa0\xa0MAQUINAS\xa0CORTADORAS\xa0DE\xa0PELO\xa0O\xa0DE\xa0ESQUILAR\xa0Y\xa0APARATOS\xa0DE\xa0DEPILAR',
 'MÁQUINA;\xa0APARATOS\xa0Y\xa0MATERIAL\xa0ELÉCTRICO\xa0Y\xa0SUS\xa0PARTES;\xa0APARATOS\xa0DE\xa0DESPERDICIOS\xa0Y\xa0DESECHOS\xa0DE\xa0PILAS;\xa0\xa0BATERIAS\xa0DE\xa0PILAS\xa0O\xa0ACUMULADORES\xa0ELECTRICOS',
 'MÁQUINA;\xa0APARATOS\xa0Y\xa0MATERIAL\xa0ELÉCTRICO\xa0Y\xa0SUS\xa0PARTES;\xa0APARATOS\xa0DE\xa0ELECTRODOS\xa0Y\xa0ESCOBILLAS\xa0DE\xa0CARBON;\xa0\xa0CARBON\xa0PARA\xa0LAMPARAS\xa0O\xa0PARA\xa0PILAS\xa0Y\xa0DEMAS',
 'MÁQUINA;\xa0APARATOS\xa0Y\xa0MATERIAL\xa0ELÉCTRICO\xa0Y\xa0SUS\xa0PARTES;\xa0APARATOS\xa0DE\xa0GRUPOS\xa0ELECTROGENOS\xa0Y\xa0CONVERTIDORES\xa0ROTATIVOS\xa0ELECTRICOS',
 'MÁQUINA;\xa0APARATOS\xa0Y\xa0MATERIAL\xa0ELÉCTRICO\xa0Y\xa0SUS\xa0PARTES;\xa0APARATOS\xa0DE\xa0HORNOS\xa0ELECTRICOS\xa0INDUSTRIALES\xa0O\xa0DE\xa0LABORATORIO;\xa0\xa0INCLUIDOS\xa0LOS\xa0DE\xa0INDUCCION',
 'MÁQUINA;\xa0APARATOS\xa0Y\xa0MATERIAL\xa0ELÉCTRICO\xa0Y\xa0SUS\xa0PARTES;\xa0APARATOS\xa0DE\xa0SOPORTES\xa0PREPARADOS\xa0PARA\xa0GRABAR\xa0SONIDO\xa0O\xa0PARA\xa0GRABACIONES\xa0ANALOGAS;\xa0\xa0SIN\xa0GRABA',
 'NAVEGACIÓN AÉREA O ESPACIAL; APARATOS Y DISPOSITIVOS PARA LANZAMIENTO DE AERONAVES; APARATOS Y DISPOSITIVOS P',
 'NAVEGACIÓN AÉREA O ESPACIAL; GLOBOS Y DIRIGIBLES; PLANEADORES;  ALAS VOLANTES Y DEMAS AERONAVES NO CONCEBID',
 'NAVEGACIÓN AÉREA O ESPACIAL; LAS DEMAS AERONAVES (POR EJEMPLO HELICOPTEROS O AVIONES); VEHICULOS ESPACIALES (',
 'NAVEGACIÓN AÉREA O ESPACIAL; PARACAIDAS (INCLUIDOS LOS PARACAIDAS DIRIGIBLES;  PLANEADORES O GIRATORIOS); S',
 'NAVEGACIÓN AÉREA O ESPACIAL; PARTES DE LOS APARATOS DE LAS PARTIDAS 8801 A 8802',
 'NAVEGACIÓN MARÍTIMA O FLUVIAL; ARTEFACTOS FLOTANTES (POR EJEMPLO BALSAS;  DEPOSITOS;  CAJONES;  INCLUSO DE AM',
 'NAVEGACIÓN MARÍTIMA O FLUVIAL; BARCOS DE PESCA; BARCOS FACTORIA Y DEMAS BARCOS PARA EL TRATAMIENTO O LA PREPARA',
 'NAVEGACIÓN MARÍTIMA O FLUVIAL; BARCOS Y DEMAS ARTEFACTOS FLOTANTES PARA DESGUACE(1)',
 'NAVEGACIÓN MARÍTIMA O FLUVIAL; REMOLCADORES Y BARCOS EMPUJADORES',
 'NAVEGACIÓN MARÍTIMA O FLUVIAL; TRANSATLANTICOS;  BARCOS PARA EXCURSIONES;  TRANSBORDADORES;  CARGUEROS;  GABA',
 'NAVEGACIÓN MARÍTIMA O FLUVIAL; YATES Y DEMAS BARCOS Y EMBARCACIONES DE RECREO O DE DEPORTE; BARCAS DE REMO Y CA',
 'NAVEGACIÓN\xa0AÉREA\xa0O\xa0ESPACIAL;\xa0APARATOS\xa0Y\xa0DISPOSITIVOS\xa0PARA\xa0LANZAMIENTO\xa0DE\xa0AERONAVES;\xa0APARATOS\xa0Y\xa0DISPOSITIVOS\xa0P',
 'NAVEGACIÓN\xa0AÉREA\xa0O\xa0ESPACIAL;\xa0PARTES\xa0DE\xa0LOS\xa0APARATOS\xa0DE\xa0LAS\xa0PARTIDAS\xa08801\xa0A\xa08802',
 'NÍQUEL Y MANUFACTURAS DE NÍQUEL; LAS DEMAS MANUFACTURAS DE NIQUEL',
 'NÍQUEL Y MANUFACTURAS DE NÍQUEL; NIQUEL EN BRUTO',
 'NÍQUEL\xa0Y\xa0MANUFACTURAS\xa0DE\xa0NÍQUEL;\xa0NIQUEL\xa0EN\xa0BRUTO',
 'NÍQUEL\xa0Y\xa0MANUFACTURAS\xa0DE\xa0NÍQUEL;\xa0TUBOS\xa0Y\xa0ACCESORIOS\xa0DE\xa0TUBERIA\xa0(POR\xa0EJEMPLO\xa0RACORES\xa0(EMPALMES);\xa0\xa0CODOS\xa0O\xa0MANGUI',
 'OBJETOS DE ARTE; DE COLECCIÓN O DE ANTIGÜEDAD; ANTIGUEDADES DE MAS DE CIEN A?OS',
 'OBJETOS DE ARTE; DE COLECCIÓN O DE ANTIGÜEDAD; COLECCIONES Y ESPECIMENES PARA COLECCIONES DE ZOOLOGIA;  BOTANICA;  MINERALOGI',
 'OBJETOS DE ARTE; DE COLECCIÓN O DE ANTIGÜEDAD; CUADROS;  PINTURAS Y DIBUJOS;  HECHOS TOTALMENTE A MANO;  CON EXCLUSION DE LO',
 'OBJETOS DE ARTE; DE COLECCIÓN O DE ANTIGÜEDAD; GRABADOS;  ESTAMPAS Y LITOGRAFIAS ORIGINALES',
 'OBJETOS DE ARTE; DE COLECCIÓN O DE ANTIGÜEDAD; OBRAS ORIGINALES DE ESTATUARIA O DE ESCULTURA;  DE CUALQUIER MATERIA',
 'OBJETOS DE ARTE; DE COLECCIÓN O DE ANTIGÜEDAD; SELLOS (ESTAMPILLAS) DE CORREOS;  TIMBRES FISCALES;  MARCAS POSTALES;  SOBRES',
 'OBJETOS\xa0DE\xa0ARTE;\xa0DE\xa0COLECCIÓN\xa0O\xa0DE\xa0ANTIGÜEDAD;\xa0CUADROS;\xa0\xa0PINTURAS\xa0Y\xa0DIBUJOS;\xa0\xa0HECHOS\xa0TOTALMENTE\xa0A\xa0MANO;\xa0\xa0CON\xa0EXCLUSION\xa0DE\xa0LO',
 'OTROS CONJUNTOS INDUSTRIALES EXPORTADOS DE CONFORMIDAD CON EL REGLAMNETO (CEE) N? 518/',
 'OTROS OBJETOS DE ARTE O COLECCION Y ANTIGÜEDADES CLASIFICADOS POR EL CAPÍTULO 97, DE VALOR CULTURAL NACIONAL O INTERNACIONAL QUE IMPORTEN ENTIDADES PÚBLICAS, O PRIVADAS SIN ANIMO DE LUCRO.',
 'OTROS TRATAMIENTO ESPECIAL AVIONES DE PESO MAXIMO DE DESPEGUE INFERIOR O IGUAL A 5700 KG.',
 'OTROS TRATAMIENTO ESPECIAL MOTOCICLETAS (INCLUSO CON PEDALES) Y CICLOS CON MOTOR AUXILIAR, MOTOCICLETAS CILINDRADA INFERIOR A 185 CM3',
 'PARAGUAS; SOMBRILLAS; QUITASOLES; BASTONES; BASTONES ASIENTO; LÁTIGOS; BASTONES;  BASTONES ASIENTO;  LATIGOS;  FUSTAS Y ARTICULOS SIMILARES',
 'PARAGUAS; SOMBRILLAS; QUITASOLES; BASTONES; BASTONES ASIENTO; LÁTIGOS; PARAGUAS;  SOMBRILLAS Y QUITASOLES (INCLUIDOS LOS PARAGUAS BASTON;  LOS QUITAS',
 'PARAGUAS; SOMBRILLAS; QUITASOLES; BASTONES; BASTONES ASIENTO; LÁTIGOS; PARTES;  GUARNICIONES Y ACCESORIOS PARA LOS ARTICULOS DE LAS PARTIDAS 6601 O 6',
 'PARAGUAS;\xa0SOMBRILLAS;\xa0QUITASOLES;\xa0BASTONES;\xa0BASTONES\xa0ASIENTO;\xa0LÁTIGOS;\xa0PARAGUAS;\xa0\xa0SOMBRILLAS\xa0Y\xa0QUITASOLES\xa0(INCLUIDOS\xa0LOS\xa0PARAGUAS\xa0BASTON;\xa0\xa0LOS\xa0QUITAS',
 'PASTA DE MADERA O DE OTRAS MATERIAS FIBROSAS CELULÓSICAS; DESPERDICIOS Y PASTA QUIMICA DE MADERA A LA SOSA O AL SULFATO;  EXCEPTO LA PASTA PARA DISOLVE',
 'PLOMO Y MANUFACTURAS DE PLOMO; CHAPAS;  HOJAS;  TIRAS;  DE PLOMO; POLVO Y ESCAMILLAS;  DE PLOMO',
 'PLOMO Y MANUFACTURAS DE PLOMO; DESPERDICIOS Y DESECHOS;  DE PLOMO',
 'PLOMO Y MANUFACTURAS DE PLOMO; TUBOS Y ACCESORIOS DE TUBERIA (POR EJEMPLO RACORES (EMPALMES);  CODOS O MANGUI',
 'PLOMO Y MANUFACTURAS DE PLOMO; BARRAS;  PERFILES Y ALAMBRE;  DE PLOMO',
 'PLOMO Y MANUFACTURAS DE PLOMO; LAS DEMAS MANUFACTURAS DE PLOMO',
 'PLOMO Y MANUFACTURAS DE PLOMO; PLOMO EN BRUTO',
 'PLOMO\xa0Y\xa0MANUFACTURAS\xa0DE\xa0PLOMO;\xa0PLOMO\xa0EN\xa0BRUTO',
 'PLOMO\xa0Y\xa0MANUFACTURAS\xa0DE\xa0PLOMO;\xa0TUBOS\xa0Y\xa0ACCESORIOS\xa0DE\xa0TUBERIA\xa0(POR\xa0EJEMPLO\xa0RACORES\xa0(EMPALMES);\xa0\xa0CODOS\xa0O\xa0MANGUI',
 'PLUMAS Y PLUMÓN PREPARADOS Y ARTÍCULOS DE PLUMAS O PLUMÓN; FLORES CABELLO PEINADO;  AFINADO;  BLANQUEADO O PREPARADO DE OTRA FORMA; LANA;  PELO',
 'PLUMAS Y PLUMÓN PREPARADOS Y ARTÍCULOS DE PLUMAS O PLUMÓN; FLORES FLORES;  FOLLAJES Y FRUTOS;  ARTIFICIALES Y SUS PARTES; ARTICULOS CONFECCIONAD',
 'PLUMAS Y PLUMÓN PREPARADOS Y ARTÍCULOS DE PLUMAS O PLUMÓN; FLORES PELUCAS;  BARBAS;  CEJAS;  PESTA?AS;  MECHONES Y ARTICULOS ANALOGOS;  DE CABEL',
 'PLUMAS Y PLUMÓN PREPARADOS Y ARTÍCULOS DE PLUMAS O PLUMÓN; FLORES PIELES Y OTRAS PARTES DE AVES CON LAS PLUMAS O EL PLUMON; PLUMAS;  PARTES DE P',
 'PRODUCTOS FOTOGRÁFICOS O CINEMATOGRÁFICOS; PLACAS Y PELICULAS;  FOTOGRAFICAS;  IMPRESIONADAS Y REVELADAS;  EXCEPTO LAS CI',
 'PRODUCTOS FOTOGRÁFICOS O CINEMATOGRÁFICOS; PLACAS;  PELICULAS;  PAPEL;  CARTON Y TEXTILES;  FOTOGRAFICOS;  IMPRESIONADOS'
 'PRODUCTOS FOTOGRÁFICOS O CINEMATOGRÁFICOS; PAPEL;  CARTON Y TEXTILES;  FOTOGRAFICOS;  SENSIBILIZADOS;  SIN IMPRESIONAR',
 'PRODUCTOS FOTOGRÁFICOS O CINEMATOGRÁFICOS; PELICULAS CINEMATOGRAFICAS;  IMPRESIONADAS Y REVELADAS;  CON REGISTRO DE SONID',
 'PRODUCTOS FOTOGRÁFICOS O CINEMATOGRÁFICOS; PELICULAS FOTOGRAFICAS EN ROLLOS;  SENSIBILIZADAS;  SIN IMPRESIONAR;  EXCEPTO',
 'PRODUCTOS FOTOGRÁFICOS O CINEMATOGRÁFICOS; PLACAS Y PELICULAS PLANAS;  FOTOGRAFICAS;  SENSIBILIZADAS;  SIN IMPRESIONAR;',
 'PRODUCTOS FOTOGRÁFICOS O CINEMATOGRÁFICOS; PLACAS Y PELICULAS;  FOTOGRAFICAS;  IMPRESIONADAS Y REVELADAS;  EXCEPTO LAS CI',
 'PRODUCTOS FOTOGRÁFICOS O CINEMATOGRÁFICOS; PREPARACIONES QUIMICAS PARA USO FOTOGRAFICO;  EXCEPTO LOS BARNICES;  COLAS Y P',
 'PRODUCTOS\xa0FOTOGRÁFICOS\xa0O\xa0CINEMATOGRÁFICOS;\xa0PELICULAS\xa0FOTOGRAFICAS\xa0EN\xa0ROLLOS;\xa0\xa0SENSIBILIZADAS;\xa0\xa0SIN\xa0IMPRESIONAR;\xa0\xa0EXCEPTO',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS APARATOS AUXILIARES PARA LAS CALDERAS DE LAS PARTIDAS 8402 U 8403 (POR EJEMPL',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS APARATOS E INSTRUMENTOS PARA PESAR;  INCLUIDAS LAS BASCULAS Y BALANZAS PARA L',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS APARATOS MECANICOS (INCLUSO MANUALES) PARA PROYECTAR;  DISPERSAR O PULVERIZAR',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS APARATOS Y DISPOSITIVOS;  AUNQUE SE CALIENTEN ELECTRICAMENTE;  PARA EL TRATAMI',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS ARBOLES DE TRANSMISION; CAJAS DE COJINETES Y COJINETES; ENGRANAJES Y RUEDAS D',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS ARTICULOS DE GRIFERIA Y ORGANOS SIMILARES PARA TUBERIAS;  CALDERAS;  DEPOSITOS',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS BOMBAS DE AIRE O DE VACIO;  COMPRESORES DE AIRE U OTROS GASES Y VENTILADORES;',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS CAJAS DE FUNDICION; PLACAS DE FONDO PARA MOLDES; MODELOS PARA MOLDES; MOLDES P',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS CALANDRIAS Y LAMINADORES;  EXCEPTO PARA METAL O VIDRIO;  Y CILINDROS PARA ESTA',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS CALDERAS DE VAPOR (GENERADORES DE VAPOR);  EXCEPTO LAS DE CALEFACCION CENTRAL',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS CARRETILLAS APILADORAS; LAS DEMAS CARRETILLAS DE MANIPULACION CON UN DISPOSITIVO',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS CENTRIFUGADORAS INCLUIDAS LAS SECADORAS CENTRIFUGAS; APARATOS PARA FILTRAR O DEP',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS CENTROS DE MECANIZADO;  MAQUINAS DE PUESTO FIJO Y MAQUINAS DE PUESTOS MULTIPLE',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS CONVERTIDORES;  CUCHARAS DE COLADA;  LINGOTERAS Y MAQUINAS DE COLAR (MOLDEAR)',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS GRUAS Y APARATOS DE ELEVACION SOBRE CABLE AEREO; PUENTES RODANTES;  PORTICOS D',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS HERRAMIENTAS NEUMATICAS;  HIDRAULICAS O CON MOTOR INCORPORADO QUE NO SEA ELECT',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS HORNOS INDUSTRIALES O DE LABORATORIO;  INCLUIDOS LOS INCINERADORES;  QUE NO SE',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS LAMINADORES PARA METAL Y SUS CILINDROS',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS LAS DEMAS MAQUINAS HERRAMIENTA PARA TRABAJAR METALES;  <CERMETS>;  QUE NO TRAB',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS LAS DEMAS MAQUINAS Y APARATOS DE ELEVACION;  CARGA;  DESCARGA O MANIPULACION (',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS LAS DEMAS MAQUINAS Y APARATOS DE OFICINA (POR EJEMPLO COPIADORAS HECTOGRAFICA',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS LAS DEMAS MAQUINAS Y APARATOS PARA EL TRABAJO DE LA PASTA DE PAPEL;  DEL PAPE',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS LAS DEMAS MAQUINAS Y APARATOS PARA EXPLANAR;  NIVELAR;  TRAILLAR ( <SCRAPING>',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS LAS DEMAS MAQUINAS Y APARATOS;  PARA LA AGRICULTURA;  HORTICULTURA;  SILVICULT',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS LOS DEMAS MOTORES Y MAQUINAS MOTRICES',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS (INCLUIDAS LAS PRENSAS) PARA FORJAR O ESTAMPAR;  MARTILLOS PILON Y OT',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS (INCLUIDAS LAS UNIDADES DE MECANIZADO DE CORREDERAS) DE TALADRAR;  ES',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS AUTOMATICAS PARA LA VENTA DE PRODUCTOS (POR EJEMPLO SELLOS;  CIGARRIL',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS AUTOMATICAS PARA TRATAMIENTO DE INFORMACION Y SUS UNIDADES; LECTORES',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS DE CALCULADOR Y MAQUINAS DE BOLSILLO REGISTRADORAS;  REPRODUCTORAS Y',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS DE CEPILLAR;  LIMAR;  MORTAJAR;  BROCHAR; TALLAR O ACABAR LOS ENGRANA',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS DE COSER;  EXCEPTO LAS DE COSER PLIEGOS DE LA PARTIDA 8440; MUEBLES',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS DE DESBARBAR;  AFILAR;  AMOLAR;  RECTIFICAR;  LAPEAR( BRU?IR);   PULI',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS DE ESCRIBIR;  EXCEPTO LAS IMPRESORAS DE LA PARTIDA 8471; MAQUINAS PAR',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS DE TRICOTAR;  DE COSER POR CADENETA;  DE ENTORCHAR;  DE FABRICAR TUL',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS HERRAMIENTA (INCLUIDAS LAS DE CLAVAR;  GRAPAR;  ENCOLAR O ENSAMBLAR D',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS HERRAMIENTA PARA TRABAJAR LA PIEDRA;  CERAMICA;  HORMIGON;  AMIANTOCE',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS HERRAMIENTA QUE TRABAJEN POR ARRANQUE DE CUALQUIER MATERIA MEDIANTE L',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS PARA EL EXTRUDIR;  ESTIRAR;  TEXTURAR O CORTAR MATERIAL TEXTIL;  SINT',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS PARA LA LIMPIEZA;  CLASIFICACION O CRIBADO DE SEMILLAS;  GRANOS U HOR',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS PARA LA PREPARACION DE MATERIA TEXTIL; MAQUINAS PARA HILAR;  DOBLAR',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS PARA LAVAR ROPA;  INCLUSO CON DISPOSITIVO DE SECADO',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS PARA LAVAR VAJILLA; MAQUINAS Y APARATOS PARA LIMPIAR O SECAR BOTELLA',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS PARA MONTAR LAMPARAS;  TUBOS O VALVULAS ELECTRICOS O ELECTRONICOS O L',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS PARA ORDE?AR Y MAQUINAS Y APARATOS PARA LA INDUSTRIA LECHERA',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS (EXCEPTO LAS MAQUINAS DE LA PARTIDA 8450) PARA LAVAR;  LIM',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS AUXILIARES PARA LAS MAQUINAS DE LAS PARTIDAS 8444;  8445',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS MECANICOS CON UNA FUNCION PROPIA;  NO EXPRESADOS NI COMPRE',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS PARA ACONDICIONAMIENTO DE AIRE QUE COMPRENDAN UN VENTILADO',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS PARA CLASIFICAR;  CRIBAR;  SEPARAR;  LAVAR;  QUEBRANTAR;',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS PARA ENCUADERNACION;  INCLUIDAS LAS MAQUINAS PARA COSER PL',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS PARA IMPRIMIR;  INCLUIDAS LAS MAQUINAS PARA IMPRIMIR POR C',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS PARA LA FABRICACION DE PASTA DE MATERIAS FIBROSAS CELULOSICA',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS PARA LA FABRICACION O EL ACABADO DEL FIELTRO O TELAS SIN T',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS PARA LA PREPARACION;  EL CURTIDO L TRABAJO DE CUERO O PIEL',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS PARA PREPARAR O ELABORAR TABACO;  NO EXPRESADOS NI COMPREN',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS PARA SOLDAR;  AUNQUE PUEDAN CORTAR;  EXCEPTO LOS DE LA PAR',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS PARA TRABAJAR CAUCHO O PLASTICOS O PARA FABRICAR PRODUCTO',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS Y APARATOS;  NO EXPRESADOS NI COMPRENDIDOS EN OTRAS PARTES DE ESTE CA',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS;  APARATOS Y ARTEFACTOS AGRICOLAS;  HORTICOLAS O SILVICOLAS;  PARA L',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS;  APARATOS Y ARTEFACTOS PARA COSECHAR O TRILLAR;  INCLUIDAS LAS PRENS',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MAQUINAS;  APARATOS Y MATERIAL (EXCEPTO LAS MAQUINAS HERRAMIENTA DE LAS PARTID',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MOTORES DE EMBOLO (PISTON) ALTERNATIVO Y MOTORES ROTATIVOS;  DE ENCENDIDO POR',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MOTORES DE EMBOLO (PISTON) DE ENCENDIDO POR COMPRESION (MOTORES DIESEL O SEMIDIE',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS MÁQUINAS Y APARATOS UTILIZADOS, EXCLUSIVA O PRINCIPALMENTE, PARA LA FABRICACIÓN DE SEMICONDUCTORES EN FORMA DE MONOCRISTALES PERIFORMES U',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS PARTES DE MAQUINAS O DE APARATOS;  NO EXPRESADAS NI COMPRENDIDAS EN OTRA PART',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS PARTES DE MÁQUINAS O APARATOS, NO EXPRESADAS NI COMPRENDIDAS EN OTRA PARTE DE ESTE CAPÍTULO, SIN CONEXIONES ELÉCTRICAS, PARTES AISLADAS E',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS PARTES IDENTIFICABLES COMO DESTINADAS;  EXCLUSIVA O PRINCIPALMENTE;  A LAS MAQ',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS PARTES INDENTIFICABLES COMO DESTINADAS;  EXCLUSIVA O PRINCIPALMENTE;  A LOS MO',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS PARTES Y ACCESORIOS (EXCEPTO LOS ESTUCHES;  FUNDAS Y SIMILARES) IDENTIFICABLE',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS PARTES Y ACCESORIOS IDENTIFICABLES COMO DESTINADOS;  EXCLUSIVA O PRINCIPALMENT',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS POLIPASTOS; TORNOS Y CABRESTANTES; GATOS',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS PRENSAS;  ESTRUJADORAS Y MAQUINAS Y APARATOS ANALOGOS PARA LA PRODUCCION DE VI',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS QUEMADORES PARA LA ALIMENTACION DE HOGARES;  DE COMBUSTIBLES LIQUIDOS;  SOLIDO',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS REACTORES NUCLEARES; ELEMENTOS COMBUSTIBLES (CARTUCHOS) SIN IRRADIAR PARA REACTO',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS REFRIGERADORES;  CONGELADORES Y DEMAS MATERIAL;  MAQUINAS Y APARATOS PARA LA P',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS RODAMIENTOS DE BOLAS;  DE RODILLOS O DE AGUJAS',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS TELARES',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS TOPADORAS FRONTALES (<BULLDOZERS> );  TOPADORAS ANGULARES ( <ANGLEDOZERS> );',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS TORNOS (INCLUIDOS LOS CENTROS DE TORNEADO) QUE TRABAJEN POR ARRANQUE DE METAL',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS TURBINAS DE VAPOR',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS TURBINAS HIDRAULICAS;  RUEDAS HIDRAULICAS Y SUS REGULADORES',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS TURBORREACTORES;  TURBOPROPULSORES Y DEMAS TURBINAS DE GAS',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0APARATOS\xa0AUXILIARES\xa0PARA\xa0LAS\xa0CALDERAS\xa0DE\xa0LAS\xa0PARTIDAS\xa08402\xa0U\xa08403\xa0(POR\xa0EJEMPL',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0APARATOS\xa0Y\xa0DISPOSITIVOS;\xa0\xa0AUNQUE\xa0SE\xa0CALIENTEN\xa0ELECTRICAMENTE;\xa0\xa0PARA\xa0EL\xa0TRATAMI',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0LAS\xa0DEMAS\xa0MAQUINAS\xa0HERRAMIENTA\xa0PARA\xa0TRABAJAR\xa0METALES;\xa0\xa0<CERMETS>;\xa0\xa0QUE\xa0NO\xa0TRAB',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0LAS\xa0DEMAS\xa0MAQUINAS\xa0Y\xa0APARATOS;\xa0\xa0PARA\xa0LA\xa0AGRICULTURA;\xa0\xa0HORTICULTURA;\xa0\xa0SILVICULT',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0MAQUINAS\xa0HERRAMIENTA\xa0(INCLUIDAS\xa0LAS\xa0DE\xa0CLAVAR;\xa0\xa0GRAPAR;\xa0\xa0ENCOLAR\xa0O\xa0ENSAMBLAR\xa0D',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0MAQUINAS\xa0HERRAMIENTA\xa0QUE\xa0TRABAJEN\xa0POR\xa0ARRANQUE\xa0DE\xa0CUALQUIER\xa0MATERIA\xa0MEDIANTE\xa0L',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0MAQUINAS\xa0PARA\xa0LA\xa0LIMPIEZA;\xa0\xa0CLASIFICACION\xa0O\xa0CRIBADO\xa0DE\xa0SEMILLAS;\xa0\xa0GRANOS\xa0U\xa0HOR',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0MAQUINAS\xa0PARA\xa0LA\xa0PREPARACION\xa0DE\xa0MATERIA\xa0TEXTIL;\xa0MAQUINAS\xa0PARA\xa0HILAR;\xa0\xa0DOBLAR',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0MAQUINAS\xa0PARA\xa0ORDE?AR\xa0Y\xa0MAQUINAS\xa0Y\xa0APARATOS\xa0PARA\xa0LA\xa0INDUSTRIA\xa0LECHERA',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0MAQUINAS\xa0Y\xa0APARATOS\xa0(EXCEPTO\xa0LAS\xa0MAQUINAS\xa0DE\xa0LA\xa0PARTIDA\xa08450)\xa0PARA\xa0LAVAR;\xa0\xa0LIM',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0MAQUINAS\xa0Y\xa0APARATOS\xa0PARA\xa0SOLDAR;\xa0\xa0AUNQUE\xa0PUEDAN\xa0CORTAR;\xa0\xa0EXCEPTO\xa0LOS\xa0DE\xa0LA\xa0PAR',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0MOTORES\xa0DE\xa0EMBOLO\xa0(PISTON)\xa0ALTERNATIVO\xa0Y\xa0MOTORES\xa0ROTATIVOS;\xa0\xa0DE\xa0ENCENDIDO\xa0POR',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0PARTES\xa0DE\xa0MÁQUINAS\xa0O\xa0APARATOS,\xa0NO\xa0EXPRESADAS\xa0NI\xa0COMPRENDIDAS\xa0EN\xa0OTRA\xa0PARTE\xa0DE\xa0ESTE\xa0CAPÍTULO,\xa0SIN\xa0CONEXIONES\xa0ELÉCTRICAS,\xa0PARTES\xa0AISLADAS\xa0E',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0PARTES\xa0Y\xa0ACCESORIOS\xa0IDENTIFICABLES\xa0COMO\xa0DESTINADOS;\xa0\xa0EXCLUSIVA\xa0O\xa0PRINCIPALMENT',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0PRENSAS;\xa0\xa0ESTRUJADORAS\xa0Y\xa0MAQUINAS\xa0Y\xa0APARATOS\xa0ANALOGOS\xa0PARA\xa0LA\xa0PRODUCCION\xa0DE\xa0VI',
 'REACTORES\xa0NUCLEARES;\xa0CALDERAS;\xa0MÁQUINAS;\xa0APARATOS\xa0Y\xa0ARTEFACTOS\xa0REACTORES\xa0NUCLEARES;\xa0ELEMENTOS\xa0COMBUSTIBLES\xa0(CARTUCHOS)\xa0SIN\xa0IRRADIAR\xa0PARA\xa0REACTO',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS BOMBAS PARA LIQUIDOS;  INCLUSO CON DISPOSITIVO MEDIDOR INCORPORADO; ELEVADORE',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS CALDERAS PARA CALEFACCION CENTRAL;  EXCEPTO LAS DE LA PARTIDA 8402',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS GENERADORES DE GAS POBRE (DE GAS DE AIRE) O DE GAS DE AGUA;  INCLUSO CON LOS D',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS GRUAS Y APARATOS DE ELEVACION SOBRE CABLE AEREO; PUENTES RODANTES;  PORTICOS D',
 'REACTORES NUCLEARES; CALDERAS; MÁQUINAS; APARATOS Y ARTEFACTOS JUNTAS METALOPLASTICAS; JUEGOS O SURTIDOS DE JUNTAS DE DISTINTA COMPOSICION PR',
 'RELOJERÍA; APARATOS DE CONTROL DE TIEMPO Y CONTADORES DE TIEMPO;  CON MECANISMO DE RELOJE',
 'RELOJERÍA; CAJAS DE LOS RELOJES DE LAS PARTIDAS 9101 O 9102 Y SUS PARTES',
 'RELOJERÍA; CAJAS Y SIMILARES PARA APARATOS DE RELOJERIA Y SUS PARTES',
 'RELOJERÍA; DESPERTADORES Y DEMAS RELOJES CON PEQUE?O MECANISMO DE RELOJERIA',
 'RELOJERÍA; INTERRUPTORES HORARIOS Y DEMAS APARATOS QUE PERMITAN ACCIONAR UN DISPOSITIVO E',
 'RELOJERÍA; LAS DEMAS PARTES DE RELOJERIA',
 'RELOJERÍA; LOS DEMAS RELOJES;  EXCEPTO LOS QUE LLEVEN PEQUE?O MECANISMO',
 'RELOJERÍA; MECANISMOS DE RELOJERIA COMPLETOS;  SIN MONTAR O PARCIALMENTE MONTADOS (<CHABL',
 'RELOJERÍA; PEQUE?OS MECANISMOS DE RELOJERIA COMPLETOS Y MONTADOS',
 'RELOJERÍA; RELOJES DE PULSERA;  DE BOLSILLO Y RELOJES SIMILARES (INCLUIDOS LOS CONTADORE',
 'RELOJERÍA; RELOJES DE TABLERO DE INSTRUMENTOS Y RELOJES SIMILARES PARA AUTOMOVILES;  AERO',
 'RELOJERÍA;\xa0APARATOS\xa0DE\xa0CONTROL\xa0DE\xa0TIEMPO\xa0Y\xa0CONTADORES\xa0DE\xa0TIEMPO;\xa0\xa0CON\xa0MECANISMO\xa0DE\xa0RELOJE',
 'RELOJERÍA;\xa0DESPERTADORES\xa0Y\xa0DEMAS\xa0RELOJES\xa0CON\xa0PEQUE?O\xa0MECANISMO\xa0DE\xa0RELOJERIA',
 'RELOJERÍA;\xa0RELOJES\xa0DE\xa0PULSERA;\xa0\xa0DE\xa0BOLSILLO\xa0Y\xa0RELOJES\xa0SIMILARES\xa0(INCLUIDOS\xa0LOS\xa0CONTADORE',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS BICICLETAS Y DEMAS CICLOS (INCLUIDOS LOS TRICICLOS DE REPARTO);  SIN MOTOR',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS CARRETILLAS AUTOMOVIL SIN DISPOSITIVO DE ELEVACION DEL TIPO DE LAS UTILIZADAS',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS CARROCERIAS DE VEHICULOS DE LAS PARTIDAS 8701 A 8705;  INCLUSO LAS CABINAS',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS CARROS Y AUTOMOVILES BLINDADOS DE COMBATE;  INCLUSO ARMADOS; PARTES',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS CHASIS DE VEHICULOS AUTOMOVILES DE LAS PARTIDAS 8701 A 8705;  CON EL MOTOR',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS COCHES DE TURISMO Y DEMAS VEHICULOS AUTOMOVILES PROYECTADOS PRINCIPALMENTE PAR',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS COCHES;  SILLAS Y VEHICULOS SIMILARES PARA EL TRANSPORTE DE NI?OS;  Y SUS PART',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS MOTOCICLETAS (INCLUSO CON PEDALES) Y CICLOS CON MOTOR AUXILIAR;  CON SIDECAR',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS PARTES Y ACCESORIOS DE LOS VEHICULOS DE LAS PARTIDAS 8711 A 8713',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS PARTES Y ACCESORIOS DE VEHICULOS AUTOMOVILES DE LAS PARTIDAS 8701 A 8705',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS REMOLQUES Y SEMIRREMOLQUES PARA CUALQUIER VEHICULO; LOS DEMAS VEHICULOS NO AUTOM',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS SILLONES DE RUEDAS Y DEMAS VEHICULOS PARA INVALIDOS;  INCLUSO CON MOTOR U OTR',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS TRACTORES (EXCEPTO LAS CARRETILLAS TRACTOR DE LA PARTIDA 8709)',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS VEHICULOS AUTOMOVILES PARA EL TRANSPORTE DE DIEZ PERSONAS O MAS;  CONDUCTOR IN',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS VEHICULOS AUTOMOVILES PARA EL TRANSPORTE DE MERCANCIAS',
 'VEHÍCULOS AUTOMÓVILES; TRACTORES; CICLOS Y DEMÁS VEHÍCULOS TERRESTRES; SUS VEHICULOS AUTOMOVILES PARA USOS ESPECIALES;  EXCEPTO LOS PROYECTADOS PRINCIPAL',
 'VEHÍCULOS Y MATERIAL PARA VÍAS FÉRREAS O SIMILARES Y SUS PARTES; APARATOS VAGONES PARA EL TRANSPORTE DE MERCANCIAS SOBRE CARRILES',
 'VEHÍCULOS\xa0AUTOMÓVILES;\xa0TRACTORES;\xa0CICLOS\xa0Y\xa0DEMÁS\xa0VEHÍCULOS\xa0TERRESTRES;\xa0SUS\xa0CARROCERIAS\xa0DE\xa0VEHICULOS\xa0DE\xa0LAS\xa0PARTIDAS\xa08701\xa0A\xa08705;\xa0\xa0INCLUSO\xa0LAS\xa0CABINAS',
 'VEHÍCULOS\xa0AUTOMÓVILES;\xa0TRACTORES;\xa0CICLOS\xa0Y\xa0DEMÁS\xa0VEHÍCULOS\xa0TERRESTRES;\xa0SUS\xa0CHASIS\xa0DE\xa0VEHICULOS\xa0AUTOMOVILES\xa0DE\xa0LAS\xa0PARTIDAS\xa08701\xa0A\xa08705;\xa0\xa0CON\xa0EL\xa0MOTOR',
 'VEHÍCULOS\xa0Y\xa0MATERIAL\xa0PARA\xa0VÍAS\xa0FÉRREAS\xa0O\xa0SIMILARES\xa0Y\xa0SUS\xa0PARTES;\xa0APARATOS\xa0VAGONES\xa0PARA\xa0EL\xa0TRANSPORTE\xa0DE\xa0MERCANCIAS\xa0SOBRE\xa0CARRILES',
'NAVEGACIÓN MARÍTIMA O FLUVIAL; BARCOS FARO;  BARCOS BOMBA;  DRAGAS;  PONTONES GRUA Y DEMAS BARCOS EN LOS QUE',
 'NÍQUEL Y MANUFACTURAS DE NÍQUEL; CHAPAS;  TIRAS Y HOJAS;  DE NIQUEL',
 'NÍQUEL Y MANUFACTURAS DE NÍQUEL; TUBOS Y ACCESORIOS DE TUBERIA (POR EJEMPLO RACORES (EMPALMES);  CODOS O MANGUI',
 'CINC Y MANUFACTURAS DE CINC; LAS DEMAS MANUFACTURAS DE CINC',
 'CINC Y MANUFACTURAS DE CINC; POLVO Y ESCAMILLAS;  DE CINC',
 'COBRE Y MANUFACTURAS DE COBRE; APARATOS NO ELECTRICOS DE COCCION O DE CALEFACCION;  DE LOS TIPOS DOMESTICOS',
 'COBRE Y MANUFACTURAS DE COBRE; DESPERDICIOS Y DESECHOS;  DE COBRE',
 'COBRE Y MANUFACTURAS DE COBRE; MUELLES ( RESORTES) DE COBRE',
 'COBRE Y MANUFACTURAS DE COBRE; POLVO Y ESCAMILLAS;  DE COBRE',
 'COBRE Y MANUFACTURAS DE COBRE; TUBOS DE COBRE',
 'ESTAÑO Y MANUFACTURAS DE ESTAÑO; LAS DEMAS MANUFACTURAS DE ESTA?O',
 'PRODUCTOS FOTOGRÁFICOS O CINEMATOGRÁFICOS; PAPEL;  CARTON Y TEXTILES;  FOTOGRAFICOS;  SENSIBILIZADOS;  SIN IMPRESIONAR',
'PRODUCTOS FOTOGRÁFICOS O CINEMATOGRÁFICOS; PLACAS;  PELICULAS;  PAPEL;  CARTON Y TEXTILES;  FOTOGRAFICOS;  IMPRESIONADOS',
'PRODUCTOS FOTOGRÁFICOS O CINEMATOGRÁFICOS; PAPEL;  CARTON Y TEXTILES;  FOTOGRAFICOS;  SENSIBILIZADOS;  SIN IMPRESIONAR',
'PRODUCTOS FOTOGRÁFICOS O CINEMATOGRÁFICOS; PLACAS;  PELICULAS;  PAPEL;  CARTON Y TEXTILES;  FOTOGRAFICOS;  IMPRESIONADOS']
manufacturas = sorted(set(manufacturas))



In [13]:
# Mineria
mineria = [ 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE LOS METALES PRECIOSOS Y SUS CONCENTRADOS',
'FUNDICIÓN; HIERRO Y ACERO; ALAMBRE DE ACERO INOXIDABLE',
 'FUNDICIÓN; HIERRO Y ACERO; ALAMBRON DE HIERRO O DE ACERO SIN ALEAR',
 'FUNDICIÓN; HIERRO Y ACERO; BARRAS Y PERFILES;  DE ACERO INOXIDABLE',
 'FUNDICIÓN; HIERRO Y ACERO; FERROALEACIONES',
 'FUNDICIÓN; HIERRO Y ACERO; HIERRO Y ACERO SIN ALEAR;  EN LINGOTES U DEMAS FORMAS PRIMARIAS;  EXCEPTO EL H',
 'FUNDICIÓN; HIERRO Y ACERO; PERFILES DE HIERRO O DE ACERO SIN ALEAR',
 'FUNDICIÓN; HIERRO Y ACERO; PRODUCTOS INTERMEDIOS DE HIERRO O DE ACERO SIN ALEAR',
 'FUNDICIÓN; HIERRO Y ACERO; PRODUCTOS LAMINADOS PLANOS DE LOS DEMAS ACEROS ALEADOS;  DE ANCHURA INFERIOR',
 'FUNDICIÓN; HIERRO Y ACERO; PRODUCTOS LAMINADOS PLANOS DE LOS DEMAS ACEROS ALEADOS;  DE ANCHURA SUPERIOR',
'PERLAS\xa0FINAS\xa0O\xa0CULTIVADAS;\xa0PIEDRAS\xa0PRECIOSAS\xa0Y\xa0SEMIPRECIOSAS\xa0O\xa0SIMILARES;\xa0BISUTERIA',
 'PERLAS\xa0FINAS\xa0O\xa0CULTIVADAS;\xa0PIEDRAS\xa0PRECIOSAS\xa0Y\xa0SEMIPRECIOSAS\xa0O\xa0SIMILARES;\xa0LAS\xa0DEMAS\xa0MANUFACTURAS\xa0DE\xa0METAL\xa0PRECIOSO\xa0O\xa0DE\xa0CHAPADOS\xa0DE\xa0METAL\xa0PRECIOSO\xa0(\xa0PLAQU',
 'FUNDICIÓN; HIERRO Y ACERO; ACERO INOXIDABLE EN LINGOTES O DEMAS  FORMAS PRIMARIAS;PRODUCTOS INTERMEDIOS DE',
 'FUNDICIÓN; HIERRO Y ACERO; ALAMBRE DE HIERRO O DE ACERO SIN ALEAR',
 'FUNDICIÓN; HIERRO Y ACERO; ALAMBRE DE LOS DEMAS ACEROS ALEADOS',
 'FUNDICIÓN; HIERRO Y ACERO; ALAMBRON DE ACERO INOXIDABLE',
 'FUNDICIÓN; HIERRO Y ACERO; ALAMBRON DE HIERRO O DE ACERO SIN ALEAR',
 'FUNDICIÓN; HIERRO Y ACERO; ALAMBRON DE LOS DEMAS ACEROS',
 'FUNDICIÓN; HIERRO Y ACERO; BARRAS DE HIERRO O DE ACERO SIN ALEAR;  SIMPLEMENTE FORJADAS;  LAMINADAS O EXT',
 'FUNDICIÓN; HIERRO Y ACERO; DESPERDICIOS Y DESECHOS ( CHATARRA );  DE FUNDICION;  HIERRO O ACERO; LINGOTE',
 'FUNDICIÓN; HIERRO Y ACERO; FUNDICION EN BRUTO Y FUNDICION ESPECULAR;  EN LINGOTES;  BLOQUES U OTRAS FORMA',
 'FUNDICIÓN; HIERRO Y ACERO; GRANALLAS Y POLVO;  DE FUNDICION EN BRUTO;  DE FUNDICION ESPECULAR;  DE HIERR',
 'FUNDICIÓN; HIERRO Y ACERO; HIERRO Y ACERO SIN ALEAR;  EN LINGOTES U DEMAS FORMAS PRIMARIAS;  EXCEPTO EL H',
 'FUNDICIÓN; HIERRO Y ACERO; LAS DEMAS BARRAS DE HIERRO O DE ACERO SIN ALEAR',
 'FUNDICIÓN; HIERRO Y ACERO; LOS DEMAS ACEROS ALEADOS EN LINGOTES O DEMAS FORMAS PRIMARIAS;PRODUCTOS INTERMED',
 'FUNDICIÓN; HIERRO Y ACERO; PERFILES DE HIERRO O DE ACERO SIN ALEAR',
 'FUNDICIÓN; HIERRO Y ACERO; PRODUCTOS FERREOS OBTENIDOS POR REDUCCION DIRECTA DE MINERALES DE HIERRO Y DEM',
 'FUNDICIÓN; HIERRO Y ACERO; PRODUCTOS LAMINADOS PLANOS DE ACERO INOXIDABLE;  DE ANCHURA SUPERIOR O IGUAL',
 'FUNDICIÓN; HIERRO Y ACERO; PRODUCTOS LAMINADOS PLANOS DE HIERRO O DE ACERO SIN ALEAR;  DE ANCHURA INFERIO',
 'FUNDICIÓN; HIERRO Y ACERO; PRODUCTOS LAMINADOS PLANOS DE HIERRO O DE ACERO SIN ALEAR;  DE ANCHURA SUPERIO',
 'FUNDICIÓN; HIERRO Y ACERO; PRODUCTOS LAMINADOS PLANOS DE LOS DEMAS ACEROS ALEADOS;  DE ANCHURA INFERIOR',
 'FUNDICIÓN; HIERRO Y ACERO; BARRAS Y PERFILES;  DE ACERO INOXIDABLE',
 'FUNDICIÓN; HIERRO Y ACERO; BARRAS Y PERFILES;  DE LOS DEMAS ACEROS ALEADOS; BARRAS HUECAS PARA PERFORACIO',
 'FUNDICIÓN; HIERRO Y ACERO; FERROALEACIONES',
 'FUNDICIÓN; HIERRO Y ACERO; PRODUCTOS LAMINADOS PLANOS DE ACERO INOXIDABLE;  DE ANCHURA INFERIOR A 600 MM',
 'FUNDICIÓN; HIERRO Y ACERO; PRODUCTOS LAMINADOS PLANOS DE LOS DEMAS ACEROS ALEADOS;  DE ANCHURA SUPERIOR',

 'MINERALES; ESCORIAS Y CENIZAS; CENIZAS Y RESIDUOS (EXCEPTO LOS DE LA SIDERURGIA) QUE CONTENGAN METAL O COMPUEST',
 'MINERALES; ESCORIAS Y CENIZAS; ESCORIAS (EXCEPTO LAS GRANULADAS);  BATIDURAS Y DEMAS DESPERDICIOS DE LA SIDER',
 'MINERALES; ESCORIAS Y CENIZAS; ESCORIAS GRANULADAS (ARENA DE ESCORIAS) DE LA SIDERURGIA',
 'MINERALES; ESCORIAS Y CENIZAS; LAS DEMAS ESCORIAS Y CENIZAS;  INCLUIDAS LAS CENIZAS DE ALGAS',
 'MINERALES; ESCORIAS Y CENIZAS; LOS DEMAS MINERALES Y SUS CONCENTRADOS',
 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE ALUMINIO Y SUS CONCENTRADOS',
 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE CINC Y SUS CONCENTRADOS',
 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE COBRE Y SUS CONCENTRADOS',
 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE CROMO Y SUS CONCENTRADOS',
 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE ESTAÑO Y SUS CONCENTRADOS',
 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE HIERRO Y SUS CONCENTRADOS;  INCLUIDAS LAS PIRITAS DE HIERRO;  TOS',
 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE LOS METALES PRECIOSOS Y SUS CONCENTRADOS',
 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE MANGANESO Y SUS CONCENTRADOS;  INCLUIDOS LOS MINERALES DE MANGANE',
 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE NIOBIO;  DE TANTALO;  DE VANADIO O DE CIRCONIO Y SUS CONCENTRADOS',
 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE NIQUEL Y SUS CONCENTRADOS',
 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE PLOMO Y SUS CONCENTRADOS',
 'MINERALES; ESCORIAS Y CENIZAS; MINERALES DE TITANIO Y SUS CONCENTRADOS',
 'PERLAS FINAS O CULTIVADAS; PIEDRAS PRECIOSAS Y SEMIPRECIOSAS O SIMILARES; ARTICULOS DE JOYERIA Y SUS PARTES DE METAL PRECIOSO O DE CHAPADOS DE METAL PRECI',
 'PERLAS FINAS O CULTIVADAS; PIEDRAS PRECIOSAS Y SEMIPRECIOSAS O SIMILARES; ARTICULOS DE ORFEBRERIA Y SUS PARTES;  DE METAL PRECIOSO O DE CHAPADOS DE META',
 'PERLAS FINAS O CULTIVADAS; PIEDRAS PRECIOSAS Y SEMIPRECIOSAS O SIMILARES; BISUTERIA',
 'PERLAS FINAS O CULTIVADAS; PIEDRAS PRECIOSAS Y SEMIPRECIOSAS O SIMILARES; CHAPADO ( PLAQUE ) DE PLATINO SOBRE METAL COMUN;  PLATA U ORO;  EN BRUTO O SEM',
 'PERLAS FINAS O CULTIVADAS; PIEDRAS PRECIOSAS Y SEMIPRECIOSAS O SIMILARES; LAS DEMAS MANUFACTURAS DE METAL PRECIOSO O DE CHAPADOS DE METAL PRECIOSO ( PLAQU',
 'PERLAS FINAS O CULTIVADAS; PIEDRAS PRECIOSAS Y SEMIPRECIOSAS O SIMILARES; MANUFACTURAS DE PERLAS FINAS ( NATURALES ) O CULTIVADAS;  DE PIEDRAS PRECIOSA',
 'PERLAS FINAS O CULTIVADAS; PIEDRAS PRECIOSAS Y SEMIPRECIOSAS O SIMILARES; MONEDAS',
 'PERLAS FINAS O CULTIVADAS; PIEDRAS PRECIOSAS Y SEMIPRECIOSAS O SIMILARES; ORO;  (INCLUIDO EL ORO PLATINADO );  EN BRUTO;  SEMILABRADO O EN POLVO',
 'PERLAS FINAS O CULTIVADAS; PIEDRAS PRECIOSAS Y SEMIPRECIOSAS O SIMILARES; PLATA;  INCLUIDA LA PLATA DORADA Y LA PLATINADA;  EN BRUTO;  SEMILABRADA O EN',
 'PERLAS FINAS O CULTIVADAS; PIEDRAS PRECIOSAS Y SEMIPRECIOSAS O SIMILARES; PLATINO EN BRUTO;  SEMILABRADO O EN POLVO',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; AMIANTO (ASBESTO)',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; ARENAS NATURALES DE CUALQUIER CLASE;  INCLUSO COLOREADAS;  CON EXCLUSION DE LA',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; AZUFRE DE CUALQUIER CLASE;  CON EXCLUSION DEL SUBLIMADO;  DEL PRECIPITADO Y DE',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; BORATOS NATURALES Y SUS CONCENTRADOS (INCLUSO CALCINADOS);  CON EXCLUSION DE L',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; CAL VIVA;  CAL APAGADA Y CAL HIDRAULICA;  CON EXCLUSION DEL OXIDO Y DEL HIDROX',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; CANTOS;  GRAVA;  PIEDRAS MACHACADAS;  DE LOS TIPOS GENERALMENTE UTILIZADOS PAR',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; CAOLIN Y DEMAS ARCILLAS CAOLINICAS;  INCLUSO CALCINADAS',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; CARBONATO DE MAGNESIO NATURAL (MAGNESITA); MAGNESIA ELECTROFUNDIDA; MAGNESIA C',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; CASTINAS; PIEDRAS PARA LA FABRICACION DE CAL O DE CEMENTO',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; CEMENTOS HIDRAULICOS (INCLUIDOS LOS CEMENTOS SIN PULVERIZAR O CLINKER);  AUNQU',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; CRETA',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; CUARZO;  (EXCEPTO LAS ARENAS NATURALES); CUARCITA;  INCLUSO DESBASTADA O SIMPL',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; DOLOMITA;  INCLUSO SINTERIZADA O CALCINADA; DOLOMITA DESBASTADA O SIMPLEMENTE',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; ESTEATITA NATURAL;  INCLUSO DESBASTADA O SIMPLEMENTE TROCEADA;  POR ASERRADO',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; FELDESPATO; LEUCITA; NEFELINA Y NEFELINA SIENITA; ESPATO FLUOR',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; FOSFATOS DE CALCIO NATURALES;  FOSFATOS ALUMINOCALCICOS NATURALES Y CRETAS FOS',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; GRAFITO NATURAL',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; GRANITO;  PORFIDO;  BASALTO;  ARENISCA Y DEMAS PIEDRAS DE TALLA O DE CONSTRUCC',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; LAS DEMAS ARCILLAS (CON EXCLUSION DE LAS ARCILLAS DILATADAS DE LA PARTIDA 6806',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; MARMOL;  TRAVERTINOS;  ECAUSSINES Y DEMAS PIEDRASCALIZAS DE TALLA O DE CONSTRU',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; MATERIAS MINERALES NO EXPRESADAS NI COMPRENDIDAS EN OTRAS PARTIDAS',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; MICA;  INCLUIDA LA MICA EXFOLIADA EN LAMINILLAS IRREGULARES; DESPERDICIOS DE M',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; PIEDRA POMEZ; ESMERIL; CORINDON NATURAL;  GRANATE NATURAL Y DEMAS ABRASIVOS NA',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; PIRITAS DE HIERRO SIN TOSTAR',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; PIZARRA;  INCLUSO DESBASTADA O SIMPLEMENTE TROCEADA;  POR ASERRADO O DE OTRO M',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; SAL (INCLUIDAS LA DE MESA Y LA DESNATURALIZADA) Y CLORURO DE SODIO;  PURO;  IN',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; SULFATO DE BARIO NATURAL (BARITINA); CARBONATO DE BARIO NATURAL (WITHERITA);',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; YESO NATURAL; ANHIDRITA; YESOS CALCINADOS;  INCLUSO COLOREADOS O CON PEQUE?AS',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; CARBONATO DE MAGNESIO NATURAL (MAGNESITA); MAGNESIA ELECTROFUNDIDA; MAGNESIA C',
 'SAL; AZUFRE; TIERRAS Y PIEDRAS; YESOS; CALES Y CEMENTOS; HARINAS SILICEAS FOSILES (POR EJEMPLO KIESELGUHR;  TRIPOLITA O DIATOMITA) Y DE',
]

mineria = sorted(set(mineria))


In [14]:

# Papel y carton
papel_carton = [  'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; FLEJES DE MADERA; RODRIGONES HENDIDOS; ESTACAS DE MADERA;  APUNTADAS;  SIN ASE',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; MADERA CONTRACHAPADA;  MADERA CHAPADA Y MADERA ESTRATIFICADA SIMILAR',
  'PASTA DE MADERA O DE OTRAS MATERIAS FIBROSAS CELULÓSICAS; DESPERDICIOS Y PASTA QUIMICA DE MADERA PARA DISOLVER',
 'PASTA\xa0DE\xa0MADERA\xa0O\xa0DE\xa0OTRAS\xa0MATERIAS\xa0FIBROSAS\xa0CELULÓSICAS;\xa0DESPERDICIOS\xa0Y\xa0PASTA\xa0QUIMICA\xa0DE\xa0MADERA\xa0A\xa0LA\xa0SOSA\xa0O\xa0AL\xa0SULFATO;\xa0\xa0EXCEPTO\xa0LA\xa0PASTA\xa0PARA\xa0DISOLVE',
'CAUCHO Y MANUFACTURAS DE CAUCHO; PRENDAS;  GUANTES Y DEMAS COMPLEMENTOS DE VESTIR;  PARA CUALQUIER USO;  DE CAU',
 'PRODUCTOS\xa0EDITORIALES;\xa0DE\xa0LA\xa0PRENSA\xa0O\xa0DE\xa0OTRAS\xa0INDUSTRIAS\xa0GRÁFICAS;\xa0TEXTOS\xa0LOS\xa0DEMAS\xa0IMPRESOS;\xa0\xa0INCLUIDAS\xa0LAS\xa0ESTAMPAS;\xa0\xa0GRABADOS\xa0Y\xa0FOTOGRAFIAS',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; ARTICULOS DE MESA O DE COCINA;  DE MADERA',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; CAJONES;  CAJAS;  JAULAS;  TAMBORES Y ENVASES SIMILARES;  DE MADERA; CARRETES',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; CARBON VEGETAL (INCLUIDO EL DE CASCARAS O DE HUESOS DE FRUTAS);  AUNQUE ESTE A',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; LANA (VIRUTA) DE MADERA; HARINA DE MADERA',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; MADERA (INCLUIDAS LAS TABLILLAS Y FRISOS PARA PARQUES;  SIN ENSAMBLAR) PERFILA',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; MADERA ASERRADA O DESBASTADA LONGITUDINALMENTE;  CORTADA O DESENROLLADA;  INCL',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; MADERA CONTRACHAPADA;  MADERA CHAPADA Y MADERA ESTRATIFICADA SIMILAR',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; MADERA DENSIFICADA EN BLOQUES;  PLANCHAS;  TABLAS O PERFILES',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; MADERA EN BRUTO;  INCLUSO DESCORTEZADA;  DESALBURADA O ESCUADRADA',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; MARCOS DE MADERA PARA CUADROS;  FOTOGRAFIAS;  ESPEJOS U OBJETOS SIMILARES',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; MARQUETERIA Y TARACEA; COFRES;  CAJAS Y ESTUCHES PARA JOYERIA U ORFEBRERIA Y M',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; BARRILES;  CUBAS;  TINAS Y DEMAS MANUFACTURAS DE TONELERIA Y SUS PARTES;  DE M',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; HERRAMIENTAS;  MONTURAS Y MANGOS DE HERRAMIENTAS;  MONTURAS DE CEPILLOS;  MANG',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; HOJAS PARA CHAPADO Y CONTRACHAPADO (INCLUSO UNIDAS) Y DEMAS MADERAS ASERRADAS',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; LAS DEMAS MANUFACTURAS DE MADERA',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; LE?A; MADERA EN PLAQUITAS O PARTICULAS; ASERRIN;  DESPERDICIOS Y DESECHOS;  D',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; OBRAS Y PIEZAS DE CARPINTERIA PARA CONSTRUCCIONES;  INCLUIDOS LOS TABLEROS CEL',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; TABLEROS DE FIBRA DE MADERA U OTRAS MATERIAS LE?OSAS;  INCLUSO AGLOMERADOS CO',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; TABLEROS DE PARTICULAS Y TABLEROS SIMILARES DE MADERA O DE OTRAS MATERIAS LE?O',
 'MADERA; CARBÓN VEGETAL Y MANUFACTURAS DE MADERA; TRAVIESAS DE MADERA PARA VIAS FERREAS O SIMILARES',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; BLOQUES Y PLACAS;  FILTRANTES;  DE PASTA DE PAPEL       -',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; BLOQUES Y PLACAS;  FILTRANTES;  DE PASTA DE PAPEL-',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; CAJAS;  SACOS;  BOLSAS;  CUCURUCHOS Y DEMAS ENVASES DE PAPEL;  CARTON;  GUATA',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; CUBRESUELOS CON SOPORTE DE PAPEL O CARTON INCLUSO CORTADOS',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; ETIQUETAS DE TODAS CLASES;  DE PAPEL O CARTON;  INCLUSO IMPRESAS',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; LIBROS REGISTRO;  LIBROS DE CONTABILIDAD;  TALONARIOS;  AGENDAS;  MEMORANDOS',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; LOS DEMAS PAPELES Y CARTONES;  SIN ESTUCAR NI RECUBRIR;  EN BOBINAS (ROLLOS)',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; LOS DEMAS PAPELES;  CARTONES;  GUATAS DE CELULOSA Y NAPAS DE FIBRAS DE CELULOS',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL CARBON (CARBONICO);  PAPEL AUTOCOPIA Y DEMAS PAPELES PARA COPIAR O TRANS',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL CARBON;  PAPEL AUTOCOPIA Y DEMAS PAPELES PARA COPIAR O TRANSFERIR (EXCEP',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL DE FUMAR;  INCLUSO CORTADO AL TAMA?O ADECUADO;  EN LIBRILLOS O EN TUBOS',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL DEL TIPO DEL UTILIZADO PARA PAPEL HIGIENICO Y PAPELES SIMILARES;  GUATA',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL DEL TIPO DEL UTILIZADO PARA PAPEL HIGIENICO;  TOALLITAS PARA DESMAQUILLA',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL PARA DECORAR Y REVESTIMIENTOS SIMILARES DE PAREDES; PAPEL PARA VIDRIERAS',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL PRENSA EN BOBINAS O EN HOJAS',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL Y CARTON CORRUGADOS (INCLUSO REVESTIDOS POR ENCOLADO);  RIZADOS (<CREPES',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL Y CARTON KRAFT;  SIN ESTUCAR NI RECUBRIR;  EN BOBINAS O EN HOJAS;  EXCEP',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL Y CARTON OBTENIDOS POR PEGADO DE HOJAS PLANAS;  SIN ESTUCAR NI RECUBRIR',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL Y CARTON SULFURIZADO;  PAPEL RESISTENTE A LAS GRASAS;  PAPEL VEGETAL',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL Y CARTON;  ESTUCADO POR UNA O LAS DOS CARAS EXCLUSIVAMENTE CON CAOLIN U',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL Y CARTON;  SIN ESTUCAR NI RECUBRIR;  DEL TIPO DE LOS UTILIZADOS PARA ESC',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; PAPEL;  CARTON;  GUATA DE CELULOSA Y NAPAS DE FIBRAS DE CELULOSA;  ESTUCADOS',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; SOBRES;  SOBRES-CARTA;  TARJETAS POSTALES SIN ILUSTRAR Y TARJETAS PARA CORRESP',
 'PAPEL Y CARTÓN; MANUFACTURAS DE PASTA DE CELULOSA; DE PAPEL O DE CARTÓN; TAMBORES;  BOBINAS;  CANILLAS Y SOPORTES SIMILARES;  DE PASTA DE PAPEL;  PAPE',
 'PASTA DE MADERA O DE OTRAS MATERIAS FIBROSAS CELULÓSICAS; DESPERDICIOS Y PAPEL O CARTON PARA RECICLAR (DESPERDICIOS Y DESECHOS)',
 'PASTA DE MADERA O DE OTRAS MATERIAS FIBROSAS CELULÓSICAS; DESPERDICIOS Y PASTA DE FIBRAS OBTENIDAS DE PAPEL O CARTON RECICLADOS (DESPERDICIOS Y DESECHOS)',
 'PASTA DE MADERA O DE OTRAS MATERIAS FIBROSAS CELULÓSICAS; DESPERDICIOS Y PASTA MECANICA DE MADERA',
 'PASTA DE MADERA O DE OTRAS MATERIAS FIBROSAS CELULÓSICAS; DESPERDICIOS Y PASTA QUIMICA DE MADERA PARA DISOLVER',
 'PRODUCTOS EDITORIALES; DE LA PRENSA O DE OTRAS INDUSTRIAS GRÁFICAS; TEXTOS CALCOMANIAS DE CUALQUIER CLASE',
 'PRODUCTOS EDITORIALES; DE LA PRENSA O DE OTRAS INDUSTRIAS GRÁFICAS; TEXTOS CALENDARIOS DE CUALQUIER CLASE IMPRESOS;  INCLUIDOS LOS TACOS DE CALENDARIO',
 'PRODUCTOS EDITORIALES; DE LA PRENSA O DE OTRAS INDUSTRIAS GRÁFICAS; TEXTOS DIARIOS Y PUBLICACIONES PERIODICAS;  IMPRESOS;  INCLUSO ILUSTRADOS O CON PUBLI',
 'PRODUCTOS EDITORIALES; DE LA PRENSA O DE OTRAS INDUSTRIAS GRÁFICAS; TEXTOS LIBROS;  FOLLETOS E IMPRESOS SIMILARES;  INCLUSO EN HOJAS SUELTAS',
 'PRODUCTOS EDITORIALES; DE LA PRENSA O DE OTRAS INDUSTRIAS GRÁFICAS; TEXTOS LOS DEMAS IMPRESOS;  INCLUIDAS LAS ESTAMPAS;  GRABADOS Y FOTOGRAFIAS',
 'PRODUCTOS EDITORIALES; DE LA PRENSA O DE OTRAS INDUSTRIAS GRÁFICAS; TEXTOS MANUFACTURAS CARTOGRAFICAS DE TODAS CLASES;  INCLUIDOS LOS MAPAS MURALES;  LO',
 'PRODUCTOS EDITORIALES; DE LA PRENSA O DE OTRAS INDUSTRIAS GRÁFICAS; TEXTOS PLANOS Y DIBUJOS ORIGINALES HECHOS A MANO;  DE ARQUITECTURA;  DE INGENIERIA;',
 'PRODUCTOS EDITORIALES; DE LA PRENSA O DE OTRAS INDUSTRIAS GRÁFICAS; TEXTOS SELLOS DE CORREOS;  TIMBRES FISCALES Y ANALOGOS;  SIN OBLITERAR;  QUE TENGAN',
 'PRODUCTOS EDITORIALES; DE LA PRENSA O DE OTRAS INDUSTRIAS GRÁFICAS; TEXTOS TARJETAS POSTALES IMPRESAS O ILUSTRADAS; TARJETAS IMPRESAS CON FELICITACIONES',
 'PRODUCTOS EDITORIALES; DE LA PRENSA O DE OTRAS INDUSTRIAS GRÁFICAS; TEXTOS ALBUMES O LIBROS DE ESTAMPAS PARA NI?OS Y CUADERNOS INFANTILES PARA DIBUJAR O CO',
]

papel_carton = sorted(set(papel_carton))

In [15]:

# Plasticos y polimeros
plasticos = ['CAUCHO Y MANUFACTURAS DE CAUCHO; CAMARAS DE CAUCHO',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; CAUCHO ENDURECIDO (POR EJEMPLO EBONITA) EN CUALQUIER FORMA;  INCLUIDOS LOS DES',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; CAUCHO MEZCLADO SIN VULCANIZAR;  EN FORMAS PRIMARIAS O EN PLACAS;  HOJAS O BAN',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; CAUCHO NATURAL;  BALATA;  GUTAPERCHA;  GUAYULE;  CHICLE Y GOMAS NATURALES ANAL',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; CAUCHO REGENERADO EN FORMAS PRIMARIAS O EN PLACAS;  HOJAS O BANDAS',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; CAUCHO SINTETICO Y CAUCHO FACTICIO DERIVADO DE LOS ACEITES;  EN FORMAS PRIMARI',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; CORREAS TRANSPORTADORAS O DE TRANSMISION;  DE CAUCHO VULCANIZADO',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; DESECHOS;  DESPERDICIOS Y RECORTES;  DE CAUCHO SIN ENDURECER;  INCLUSO EN POLV',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; LAS DEMAS FORMAS (POR EJEMPLO VARILLAS;  TUBOS O PERFILES) Y ARTICULOS (POR EJ',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; LAS DEMAS MANUFACTURAS DE CAUCHO VULCANIZADO SIN ENDURECER',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; NEUMATICOS RECAUCHUTADOS O USADOS;  DE CAUCHO; BANDAJES;  BANDAS DE RODADURA I',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; PLACAS;  HOJAS;  BANDAS;  VARILLAS Y PERFILES;  DE CAUCHO VULCANIZADO SIN ENDU',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; TUBOS DE CAUCHO VULCANIZADO SIN ENDURECER;  INCLUSO CON SUS ACCESORIOS (POR EJ',
 'CAUCHO\xa0Y\xa0MANUFACTURAS\xa0DE\xa0CAUCHO;\xa0CAMARAS\xa0DE\xa0CAUCHO',
 'CAUCHO\xa0Y\xa0MANUFACTURAS\xa0DE\xa0CAUCHO;\xa0CAUCHO\xa0ENDURECIDO\xa0(POR\xa0EJEMPLO\xa0EBONITA)\xa0EN\xa0CUALQUIER\xa0FORMA;\xa0\xa0INCLUIDOS\xa0LOS\xa0DES',
 'CAUCHO\xa0Y\xa0MANUFACTURAS\xa0DE\xa0CAUCHO;\xa0CAUCHO\xa0MEZCLADO\xa0SIN\xa0VULCANIZAR;\xa0\xa0EN\xa0FORMAS\xa0PRIMARIAS\xa0O\xa0EN\xa0PLACAS;\xa0\xa0HOJAS\xa0O\xa0BAN',
 'CAUCHO\xa0Y\xa0MANUFACTURAS\xa0DE\xa0CAUCHO;\xa0PRENDAS;\xa0\xa0GUANTES\xa0Y\xa0DEMAS\xa0COMPLEMENTOS\xa0DE\xa0VESTIR;\xa0\xa0PARA\xa0CUALQUIER\xa0USO;\xa0\xa0DE\xa0CAU',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; NEUMATICOS NUEVOS DE CAUCHO',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; ARTICULOS DE HIGIENE O DE FARMACIA (INCLUIDAS LAS TETINAS);  DE CAUCHO VULCANI',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; HILOS Y CUERDAS;  DE CAUCHO VULCANIZADO',
 'CAUCHO Y MANUFACTURAS DE CAUCHO; NEUMATICOS NUEVOS DE CAUCHO',
 'GUATA; FIELTRO Y TELAS SIN TEJER; HILADOS ESPECIALES; CORDELES; CUERDAS Y REDES DE MALLAS ANUDADAS;  EN PA?OS O EN PIEZAS;  FABRICADAS CON CORDELES;  CU',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; ARTICULOS PARA EL TRANSPORTE O ENVASADO;  DE PLASTICO; TAPONES;  TAPAS;  CAPSU',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; ARTICULOS PARA LA CONSTRUCCION;  DE PLASTICO;  NO EXPRESADOS NI COMPRENDIDOS E',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; BA?ERAS;  DUCHAS;  LAVABOS;  BIDES;  INODOROS Y SUS ASIENTOS Y TAPAS;  CISTERN',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; CELULOSA Y SUS DERIVADOS QUIMICOS;  NO EXPRESADOS NI COMPRENDIDOS EN OTRAS PAR',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; DESECHOS;  RECORTES Y DESPERDICIOS;  DE PLASTICO',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; INTERCAMBIADORES DE IONES A BASE DE POLIMEROS DE LAS PARTIDAS 3901 A 3913;  E',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; LAS DEMAS MANUFACTURAS DE PLASTICO Y MANUFACTURAS DE LAS DEMAS MATERIAS DE LAS P',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; LAS DEMAS PLACAS;  HOJAS;  PELICULAS;  BANDAS Y LAMINAS;  DE PLASTICO',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; LAS DEMAS PLACAS;  HOJAS;  PELICULAS;  BANDAS Y LAMINAS;  DE PLASTICO NO CELUL',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; MONOFILAMENTOS CUYA MAYOR DIMENSION DEL CORTE TRANSVERSAL SEA SUPERIOR A 1 MM',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; PLACAS;  HOJAS;  BANDAS;  CINTAS;  PELICULAS Y DEMAS FORMAS PLANAS;  AUTOADHES',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; POLIACETALES;  LOS DEMAS POLIETERES Y RESINAS EPOXI;  EN FORMAS PRIMARIAS; POL',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; POLIAMIDAS EN FORMAS PRIMARIAS',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; POLIMEROS ACRILICOS EN FORMAS PRIMARIAS',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; POLIMEROS DE ACETATO DE VINILO O DE OTROS ESTERES VINILICOS;  EN FORMAS PRIMAR',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; POLIMEROS DE CLORURO DE VINILO O DE OTRAS OLEFINAS HALOGENADAS;  EN FORMAS PRI',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; POLIMEROS DE ESTIRENO EN FORMAS PRIMARIAS',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; POLIMEROS DE ETILENO EN FORMAS PRIMARIAS',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; POLIMEROS DE PROPILENO O DE OTRAS OLEFINAS;  EN FORMAS PRIMARIAS',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; POLIMEROS NATURALES (POR EJEMPLO ACIDO ALGINICO) Y POLIMEROS NATURALES MODIFIC',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; RESINAS AMINICAS;  RESINAS FENOLICAS Y POLIURETANOS;  EN FORMAS PRIMARIAS',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; RESINAS DE PETROLEO;  RESINAS DE CUMARONA-INDENO;  POLITERPENOS;  POLISULFUROS',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; REVESTIMIENTOS DE PLASTICO PARA SUELOS;  INCLUSO AUTOADHESIVOS;  EN ROLLOS O L',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; SILICONAS EN FORMAS PRIMARIAS',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; TUBOS Y ACCESORIOS DE TUBERIA (POR EJEMPLO JUNTAS;  CODOS O RACORES);  DE PLAS',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; VAJILLA Y DEMAS ARTICULOS DE USO DOMESTICO Y ARTICULOS DE HIGIENE O DE TOCADOR',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; POLIMEROS DE ESTIRENO EN FORMAS PRIMARIAS',
 'MATERIAS PLÁSTICAS Y MANUFACTURAS DE ESTAS MATERIAS; VAJILLA Y DEMAS ARTICULOS DE USO DOMESTICO Y ARTICULOS DE HIGIENE O DE TOCADOR',
 ]
plasticos = sorted(set(plasticos))

In [16]:
# Paqueteo
paqueteo = ['OTROS MISCELANEOS CONTENIDOS EN PAQUETES ( PAQUETEO )',
 'OTROS TRAFICO POSTAL Y ENVIOS DE ENTREGA RAPIDA O MENSAJERIA EXPRESA']
 


In [17]:
# Quimicos
quimicos = ['EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y TINTAS DE IMPRENTA;  TINTAS PARA ESCRIBIR O DIBUJAR Y DEMAS TINTAS;  INCLUSO C',
 'EXTRACTOS\xa0CURTIENTES\xa0O\xa0TINTÓREOS;\xa0TANINOS\xa0Y\xa0SUS\xa0DERIVADOS;\xa0PIGMENTOS\xa0Y\xa0EXTRACTOS\xa0CURTIENTES\xa0DE\xa0ORIGEN\xa0VEGETAL;\xa0TANINOS\xa0Y\xa0SUS\xa0SALES;\xa0\xa0ETERES;\xa0\xa0ESTERE',
 'EXTRACTOS\xa0CURTIENTES\xa0O\xa0TINTÓREOS;\xa0TANINOS\xa0Y\xa0SUS\xa0DERIVADOS;\xa0PIGMENTOS\xa0Y\xa0LACAS\xa0COLORANTES;\xa0PREPARACIONES\xa0A\xa0QUE\xa0SE\xa0REFIERE\xa0LA\xa0NOTA\xa03\xa0DE\xa0ESTE\xa0CAPITULO\xa0A\xa0BA',
 'EXTRACTOS\xa0CURTIENTES\xa0O\xa0TINTÓREOS;\xa0TANINOS\xa0Y\xa0SUS\xa0DERIVADOS;\xa0PIGMENTOS\xa0Y\xa0PIGMENTOS\xa0(INCLUIDOS\xa0EL\xa0POLVO\xa0Y\xa0LAS\xa0LAMINILLASMETALICOS)\xa0DISPERSOS\xa0EN\xa0MEDIOS\xa0N',
'ABONOS; ABONOS DE ORIGEN ANIMAL O VEGETAL;  INCLUSO MEZCLADOS ENTRE SI O TRATADOS QUIM',
'JABONES;\xa0AGENTES\xa0DE\xa0SUPERFICIE\xa0ORGÁNICOS;\xa0PREPARACIONES\xa0PARA\xa0LAVAR;\xa0BETUNES\xa0Y\xa0CREMAS\xa0PARA\xa0EL\xa0CALZADO;\xa0\xa0ENCAUSTICOS;\xa0\xa0ABRILLANTADORES\xa0PARA\xa0CARROCER',
 'JABONES;\xa0AGENTES\xa0DE\xa0SUPERFICIE\xa0ORGÁNICOS;\xa0PREPARACIONES\xa0PARA\xa0LAVAR;\xa0CERAS\xa0ARTIFICIALES\xa0Y\xa0CERAS\xa0PREPARADAS',
 'JABONES;\xa0AGENTES\xa0DE\xa0SUPERFICIE\xa0ORGÁNICOS;\xa0PREPARACIONES\xa0PARA\xa0LAVAR;\xa0VELAS;\xa0\xa0CIRIOS\xa0Y\xa0ARTICULOS\xa0SIMILARES',
 'EXPLOSIVOS 0159 - 1.3C - GALLETA DE PÓLVORA HUMIDIFICADA con un mínimo del 25%; en masa; de',
 'EXPLOSIVOS 0249 - 1.3L - DISPOSITIVOS ACTIVADOS POR EL AGUA; con carga dispersora; carga ex',
'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y PIGMENTOS;  OPACIFICANTES Y COLORES PREPARADOS; COMPOSICIONES VITRIFICABLES;',
 'ABONOS; ABONOS MINERALES O QUIMICOS FOSFATADOS',
 'ABONOS; ABONOS MINERALES O QUIMICOS NITROGENADOS',
 'ABONOS; ABONOS MINERALES O QUIMICOS POTASICOS',
 'ABONOS; ABONOS MINERALES O QUIMICOS;  CON DOS O TRES DE LOS ELEMENTOS FERTILIZANTES NI',
 'ABONOS;\xa0ABONOS\xa0MINERALES\xa0O\xa0QUIMICOS\xa0POTASICOS',
 'ACEITES ESENCIALES Y RESINOIDES; PRODUCTOS PREPARADOS Y PREPARACIONES; ACEITES ESENCIALES (DESTERPENADOS O NO);  INCLUIDOS LOS <CONCRETOS> O <ABSOLUT',
 'ACEITES ESENCIALES Y RESINOIDES; PRODUCTOS PREPARADOS Y PREPARACIONES; MEZCLAS DE SUSTANCIAS ODORIFERAS Y MEZCLAS A BASE DE UNA O VARIAS SUSTANCIAS',
 'ACEITES ESENCIALES Y RESINOIDES; PRODUCTOS PREPARADOS Y PREPARACIONES; PERFUMES Y AGUAS DE TOCADOR',
 'ACEITES ESENCIALES Y RESINOIDES; PRODUCTOS PREPARADOS Y PREPARACIONES; PREPARACIONES CAPILARES',
 'ACEITES ESENCIALES Y RESINOIDES; PRODUCTOS PREPARADOS Y PREPARACIONES; PREPARACIONES DE BELLEZA;  DE MAQUILLAJE Y PARA EL CUIDADO DE LA PIEL;  EXCEPT',
 'ACEITES ESENCIALES Y RESINOIDES; PRODUCTOS PREPARADOS Y PREPARACIONES; PREPARACIONES PARA AFEITAR O PARA ANTES O DESPUES DEL AFEITADO;  DESODORANTES',
 'ACEITES ESENCIALES Y RESINOIDES; PRODUCTOS PREPARADOS Y PREPARACIONES; PREPARACIONES PARA LA HIGIENE BUCAL O DENTAL;  INCLUIDOS LOS POLVOS Y CREMAS P',
 'EXPLOSIVOS 0004 - 1.1D - PICRATO AMÓNICO seco o humidificado con menos del 10%; en masa; de',
 'EXPLOSIVOS 0015 - 1.2G - MUNICIONES FUMÍGENAS con o sin carga dispersora; carga expulsora o',
 'EXPLOSIVOS 0021 - 1.3K - MUNICIONES TÓXICAS con carga dispersora; carga expulsora o carga p',
 'EXPLOSIVOS 0027 - 1.1D - PÓLVORA NEGRA (PÓLVORA DE CAÑÓN) en forma de granos o polvo.',
 'EXPLOSIVOS 0029 - 1.1B - DETONADORES NO ELÉCTRICOS para voladuras.',
 'EXPLOSIVOS 0030 - 1.1B - DETONADORES ELÉCTRICOS para voladuras.',
 'EXPLOSIVOS 0033 - 1.1F - BOMBAS con carga explosiva.',
 'EXPLOSIVOS 0042 - 1.1D - PETARDOS MULTIPLICADORES (CARTUCHOS MULTIPLICADORES) sin detonador',
 'EXPLOSIVOS 0043 - 1.1D - CARGAS DISPERSORAS.',
 'EXPLOSIVOS 0048 - 1.1D - CARGAS DE DEMOLICIÓN.',
 'EXPLOSIVOS 0049 - 1.1G - CARTUCHOS FULGURANTES.',
 'EXPLOSIVOS 0121 - 1.1G - INFLAMADORES.',
 'EXPLOSIVOS 0155 - 1.1D - TRINITROCLOROBENCENO (CLORURO DE PICRILO).',
 'EXPLOSIVOS 0222 - 1.1D - NITRATO AMÓNICO con más',
 'EXPLOSIVOS 0277 - 1.3C - CARTUCHOS DE PERFORACIÓN DE POZOS PETROLÍFEROS.',
 'EXPLOSIVOS 0315 - 1.3G - INFLAMADORES.',
 'EXPLOSIVOS 0331 - 1.5D - EXPLOSIVO PARA VOLADURAS; TIPO B (AGENTE PARA VOLADURAS; TIPO B).',
 'EXPLOSIVOS 0334 - 1.2G - ARTIFICIOS DE PIROTECNIA.',
 'EXPLOSIVOS 0335 - 1.3G - ARTIFICIOS DE PIROTECNIA.',
 'EXPLOSIVOS 0340 - 1.1D - NITROCELULOSA seca o humidificada con menos del 25%; en masa; de a',
 'EXPLOSIVOS 0341 - 1.1D - NITROCELULOSA no modificada o plastificada con menos del 18%; en m',
 'EXPLOSIVOS 0343 - 1.3C - NITROCELULOSA PLASTIFICADA con un mínimo del 18%; en masa; de plas',
 'EXPLOSIVOS 0376 - 1.4S - CEBOS TUBULARES.',
 'EXPLOSIVOS 0383 - 1.4B - COMPONENTES DE CADENAS DE EXPLOSIVOS; N.E.P..',
 'EXPLOSIVOS 0399 - 1.1J - BOMBAS QUE CONTIENEN UN LÍQUIDO INFLAMABLE; con carga explosiva.',
 'EXPLOSIVOS 0403 - 1.4G - BENGALAS AÉREAS.',
 'EXPLOSIVOS 0433 - 1.1C - GALLETA DE PÓLVORA HUMIDIFICADA con un mínimo del 17%; en masa; de',
 'EXPLOSIVOS 0435 - 1.4G - PROYECTILES con carga dispersora o carga expulsora.',
 'EXPLOSIVOS 0455 - 1.4S - DETONADORES NO ELÉCTRICOS para voladuras.',
 'EXPLOSIVOS 0482 - 1.5D - SUSTANCIAS EXPLOSIVAS MUY INSENSIBLES (SUSTANCIAS EMI); N.E.P..',
 'EXPLOSIVOS 0484 - 1.1D - CICLOTETRAMETILEN-TETRANITRAMINA (OCTÓGENO; HMX) DESENSIBILIZADA',
 'EXPLOSIVOS 0503 - 1.4G - INFLADORES DE BOLSAS NEUMÁTICAS o MÓDULOS DE BOLSAS NEUMÁTICAS o P',
 'EXPLOSIVOS\xa00015\xa0-\xa01.2G\xa0-\xa0MUNICIONES\xa0FUMÍGENAS\xa0con\xa0o\xa0sin\xa0carga\xa0dispersora;\xa0carga\xa0expulsora\xa0o',
 'EXPLOSIVOS\xa00030\xa0-\xa01.1B\xa0-\xa0DETONADORES\xa0ELÉCTRICOS\xa0para\xa0voladuras.',
 'EXPLOSIVOS\xa00155\xa0-\xa01.1D\xa0-\xa0TRINITROCLOROBENCENO\xa0(CLORURO\xa0DE\xa0PICRILO).',
 'EXPLOSIVOS\xa00482\xa0-\xa01.5D\xa0-\xa0SUSTANCIAS\xa0EXPLOSIVAS\xa0MUY\xa0INSENSIBLES\xa0(SUSTANCIAS\xa0EMI);\xa0N.E.P..',
 'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y COLORES PARA LA PINTURA ARTISTICA;  LA ENSE?ANZA;  LA PINTURA DE LETREROS;  PA',
 'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y EXTRACTOS CURTIENTES DE ORIGEN VEGETAL; TANINOS Y SUS SALES;  ETERES;  ESTERE',
 'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y LACAS COLORANTES; PREPARACIONES A QUE SE REFIERE LA NOTA 3 DE ESTE CAPITULO A BA',
 'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y LAS DEMAS MATERIAS COLORANTES; PREPARACIONES A QUE SE REFIERE LA NOTA 3 DE EST',
 'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y LAS DEMAS PINTURAS Y BARNICES; PIGMENTOS AL AGUA PREPARADOS DEL TIPO DE LOS UTIL',
 'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y MATERIAS COLORANTES DE ORIGEN VEGETAL O ANIMAL(INCLUIDOS LOS EXTRACTOS TINTORE',
 'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y MATERIAS COLORANTES ORGANICAS SINTETICAS;  AUNQUE SEAN DE CONSTITUCION QUIMIC',
 'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y PIGMENTOS (INCLUIDOS EL POLVO Y LAS LAMINILLASMETALICOS) DISPERSOS EN MEDIOS N',
 'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y PINTURAS Y BARNICES A BASE DE POLIMEROS SINTETICOS O NATURALES MODIFICADOS;  D',
 'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y PRODUCTOS CURTIENTES ORGANICOS SINTETICOS; PRODUCTOS CURTIENTES INORGANICOS; P',
 'EXTRACTOS CURTIENTES O TINTÓREOS; TANINOS Y SUS DERIVADOS; PIGMENTOS Y SECATIVOS PREPARADOS',
 'FILAMENTOS SINTÉTICOS O ARTIFICIALES; MONOFILAMENTOS ARTIFICIALES DE 67 DECITEX O MAS Y CUYA MAYOR DIMENSION DE LA S',
 'GASES 1001 - 2.1 - ACETILENO DISUELTO',
 'GASES 1002 - 2.2 - AIRE COMPRIMIDO',
 'GASES 1005 - 2.3 - AMONIACO; ANHIDRO',
 'GASES 1012 - 2.1 - BUTILENO',
 'GASES 1017 - 2.3 - CLORO',
 'GASES 1021 - 2.2 - 1-CLORO-1;2;2;2 TETRAFLUOROETANO (GAS REFRIGERANTE R\xa0124)',
 'GASES 1023 - 2.3 - GAS DE HULLA COMPRIMIDO',
 'GASES 1040 - 2.3 - ÓXIDO DE ETILENO u ÓXIDO DE ETILENO CON NITRÓGENO hasta una presión',
 'GASES 1043 - 2.2 - SOLUCIÓN AMONIACAL FERTILIZANTE que contiene amoníaco libre',
 'GASES 1044 - 2.2 - EXTINTORES DE INCENDIOS que contienen gases comprimidos o licuados',
 'GASES 1046 - 2.2 - HELIO COMPRIMIDO',
 'GASES 1049 - 2.1 - HIDRÓGENO COMPRIMIDO',
 'GASES 1057 - 2.1 - ENCENDEDORES o RECARGAS DE ENCENDEDORES que contienen gas inflamabl',
 'GASES 1063 - 2.1 - CLORURO DE METILO (GAS REFRIGERANTE R\xa040)',
 'GASES 1066 - 2.2 - NITRÓGENO COMPRIMIDO',
 'GASES 1073 - 2.2 - OXÍGENO LÍQUIDO REFRIGERADO',
 'GASES 1075 - 2.1 - GASES DE PETRÓLEO; LICUADOS',
 'GASES 1077 - 2.1 - PROPILENO',
 'GASES 1078 - 2.2 - GAS REFRIGERANTE; N.E.P.',
 'GASES 1912 - 2.1 - MEZCLAS DE CLORURO DE METILO Y CLORURO DE METILENO',
 'GASES 1950 - 2 - AEROSOLES',
 'GASES 1951 - 2.2 - ARGON LÍQUIDO REFRIGERADO',
 'GASES 1954 - 2.1 - GAS COMPRIMIDO INFLAMABLE; N.E.P.',
 'GASES 1962 - 2.1 - ETILENO',
 'GASES 1963 - 2.2 - HELIO LÍQUIDO REFRIGERADO',
 'GASES 1967 - 2.3 - INSECTICIDA GASEOSO TÓXICO; N.E.P.',
 'GASES 1968 - 2.2 - INSECTICIDA GASEOSO; N.E.P.',
 'GASES 1971 - 2.1 - METANO COMPRIMIDO o GAS NATURAL COMPRIMIDO con alta proporción de m',
 'GASES 1974 - 2.2 - CLORODIFLUOROBROMO - METANO (GAS REFRIGERANTE R\xa012B1)',
 'GASES 1976 - 2.2 - OCTAFLUOROCICLOBUTANO (GAS REFRIGERANTE RC\xa0318)',
 'GASES 1977 - 2.2 - NITRÓGENO LÍQUIDO REFRIGERADO',
 'GASES 2187 - 2.2 - DIÓXIDO DE CARBONO LÍQUIDO REFRIGERADO',
 'GASES 2202 - 2.3 - SELENIURO DE HIDRÓGENO ANHIDRO',
 'GASES 2203 - 2.1 - SILANO',
 'GASES 2420 - 2.3 - HEXAFLUOROACETONA',
 'GASES 2421 - 2.3 - TRIÓXIDO DE NITRÓGENO',
 'GASES 2422 - 2.2 - 2-OCTAFLUOROBUTENO (GAS REFRIGERANTE R\xa01318)',
 'GASES 2424 - 2.2 - OCTAFLUOROPROPANO (GAS REFRIGERANTE R\xa0218)',
 'GASES 2451 - 2.2 - TRIFLUORURO DE NITRÓGENO',
 'GASES 2452 - 2.1 - ETILACETILENO ESTABILIZADO',
 'GASES 2453 - 2.1 - FLUORURO DE ETILO (GAS REFRIGERANTE R\xa0161)',
 'GASES 2454 - 2.1 - FLUORURO DE METILO (GAS REFRIGERANTE R\xa041)',
 'GASES 2455 - 2.2 - NITRITO DE METILO',
 'GASES 2857 - 2.2 - MÁQUINAS REFRIGERADORAS que contengan gases  no inflamables ni tóxi',
 'GASES 3156 - 2.2 - GAS COMPRIMIDO; COMBURENTE; N.E.P.',
 'GASES 3159 - 2.2 - 1;1;1;2-TETRAFLUOROETANO (GAS REFRIGERANTE R 134a)',
 'GASES 3164 - 2.2 - OBJETOS CON PRESIÓN INTERIOR; NEUMÁTICOS o HIDRAÚLICOS (que contien',
 'GASES 3296 - 2.2 - HEPTAFLUOROPROPANO (GAS REFRIGERANTE R 227)',
 'GASES 3303 - 2.3 - GAS COMPRIMIDO; TÓXICO; OXIDANTE; N.E.P.',
 'GASES 3304 - 2.3 - GAS COMPRIMIDO; TÓXICO; CORROSIVO; N.E.P.',
 'GASES 3307 - 2.3 - GAS LICUADO; TÓXICO; OXIDANTE; N.E.P.',
 'GASES 3354 - 2.1 - GAS INSECTICIDA; INFLAMABLE; N.E.P.',
 'GASES 3358 - 2.1 - MÁQUINAS REFRIGERADORAS que contengan gas líquido inflamable; no tó',
 'GASES 3374 - 2.1 - ACETILENO EXENTO DE SOLVENTE',
 'GASES 3468 - 2.1 - HIDRÓGENO EN UN DISPOSITIVO DE ALMACENAMIENTO CON HIDRURO METÁLICO',
 'GASES 3500 - 2.2 - PRODUCTO QUÍMICO A PRESIÓN; N.E.P.',
 'GASES\xa01002\xa0-\xa02.2\xa0-\xa0AIRE\xa0COMPRIMIDO',
 'GASES\xa01021\xa0-\xa02.2\xa0-\xa01-CLORO-1;2;2;2\xa0TETRAFLUOROETANO\xa0(GAS\xa0REFRIGERANTE\xa0R\xa0124)',
 'GASES\xa01044\xa0-\xa02.2\xa0-\xa0EXTINTORES\xa0DE\xa0INCENDIOS\xa0que\xa0contienen\xa0gases\xa0comprimidos\xa0o\xa0licuados',
 'GASES\xa01058\xa0-\xa02.2\xa0-\xa0MEZCLAS\xa0DE\xa0GASES\xa0LICUADOS\xa0ininflamables\xa0con\xa0nitrógeno;\xa0dióxido\xa0de\xa0c',
 'GASES\xa01950\xa0-\xa02\xa0-\xa0AEROSOLES',
 'GASES\xa01962\xa0-\xa02.1\xa0-\xa0ETILENO',
 'GASES\xa01963\xa0-\xa02.2\xa0-\xa0HELIO\xa0LÍQUIDO\xa0REFRIGERADO',
 'GASES\xa02420\xa0-\xa02.3\xa0-\xa0HEXAFLUOROACETONA',
 'GASES\xa02421\xa0-\xa02.3\xa0-\xa0TRIÓXIDO\xa0DE\xa0NITRÓGENO',
 'GASES\xa02422\xa0-\xa02.2\xa0-\xa02-OCTAFLUOROBUTENO\xa0(GAS\xa0REFRIGERANTE\xa0R\xa01318)',
 'GASES\xa02424\xa0-\xa02.2\xa0-\xa0OCTAFLUOROPROPANO\xa0(GAS\xa0REFRIGERANTE\xa0R\xa0218)',
 'GASES\xa03159\xa0-\xa02.2\xa0-\xa01;1;1;2-TETRAFLUOROETANO\xa0(GAS\xa0REFRIGERANTE\xa0R\xa0134a)',
 'JABONES; AGENTES DE SUPERFICIE ORGÁNICOS; PREPARACIONES PARA LAVAR; AGENTES DE SUPERFICIE ORGANICOS (EXCEPTO EL JABON); PREPARACIONES TENSOACTIVAS',
 'JABONES; AGENTES DE SUPERFICIE ORGÁNICOS; PREPARACIONES PARA LAVAR; BETUNES Y CREMAS PARA EL CALZADO;  ENCAUSTICOS;  ABRILLANTADORES PARA CARROCER',
 'JABONES; AGENTES DE SUPERFICIE ORGÁNICOS; PREPARACIONES PARA LAVAR; CERAS ARTIFICIALES Y CERAS PREPARADAS',
 'JABONES; AGENTES DE SUPERFICIE ORGÁNICOS; PREPARACIONES PARA LAVAR; JABON; PRODUCTOS TENSOACTIVOS USADOS COMO JABON;  EN BARRAS;  PANES O TROZOS',
 'JABONES; AGENTES DE SUPERFICIE ORGÁNICOS; PREPARACIONES PARA LAVAR; PASTAS PARA MODELAR;  INCLUIDAS LAS PRESENTADAS PARA ENTRETENIMIENTO DE LOS NI',
 'JABONES; AGENTES DE SUPERFICIE ORGÁNICOS; PREPARACIONES PARA LAVAR; PREPARACIONES LUBRICANTES Y PREPARACIONES DEL TIPO DE LAS UTILIZADAS PARA EL ENS',
 'JABONES; AGENTES DE SUPERFICIE ORGÁNICOS; PREPARACIONES PARA LAVAR; VELAS;  CIRIOS Y ARTICULOS SIMILARES',
 'LÍQUIDOS INFLAMABLES 1088 - 3 - ACETAL',
 'LÍQUIDOS INFLAMABLES 1089 - 3 - ACETALDEHIDO',
 'LÍQUIDOS INFLAMABLES 1090 - 3 - ACETONA',
 'LÍQUIDOS INFLAMABLES 1100 - 3 - CLORURO DE ALILO',
 'LÍQUIDOS INFLAMABLES 1105 - 3 - PENTANOLES',
 'LÍQUIDOS INFLAMABLES 1110 - 3 - n-AMILMETILCETONA',
 'LÍQUIDOS INFLAMABLES 1114 - 3 - BENCENO',
 'LÍQUIDOS INFLAMABLES 1120 - 3 - BUTANOLES',
 'LÍQUIDOS INFLAMABLES 1123 - 3 - ACETATOS DE BUTILO',
 'LÍQUIDOS INFLAMABLES 1128 - 3 - FORMIATO DE n-BUTILO',
 'LÍQUIDOS INFLAMABLES 1130 - 3 - ACEITE DE ALCANFOR',
 'LÍQUIDOS INFLAMABLES 1133 - 3 - ADHESIVOS que contienen líquidos inflamables',
 'LÍQUIDOS INFLAMABLES 1139 - 3 - SOLUCIONES PARA REVESTIMIENTOS (comprende los tratamientos de superfi',
 'LÍQUIDOS INFLAMABLES 1145 - 3 - CICLOHEXANO',
 'LÍQUIDOS INFLAMABLES 1155 - 3 - ÉTER DIETÍLICO (ÉTER ETÍLICO)',
 'LÍQUIDOS INFLAMABLES 1161 - 3 - CARBONATO DE METILO',
 'LÍQUIDOS INFLAMABLES 1166 - 3 - DIOXOLANO',
 'LÍQUIDOS INFLAMABLES 1169 - 3 - EXTRACTOS AROMÁTICOS LÍQUIDOS',
 'LÍQUIDOS INFLAMABLES 1170 - 3 - ETANOL (ALCOHOL ETÍLICO) o ETANOL EN SOLUCIÓN (ALCOHOL ETÍLICO EN SOL',
 'LÍQUIDOS INFLAMABLES 1171 - 3 - ÉTER MONOETÍLICO DEL ETILENGLICOL',
 'LÍQUIDOS INFLAMABLES 1172 - 3 - ACETATO DEL ÉTER MONOETÍLICO DEL ETILENGLICOL',
 'LÍQUIDOS INFLAMABLES 1173 - 3 - ACETATO DE ETILO',
 'LÍQUIDOS INFLAMABLES 1177 - 3 - ACETATO DE 2-ETILBUTILO',
 'LÍQUIDOS INFLAMABLES 1180 - 3 - BUTIRATO DE ETILO',
 'LÍQUIDOS INFLAMABLES 1193 - 3 - ETILMETILCETONA (METILETILCETONA)',
 'LÍQUIDOS INFLAMABLES 1195 - 3 - PROPIONATO DE ETILO',
 'LÍQUIDOS INFLAMABLES 1197 - 3 - EXTRACTOS SAPORÍFEROS LÍQUIDOS',
 'LÍQUIDOS INFLAMABLES 1198 - 3 - FORMALDEHIDO EN SOLUCIÓN INFLAMABLE',
 'LÍQUIDOS INFLAMABLES 1201 - 3 - ACEITE DE FUSEL',
 'LÍQUIDOS INFLAMABLES 1204 - 3 - NITROGLICERINA EN SOLUCIÓN ALCOHÓLICA con un máximo del 1% de nitrogl',
 'LÍQUIDOS INFLAMABLES 1208 - 3 - HEXANOS',
 'LÍQUIDOS INFLAMABLES 1210 - 3 - TINTA DE IMPRENTA; inflamable o MATERIALES RELACIONADOS CON LA TINTA',
 'LÍQUIDOS INFLAMABLES 1213 - 3 - ACETATO DE ISOBUTILO',
 'LÍQUIDOS INFLAMABLES 1219 - 3 - ISOPROPANOL (ALCOHOL ISOPROPÍLICO)',
 'LÍQUIDOS INFLAMABLES 1220 - 3 - ACETATO DE ISOPROPILO',
 'LÍQUIDOS INFLAMABLES 1223 - 3 - QUEROSENO',
 'LÍQUIDOS INFLAMABLES 1224 - 3 - CETONAS LÍQUIDAS; N.E.P.',
 'LÍQUIDOS INFLAMABLES 1228 - 3 - MERCAPTANOS LÍQUIDOS; INFLAMABLES; TÓXICOS; N.E.P. o MEZCLA DE MERCAP',
 'LÍQUIDOS INFLAMABLES 1230 - 3 - METANOL',
 'LÍQUIDOS INFLAMABLES 1231 - 3 - ACETATO DE METILO',
 'LÍQUIDOS INFLAMABLES 1234 - 3 - METILAL (DIMETOXIMETANO; FORMAL)',
 'LÍQUIDOS INFLAMABLES 1237 - 3 - BUTIRATO DE METILO',
 'LÍQUIDOS INFLAMABLES 1245 - 3 - METILISOBUTILCETONA',
 'LÍQUIDOS INFLAMABLES 1247 - 3 - METACRILATO DE METILO MONÓMERO ESTABILIZADO',
 'LÍQUIDOS INFLAMABLES 1250 - 3 - METILTRICLOROSILANO',
 'LÍQUIDOS INFLAMABLES 1262 - 3 - OCTANOS',
 'LÍQUIDOS INFLAMABLES 1266 - 3 - PRODUCTOS DE PERFUMERÍA que contengan disolventes inflamables',
 'LÍQUIDOS INFLAMABLES 1267 - 3 - PETRÓLEO BRUTO',
 'LÍQUIDOS INFLAMABLES 1268 - 3 - DESTILADOS DE PETRÓLEO; N.E.P. o PRODUCTOS DE PETRÓLEO; N.E.P.',
 'LÍQUIDOS INFLAMABLES 1272 - 3 - ACEITE DE PINO',
 'LÍQUIDOS INFLAMABLES 1274 - 3 - n-PROPANOL (ALCOHOL PROPÍLICO NORMAL)',
 'LÍQUIDOS INFLAMABLES 1275 - 3 - PROPIONALDEHIDO',
 'LÍQUIDOS INFLAMABLES 1276 - 3 - ACETATO DE n-PROPILO',
 'LÍQUIDOS INFLAMABLES 1277 - 3 - PROPILAMINA',
 'LÍQUIDOS INFLAMABLES 1286 - 3 - ACEITE DE COLOFONIA',
 'LÍQUIDOS INFLAMABLES 1288 - 3 - ACEITE DE ESQUISTO',
 'LÍQUIDOS INFLAMABLES 1294 - 3 - TOLUENO',
 'LÍQUIDOS INFLAMABLES 1296 - 3 - TRIETILAMINA',
 'LÍQUIDOS INFLAMABLES 1299 - 3 - TREMENTINA',
 'LÍQUIDOS INFLAMABLES 1301 - 3 - ACETATO DE VINILO ESTABILIZADO',
 'LÍQUIDOS INFLAMABLES 1302 - 3 - VINIL ETIL ÉTER ESTABILIZADO',
 'LÍQUIDOS INFLAMABLES 1306 - 3 - PRODUCTOS LÍQUIDOS PARA LA CONSERVACIÓN DE LA MADERA',
 'LÍQUIDOS INFLAMABLES 1307 - 3 - XILENOS',
 'LÍQUIDOS INFLAMABLES 1717 - 3 - CLORURO DE ACETILO',
 'LÍQUIDOS INFLAMABLES 1914 - 3 - PROPIONATOS DE BUTILO',
 'LÍQUIDOS INFLAMABLES 1915 - 3 - CICLOHEXANONA',
 'LÍQUIDOS INFLAMABLES 1918 - 3 - ISOPROPILBENCENO',
 'LÍQUIDOS INFLAMABLES 1919 - 3 - ACRILATO DE METILO ESTABILIZADO',
 'LÍQUIDOS INFLAMABLES 1986 - 3 - ALCOHOLES TÓXICOS INFLAMABLES; N.E.P.',
 'LÍQUIDOS INFLAMABLES 1987 - 3 - ALCOHOLES; N.E.P.',
 'LÍQUIDOS INFLAMABLES 1992 - 3 - LÍQUIDO INFLAMABLE; TÓXICO; N.E.P.',
 'LÍQUIDOS INFLAMABLES 1993 - 3 - LÍQUIDO INFLAMABLE; N.E.P.',
 'LÍQUIDOS INFLAMABLES 1999 - 3 - ALQUITRANES LÍQUIDOS; incluso los aglomerantes para carreteras y los',
 'LÍQUIDOS INFLAMABLES 2055 - 3 - ESTIRENO MONÓMERO ESTABILIZADO',
 'LÍQUIDOS INFLAMABLES 2056 - 3 - TETRAHIDROFURANO',
 'LÍQUIDOS INFLAMABLES 2227 - 3 - METACRILATO DE n-BUTILO ESTABILIZADO',
 'LÍQUIDOS INFLAMABLES 2271 - 3 - ETILAMILCETONA',
 'LÍQUIDOS INFLAMABLES 2283 - 3 - METACRILATO DE ISOBUTILO ESTABILIZADO',
 'LÍQUIDOS INFLAMABLES 2293 - 3 - 4-METOXI-4-METIL-2-PENTANONA',
 'LÍQUIDOS INFLAMABLES 2348 - 3 - ACRILATOS DE BUTILO ESTABILIZADOS',
 'LÍQUIDOS INFLAMABLES 2350 - 3 - BUTIL METIL ÉTER',
 'LÍQUIDOS INFLAMABLES 2352 - 3 - BUTIL VINIL ÉTER ESTABILIZADO',
 'LÍQUIDOS INFLAMABLES 2356 - 3 - 2-CLOROPROPANO',
 'LÍQUIDOS INFLAMABLES 2358 - 3 - CICLOOCTATETRAENO',
 'LÍQUIDOS INFLAMABLES 2359 - 3 - DIALILAMINA',
 'LÍQUIDOS INFLAMABLES 2360 - 3 - ÉTER DIALÍLICO',
 'LÍQUIDOS INFLAMABLES 2361 - 3 - DIISOBUTILAMINA',
 'LÍQUIDOS INFLAMABLES 2362 - 3 - 1;1-DICLOROETANO',
 'LÍQUIDOS INFLAMABLES 2363 - 3 - ETILMERCAPTANO',
 'LÍQUIDOS INFLAMABLES 2364 - 3 - n-PROPILBENCENO',
 'LÍQUIDOS INFLAMABLES 2366 - 3 - CARBONATO DE DIETILO',
 'LÍQUIDOS INFLAMABLES 2367 - 3 - alfa-METILVALERALDEHIDO',
 'LÍQUIDOS INFLAMABLES 2368 - 3 - alfa-PINENO',
 'LÍQUIDOS INFLAMABLES 2371 - 3 - ISOPENTENOS',
 'LÍQUIDOS INFLAMABLES 2372 - 3 - 1;2-DI-(DIMETILAMINO) ETANO',
 'LÍQUIDOS INFLAMABLES 2373 - 3 - DIETOXIMETANO',
 'LÍQUIDOS INFLAMABLES 2374 - 3 - 3;3-DIETOXIPROPENO',
 'LÍQUIDOS INFLAMABLES 2375 - 3 - SULFURO DE DIETILO',
 'LÍQUIDOS INFLAMABLES 2376 - 3 - 2;3-DIHIDROPIRANO',
 'LÍQUIDOS INFLAMABLES 2377 - 3 - 1;1-DIMETOXIETANO',
 'LÍQUIDOS INFLAMABLES 2378 - 3 - 2-DIMETILAMINOACETO-NITRILO',
 'LÍQUIDOS INFLAMABLES 2379 - 3 - 1;3-DIMETILBUTILAMINA',
 'LÍQUIDOS INFLAMABLES 2380 - 3 - DIMETILDIETOXISILANO',
 'LÍQUIDOS INFLAMABLES 2381 - 3 - DISULFURO DE DIMETILO',
 'LÍQUIDOS INFLAMABLES 2383 - 3 - DIPROPILAMINA',
 'LÍQUIDOS INFLAMABLES 2384 - 3 - ÉTER DI-n-PROPÍLICO',
 'LÍQUIDOS INFLAMABLES 2385 - 3 - ISOBUTIRATO DE ETILO',
 'LÍQUIDOS INFLAMABLES 2386 - 3 - 1-ETILPIPERIDINA',
 'LÍQUIDOS INFLAMABLES 2387 - 3 - FLUOROBENCENO',
 'LÍQUIDOS INFLAMABLES 2388 - 3 - FLUOROTOLUENOS',
 'LÍQUIDOS INFLAMABLES 2389 - 3 - FURANO',
 'LÍQUIDOS INFLAMABLES 2390 - 3 - 2-YODOBUTANO',
 'LÍQUIDOS INFLAMABLES 2391 - 3 - YODOMETILPROPANOS',
 'LÍQUIDOS INFLAMABLES 2392 - 3 - YODOPROPANOS',
 'LÍQUIDOS INFLAMABLES 2393 - 3 - FORMIATO DE ISOBUTILO',
 'LÍQUIDOS INFLAMABLES 2395 - 3 - CLORURO DE ISOBUTIRILO',
 'LÍQUIDOS INFLAMABLES 2396 - 3 - METACRILALDEHIDO ESTABILIZADO',
 'LÍQUIDOS INFLAMABLES 2397 - 3 - 3-METIL-2-BUTANONA',
 'LÍQUIDOS INFLAMABLES 2398 - 3 - METIL-terc-BUTILÉTER',
 'LÍQUIDOS INFLAMABLES 2399 - 3 - 1-METILPIPERIDINA',
 'LÍQUIDOS INFLAMABLES 2400 - 3 - ISOVALERIANATO DE METILO',
 'LÍQUIDOS INFLAMABLES 2402 - 3 - PROPANOTIOLES',
 'LÍQUIDOS INFLAMABLES 2403 - 3 - ACETATO DE ISOPROPENILO',
 'LÍQUIDOS INFLAMABLES 2404 - 3 - PROPIONITRILO',
 'LÍQUIDOS INFLAMABLES 2405 - 3 - BUTIRATO DE ISOPROPILO',
 'LÍQUIDOS INFLAMABLES 2406 - 3 - ISOBUTIRATO DE ISOPROPILO',
 'LÍQUIDOS INFLAMABLES 2409 - 3 - PROPIONATO DE ISOPROPILO',
 'LÍQUIDOS INFLAMABLES 2410 - 3 - 1;2;3;6-TETRAHIDROPIRIDINA',
 'LÍQUIDOS INFLAMABLES 2411 - 3 - BUTIRONITRILO',
 'LÍQUIDOS INFLAMABLES 2412 - 3 - TETRAHIDROTIOFENO',
 'LÍQUIDOS INFLAMABLES 2413 - 3 - ORTOTITANATO TETRAPROPÍLICO',
 'LÍQUIDOS INFLAMABLES 2414 - 3 - TIOFENO',
 'LÍQUIDOS INFLAMABLES 2416 - 3 - BORATO DE TRIMETILO',
 'LÍQUIDOS INFLAMABLES 2436 - 3 - ÁCIDO TIOACÉTICO',
 'LÍQUIDOS INFLAMABLES 2456 - 3 - 2-CLOROPROPENO',
 'LÍQUIDOS INFLAMABLES 2457 - 3 - 2;3-DIMETILBUTANO',
 'LÍQUIDOS INFLAMABLES 2458 - 3 - HEXADIENO',
 'LÍQUIDOS INFLAMABLES 2459 - 3 - 2-METIL-1-BUTENO',
 'LÍQUIDOS INFLAMABLES 2460 - 3 - 2-METIL-2-BUTENO',
 'LÍQUIDOS INFLAMABLES 2461 - 3 - METILPENTADIENO',
 'LÍQUIDOS INFLAMABLES 2478 - 3 - ISOCIANATOS INFLAMABLES; TÓXICOS; N.E.P. o ISOCIANATOS EN SOLUCIÓN; I',
 'LÍQUIDOS INFLAMABLES 2554 - 3 - CLORURO DE METILALILO',
 'LÍQUIDOS INFLAMABLES 2603 - 3 - CICLOHEPTATRIENO',
 'LÍQUIDOS INFLAMABLES 2621 - 3 - ACETILMETILCARBINOL',
 'LÍQUIDOS INFLAMABLES 2684 - 3 - 3-DIETILAMINOPROPILAMINA',
 'LÍQUIDOS INFLAMABLES 2709 - 3 - BUTILBENCENOS',
 'LÍQUIDOS INFLAMABLES 2710 - 3 - DIPROPILCETONA',
 'LÍQUIDOS INFLAMABLES 2733 - 3 - AMINAS INFLAMABLES; CORROSIVAS; N.E.P. o POLIAMINAS INFLAMABLES; CORR',
 'LÍQUIDOS INFLAMABLES 2758 - 3 - PLAGUICIDA A BASE DE CARBAMATO; LÍQUIDO; INFLAMABLE; TÓXICO; de punto',
 'LÍQUIDOS INFLAMABLES 2924 - 3 - LÍQUIDO INFLAMABLE; CORROSIVO; N.E.P.',
 'LÍQUIDOS INFLAMABLES 2933 - 3 - 2-CLOROPROPIONATO DE METILO',
 'LÍQUIDOS INFLAMABLES 2935 - 3 - 2-CLOROPROPIONATO DE ETILO',
 'LÍQUIDOS INFLAMABLES 2985 - 3 - CLOROSILANOS; INFLAMABLES; CORROSIVOS; N.E.P.',
 'LÍQUIDOS INFLAMABLES 3064 - 3 - NITROGLICERINA EN SOLUCIÓN ALCOHÓLICA con más del 1% pero no más del',
 'LÍQUIDOS INFLAMABLES 3065 - 3 - BEBIDAS ALCOHÓLICAS; con más del 70% de alcohol en volumen',
 'LÍQUIDOS INFLAMABLES 3092 - 3 - 1-METOXI-2-PROPANOL',
 'LÍQUIDOS INFLAMABLES 3165 - 3 - DEPÓSITO DE COMBUSTIBLE DE GRUPO MOTOR DE CIRCUITO HIDRAÚLICO DE AERO',
 'LÍQUIDOS INFLAMABLES 3256 - 3 - LÍQUIDO A TEMPERATURA ELEVADA; INFLAMABLE; N.E.P.; de punto de inflam',
 'LÍQUIDOS INFLAMABLES 3269 - 3 - BOLSA DE RESINA POLIESTÉRICA',
 'LÍQUIDOS INFLAMABLES 3343 - 3 - MEZCLA DE NITROGLICERINA; DESENSIBILIZADA; LÍQUIDA; INFLAMABLE; N.E.P',
 'LÍQUIDOS INFLAMABLES 3379 - 3 - EXPLOSIVO  DESENSIBILIZADO; LÍQUIDO; N.E.P.',
 'LÍQUIDOS INFLAMABLES 3469 - 3 - PINTURAS INFLAMABLES; CORROSIVAS (incluidos pinturas; lacas; esmaltes',
 'LÍQUIDOS INFLAMABLES 3494 - 3 - PETRÓLEO BRUTO ÁCIDO; INFLAMABLE; TÓXICO',
 'LÍQUIDOS\xa0INFLAMABLES\xa01110\xa0-\xa03\xa0-\xa0n-AMILMETILCETONA',
 'LÍQUIDOS\xa0INFLAMABLES\xa01161\xa0-\xa03\xa0-\xa0CARBONATO\xa0DE\xa0METILO',
 'LÍQUIDOS\xa0INFLAMABLES\xa01173\xa0-\xa03\xa0-\xa0ACETATO\xa0DE\xa0ETILO',
 'LÍQUIDOS\xa0INFLAMABLES\xa01231\xa0-\xa03\xa0-\xa0ACETATO\xa0DE\xa0METILO',
 'LÍQUIDOS\xa0INFLAMABLES\xa01266\xa0-\xa03\xa0-\xa0PRODUCTOS\xa0DE\xa0PERFUMERÍA\xa0que\xa0contengan\xa0disolventes\xa0inflamables',
 'LÍQUIDOS\xa0INFLAMABLES\xa01276\xa0-\xa03\xa0-\xa0ACETATO\xa0DE\xa0n-PROPILO',
 'LÍQUIDOS\xa0INFLAMABLES\xa01301\xa0-\xa03\xa0-\xa0ACETATO\xa0DE\xa0VINILO\xa0ESTABILIZADO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02348\xa0-\xa03\xa0-\xa0ACRILATOS\xa0DE\xa0BUTILO\xa0ESTABILIZADOS',
 'LÍQUIDOS\xa0INFLAMABLES\xa02358\xa0-\xa03\xa0-\xa0CICLOOCTATETRAENO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02359\xa0-\xa03\xa0-\xa0DIALILAMINA',
 'LÍQUIDOS\xa0INFLAMABLES\xa02360\xa0-\xa03\xa0-\xa0ÉTER\xa0DIALÍLICO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02361\xa0-\xa03\xa0-\xa0DIISOBUTILAMINA',
 'LÍQUIDOS\xa0INFLAMABLES\xa02362\xa0-\xa03\xa0-\xa01;1-DICLOROETANO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02363\xa0-\xa03\xa0-\xa0ETILMERCAPTANO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02364\xa0-\xa03\xa0-\xa0n-PROPILBENCENO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02366\xa0-\xa03\xa0-\xa0CARBONATO\xa0DE\xa0DIETILO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02367\xa0-\xa03\xa0-\xa0alfa-METILVALERALDEHIDO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02368\xa0-\xa03\xa0-\xa0alfa-PINENO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02370\xa0-\xa03\xa0-\xa01-HEXENO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02373\xa0-\xa03\xa0-\xa0DIETOXIMETANO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02374\xa0-\xa03\xa0-\xa03;3-DIETOXIPROPENO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02377\xa0-\xa03\xa0-\xa01;1-DIMETOXIETANO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02378\xa0-\xa03\xa0-\xa02-DIMETILAMINOACETO-NITRILO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02380\xa0-\xa03\xa0-\xa0DIMETILDIETOXISILANO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02381\xa0-\xa03\xa0-\xa0DISULFURO\xa0DE\xa0DIMETILO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02385\xa0-\xa03\xa0-\xa0ISOBUTIRATO\xa0DE\xa0ETILO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02386\xa0-\xa03\xa0-\xa01-ETILPIPERIDINA',
 'LÍQUIDOS\xa0INFLAMABLES\xa02387\xa0-\xa03\xa0-\xa0FLUOROBENCENO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02388\xa0-\xa03\xa0-\xa0FLUOROTOLUENOS',
 'LÍQUIDOS\xa0INFLAMABLES\xa02389\xa0-\xa03\xa0-\xa0FURANO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02393\xa0-\xa03\xa0-\xa0FORMIATO\xa0DE\xa0ISOBUTILO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02394\xa0-\xa03\xa0-\xa0PROPIONATO\xa0DE\xa0ISOBUTILO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02406\xa0-\xa03\xa0-\xa0ISOBUTIRATO\xa0DE\xa0ISOPROPILO',
 'LÍQUIDOS\xa0INFLAMABLES\xa02409\xa0-\xa03\xa0-\xa0PROPIONATO\xa0DE\xa0ISOPROPILO',
 'LÍQUIDOS\xa0INFLAMABLES\xa03065\xa0-\xa03\xa0-\xa0BEBIDAS\xa0ALCOHÓLICAS;\xa0con\xa0más\xa0del\xa070%\xa0de\xa0alcohol\xa0en\xa0volumen',
 'MATERIAL RADIACTIVO 2908 - 7 - MATERIALES RADIACTIVOS; BULTOS EXCEPTUADOS-EMBALAJES/ENVASES VACÍOS',
 'MATERIAL RADIACTIVO 2911 - 7 - MATERIALES RADIACTIVOS; BULTOS EXCEPTUADOS-INSTRUMENTOS o ARTÍCULOS',
 'MATERIAL RADIACTIVO 2912 - 7 - MATERIALES RADIACTIVOS; BAJA ACTIVIDAD ESPECÍFICA (BAE-I); no fisiona',
 'MATERIAL RADIACTIVO 2913 - 7 - MATERIALES RADIACTIVOS; OBJETOS CONTAMINADOS EN LA SUPERFICIE (OCS-I',
 'MATERIAL RADIACTIVO 2916 - 7 - MATERIALES RADIACTIVOS; BULTOS DEL TIPO B(U); no fisionables o fision',
 'MATERIAL RADIACTIVO 3332 - 7 - MATERIALES RADIACTIVOS; BULTOS DEL TIPO\xa0A; EN FORMA ESPECIAL; no fisi',
 'MATERIAL\xa0RADIACTIVO\xa02916\xa0-\xa07\xa0-\xa0MATERIALES\xa0RADIACTIVOS;\xa0BULTOS\xa0DEL\xa0TIPO\xa0B(U);\xa0no\xa0fisionables\xa0o\xa0fision',
 'PELIGROSO COMPUESTOS ORGANOHALOGENADOS',
 'PELIGROSO DESECHOS CLÍNICOS RESULTANTES DE LA ATENCIÓN MÉDICA PRESTADA EN HOSPITALES; CE',
 'PELIGROSO DESECHOS DE ACEITES MINERALES NO APTOS PARA EL USO A QUE ESTABAN DESTINADOS;',
 'PELIGROSO DESECHOS DE MEDICAMENTOS Y PRODUCTOS FARMACÉUTICOS;',
 'PELIGROSO DESECHOS QUE TENGAN COMO CONSTITUYENTES ASBESTO (POLVO Y FIBRAS);',
 'PELIGROSO DESECHOS QUE TENGAN COMO CONSTITUYENTES COMPUESTOS DE ZINC;',
 'PELIGROSO DESECHOS QUE TENGAN COMO CONSTITUYENTES ETERES;',
 'PELIGROSO DESECHOS QUE TENGAN COMO CONSTITUYENTES PLOMO; COMPUESTOS DE PLOMO;',
 'PELIGROSO DESECHOS QUE TENGAN COMO CONSTITUYENTES SELENIO; COMPUESTOS DE SELENIO;',
 'PELIGROSO DESECHOS QUE TENGAN COMO CONSTITUYENTES SOLVENTES ORGÁNICOS HALOGENADOS;',
 'PELIGROSO DESECHOS RESULTANTES DE LA FABRICACIÓN; PREPARACIÓN Y UTILIZACIÓN DE PRODUCTO',
 'PELIGROSO DESECHOS RESULTANTES DE LA PRODUCCIÓN; PREPARACIÓN Y UTILIZACIÓN DE PRODUCTOS QU',
 'PELIGROSO DESECHOS RESULTANTES DE LA PRODUCCIÓN; PREPARACIÓN Y UTILIZACIÓN DE RESINAS; L',
 'PELIGROSO DESECHOS RESULTANTES DE LA PRODUCCIÓN; PREPARACIÓN Y UTILIZACIÓN DE TINTAS; CO',
 'PELIGROSO DESECHOS RESULTANTES DEL TRATAMIENTO DE SUPERFICIE DE METALES Y PLÁSTICOS;',
 'PELIGROSO MEZCLAS Y EMULSIONES DE DESECHOS DE ACEITE Y AGUA O DE HIDROCARBUROS Y AGUA;',
 'PELIGROSO RESIDUOS ALQUITRANADOS RESULTANTES DE LA REFINACIÓN; DESTILACIÓN O CUALQUIER O',
 'PELIGROSO SUSTANCIAS QUÍMICAS DE DESECHO; NO IDENTIFICADAS O NUEVAS; RESULTANTES DE LA I',
 'PELIGROSO SUSTANCIAS Y ARTÍCULOS DE DESECHO QUE CONTENGAN; O ESTÉN CONTAMINADOS POR; BIF',
 'PELIGROSO\xa0DESECHOS\xa0DE\xa0MEDICAMENTOS\xa0Y\xa0PRODUCTOS\xa0FARMACÉUTICOS;',
 'PELIGROSO\xa0DESECHOS\xa0QUE\xa0TENGAN\xa0COMO\xa0CONSTITUYENTES\xa0ETERES;',
 'PÓLVORAS Y EXPLOSIVOS; ARTÍCULOS DE PIROTECNIA; FÓSFOROS (CERILLAS); EXPLOSIVOS PREPARADOS;  EXCEPTO LAS POLVORAS',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS BIODIÉSEL Y SUS MEZCLAS, SIN ACEITES DE PETRÓLEO O DE MINERAL BITUMINOSO O CON UN CONTENIDO INFERIOR AL 70 % EN PESO.',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; ACELERADORES DE VULCANIZACION PREPARADOS; PLASTIFICANTES COMPUESTOS PARA CAUCH',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; ACIDOS GRASOS MONOCARBOXILICOS INDUSTRIALES; ACEITES ACIDOS DEL REFINADO; ALCOHO',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; ALQUITRANES DE MADERA; ACEITES DE ALQUITRAN DE MADERA; CREOSOTA DE MADERA; MET',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; APRESTOS Y PRODUCTOS DE ACABADO;  ACELERADORES DE TINTURA O DE FIJACION DE MAT',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; CARBONES ACTIVADOS; MATERIAS MINERALES NATURALES ACTIVADAS; NEGRO DE ORIGEN AN',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; CEMENTOS;  MORTEROS;  HORMIGONES Y PREPARACIONES SIMILARES;  REFRACTARIOS;  EX',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; COLOFONIAS Y ACIDOS RESINICOS;  Y SUS DERIVADOS; ESENCIA Y ACEITE;  DE COLOFON',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; DISOLVENTES O DILUYENTES ORGANICOS COMPUESTOS;  NO EXPRESADOS NI COMPRENDIDOS',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; ELEMENTOS QUIMICOS IMPURIFICADOS PARA USO EN ELECTRONICA;  EN DISCOS;  PLAQUIT',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; ESENCIA DE TREMENTINA;  DE MADERA DE PINO;  DE PASTA CELULOSICA AL SULFATO Y D',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; GRAFITO ARTIFICIAL; GRAFITO COLOIDAL O SEMICOLOIDAL; PREPARACIONES A BASE DE G',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; INICIADORES Y ACELERADORES;  DE REACCION;  Y PREPARACIONES CATALITICAS;  NO EX',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; INSECTICIDAS;  RATICIDAS;  FUNGICIDAS;  HERBICIDAS;  INHIBIDORES DE GERMINACIO',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; LEJIAS RESIDUALES DE LA FABRICACION DE PASTAS DE CELULOSA;  AUNQUE ESTEN CONCE',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; LIQUIDOS PARA FRENOS HIDRAULICOS Y DEMAS PREPARACIONES LIQUIDAS PARA TRANSMISI',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; MEDIOS DE CULTIVO PREPARADOS PARA EL DESARROLLO DE MICROORGANISMOS',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; MEZCLAS DE ALQUILBENCENOS Y MEZCLAS DE ALQUILNAFTALENOS;  EXCEPTO LAS DE LAS P',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; PREPARACIONES AGLUTINANTES PARA MOLDES O PARA NUCLEOS DE FUNDICION; PRODUCTOS QU',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; PREPARACIONES ANTICONGELANTES Y LIQUIDOS PREPARADOS PARA DESCONGELAR',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; PREPARACIONES ANTIDETONANTES;  INHIBIDORES DE OXIDACION;  ADITIVOS PEPTIZANTES',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; PREPARACIONES PARA EL DECAPADO DE LOS METALES; FLUJOS Y DEMAS PREPARACIONES AUXI',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; PREPARACIONES Y CARGAS PARA APARATOS EXTINTORES; GRANADAS Y BOMBAS EXTINTORAS',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; REACTIVOS DE DIAGNOSTICO O DE LABORATORIO SOBRE CUALQUIER SOPORTE Y REACTIVOS',
 'PRODUCTOS DIVERSOS DE LAS INDUSTRIAS QUÍMICAS; [TALL OIL];  INCLUSO REFINADO',
 'PRODUCTOS FARMACÉUTICOS; GLANDULAS Y DEMAS ORGANOS PARA USOS OPOTERAPICOS;  DESECADOS;  INCLUSO PULVERI',
 'PRODUCTOS FARMACÉUTICOS; GUATAS;  GASAS;  VENDAS Y ARTICULOS ANALOGOS (POR EJEMPLO APOSITOS;  ESPARADRA',
 'PRODUCTOS FARMACÉUTICOS; MEDICAMENTOS (CON EXCLUSION DE LOS PRODUCTOS DE LAS PARTIDAS 3002;  3005 O 300',
 'PRODUCTOS FARMACÉUTICOS; PREPARACIONES Y ARTICULOS FARMACEUTICOS A QUE SE REFIERE LA NOTA 4 DEL CAPITULO',
 'PRODUCTOS FARMACÉUTICOS; SANGRE HUMANA; SANGRE ANIMAL PREPARADA PARA USOS TERAPEUTICOS;  PROFILACTICOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE ACIDO NITRICO; ACIDOS SULFONITRICOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE ACIDO SULFURICO; OLEUM',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE AMONIACO ANHIDRO O EN DISOLUCION ACUOSA',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE AZUFRE SUBLIMADO O PRECIPITADO; AZUFRE COLOIDAL       -',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE AZUFRE SUBLIMADO O PRECIPITADO; AZUFRE COLOIDAL-',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE BORATOS; PEROXOBORATOS (PERBORATOS)',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE CARBONATOS; PEROXOCARBONATOS (PERCARBONATOS); CARBONATO DE AMONIO COMERCIAL QUE',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE CARBONO (NEGROS DE HUMO Y OTRAS FORMAS DE CARBONO NO EXPRESADAS NI COMPRENDIDAS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE CARBUROS;  AUNQUE NO SEAN DE CONSTITUCION QUIMICA DEFINIDA',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE CIANUROS;  OXICIANUROS Y CIANUROS COMPLEJOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE CLORATOS Y PERCLORATOS; BROMATOS Y PERBROMATOS; YODATOS Y PERYODATOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE CLORURO DE HIDROGENO (ACIDO CLORHIDRICO); ACIDO CLOROSULFURICO',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE CLORUROS;  OXICLORUROS E HIDROXICLORUROS; BROMUROS Y OXIBROMUROS; YODUROS Y OX',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE COMPUESTOS INORGANICOS U ORGANICOS;  DE LOS METALES DE LAS TIERRAS RARAS;  DE',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE CORINDON ARTIFICIAL;  AUNQUE NO SEA QUIMICAMENTE DEFINIDO; OXIDO DE ALUMINIO;',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE DITIONITOS Y SULFOXILATOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE ELEMENTOS QUIMICOS RADIACTIVOS E ISOTOPOS RADIACTIVOS (INCLUIDOS LOS ELEMENTOS Q',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE FLUOR;  CLORO;  BROMO Y YODO',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE FLUORUROS; FLUOROSILICATOS;  FLUOROALUMINATOS Y DEMAS SALES COMPLEJAS DE FLUOR',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE FOSFINATOS (HIPOFOSFITOS);  FOSFONATOS (FOSFITOS);  FOSFATOS Y POLIFOSFATOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE FOSFUROS; AUNQUE NO SEAN DE CONSTITUCION QUIMICA DEFINIDA; EXCEPTO LOS FERROFO',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE HIDRAZINA E HIDROXILAMINA Y SUS SALES INORGANICAS; LAS DEMAS BASES INORGANICAS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE HIDROGENO;  GASES NOBLES Y DEMAS ELEMENTOS NO METALICOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE HIDROXIDO DE SODIO (SOSA CAUSTICA); HIDROXIDO DE POTASIO (POTASA CAUSTICA); PERO',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE HIDROXIDO Y PEROXIDO DE MAGNESIO; OXIDOS;  HIDROXIDOS Y PEROXIDOS;  DE ESTRONC',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE HIDRUROS;  NITRUROS;  AZIDUROS (AZIDAS);  SILICIUROS Y BORUROS;  AUNQUE NO SEA',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE HIPOCLORITOS; HIPOCLORITO DE CALCIO COMERCIAL; CLORITOS; HIPOBROMITOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE ISOTOPOS;  EXCEPTO LOS DE LA PARTIDA 2844; SUS COMPUESTOS INORGANICOS U ORGANI',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE LAS DEMAS SALES DE LOS ACIDOS O PEROXOACIDOS INORGANICOS;  CON EXCLUSION DE LO',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE LOS DEMAS ACIDOS INORGANICOS Y LOS DEMAS COMPUESTOS OXIGENADOS INORGANICOS DE LO',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE LOS DEMAS COMPUESTOS INORGANICOS (INCLUIDA EL AGUA DESTILADA;  DE CONDUCTIBILI',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE METALES ALCALINOS O ALCALINOTERREOS; METALES DE LAS TIERRAS RARAS;  ESCANDIO',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE METALES PRECIOSOS EN ESTADO COLOIDAL; COMPUESTOS INORGANICOS U ORGANICOS DE ME',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE NITRITOS; NITRATOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE OXIDO DE CINC; PEROXIDO DE CINC',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE OXIDO DE PLOMO; MINIO Y MINIO ANARANJADO',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE OXIDOS DE BORO; ACIDOS BORICOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE OXIDOS DE MANGANESO',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE OXIDOS DE TITANIO',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE OXIDOS E HIDROXIDOS DE CROMO',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE OXIDOS E HIDROXIDOS DE HIERRO; TIERRAS COLORANTES CON UN CONTENIDO DE HIERRO C',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE PENTAOXIDO DE DIFOSFORO; ACIDO FOSFORICO Y ACIDOS POLIFOSFORICOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE PEROXIDO DE HIDROGENO (AGUA OXIGENADA);  INCLUSO SOLIDIFICADO CON UREA',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE SALES DE LOS ACIDOS OXOMETALICOS O PEROXOMETALICOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE SILICATOS; SILICATOS COMERCIALES DE LOS METALES ALCALINOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE SULFATOS; ALUMBRES; PEROXOSULFATOS (PERSULFATOS)',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE SULFITOS; TIOSULFATOS',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE SULFUROS DE LOS ELEMENTOS NO METALICOS; TRISULFURO DE FOSFORO COMERCIAL',
 'PRODUCTOS QUÍMICOS INORGÁNICOS; COMPUESTOS INORGÁNICOS U ORGÁNICOS DE SULFUROS; POLISULFUROS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ACETALES Y SEMIACETALES;  INCLUSO CON OTRAS FUNCIONES OXIGENADAS;  Y SUS DERIV',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ACIDOS CARBOXILICOS CON FUNCIONES OXIGENADAS SUPLEMENTARIAS Y SUS ANHIDRIDOS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ACIDOS MONOCARBOXILICOS ACICLICOS NO SATURADOS Y ACIDOS MONOCARBOXILICOS CICLI',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ACIDOS MONOCARBOXILICOS ACICLICOS SATURADOS Y SUS ANHIDRIDOS;  HALOGENUROS;  P',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ACIDOS NUCLEICOS Y SUS SALES; LOS DEMAS COMPUESTOS HETEROCICLICOS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ACIDOS POLICARBOXILICOS;  SUS ANHIDRIDOS;  HALOGENUROS;  PEROXIDOS Y PEROXIACI',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ALCALOIDES VEGETALES;  NATURALES O REPRODUCIDOS POR SINTESIS;  SUS SALES;  ETE',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ALCOHOLES ACICLICOS Y SUS DERIVADOS HALOGENADOS;  SULFONADOS;  NITRADOS O NITR',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ALCOHOLES CICLICOS Y SUS DERIVADOS HALOGENADOS;  SULFONADOS;  NITRADOS O NITRO',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ALDEHIDOS;  INCLUSO CON OTRAS FUNCIONES OXIGENADAS; POLIMEROS CICLICOS DE LOS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ANTIBIOTICOS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; AZUCARES QUIMICAMENTE PUROS;  CON EXCEPCION DE LA SACAROSA;  LACTOSA;  MALTOSA',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; CETONAS Y QUINONAS;  INCLUSO CON OTRAS FUNCIONES OXIGENADAS;  Y SUS DERIVADOS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; COMPUESTOS AMINADOS CON FUNCIONES OXIGENADAS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; COMPUESTOS CON FUNCION AMINA',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; COMPUESTOS CON FUNCION CARBOXIAMIDA; COMPUESTOS CON FUNCION AMIDA DEL ACIDO CARB',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; COMPUESTOS CON FUNCION CARBOXIIMIDA (INCLUIDA LA SACARINA Y SUS SALES) O CON FUN',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; COMPUESTOS CON FUNCION NITRILO',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; COMPUESTOS CON OTRAS FUNCIONES NITROGENADAS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; COMPUESTOS HETEROCICLICOS CON HETEROATOMO(S) DE NITROGENO EXCLUSIVAMENTE',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; COMPUESTOS HETEROCICLICOS CON HETEROATOMO(S) DE OXIGENO EXCLUSIVAMENTE',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; DERIVADOS HALOGENADOS DE LOS HIDROCARBUROS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; DERIVADOS HALOGENADOS;  SULFONADOS;  NITRADOS O NITROSADOS DE LOS PRODUCTOS D',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; DERIVADOS HALOGENADOS;  SULFONADOS;  NITRADOS O NITROSADOS;  DE LOS FENOLES O',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; DERIVADOS SULFONADOS;  NITRADOS O NITROSADOS DE LOS HIDROCARBUROS;  INCLUSO HA',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; EPOXIDOS;  EPOXIALCOHOLES;  EPOXIFENOLES Y EPOXIETERES;  CON TRES ATOMOS EN E',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ESTERES DE LOS DEMAS ACIDOS INORGANICOS (CON EXCLUSION DE LOS ESTERES DE HALOG',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ESTERES FOSFORICOS Y SUS SALES;  INCLUIDOS LOS LACTOFOSFATOS; SUS DERIVADOS HA',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; ETERES;  ETERES-ALCOHOLES;  ETERES-FENOLES;  ETERES-ALCOHOLES-FENOLES;  PEROXI',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; FENOLES; FENOLES-ALCOHOLES',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; HETEROSIDOS;  NATURALES O REPRODUCIDOS POR SINTESIS;  SUS SALES;  ETERES;  EST',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; HIDROCARBUROS ACICLICOS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; HIDROCARBUROS CICLICOS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; HORMONAS;  NATURALES O REPRODUCIDAS POR SINTESIS; SUS DERIVADOS UTILIZADOS PRI',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; LOS DEMAS COMPUESTOS ORGANICOS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; LOS DEMAS COMPUESTOS ORGANO-INORGANICOS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; PROVITAMINAS Y VITAMINAS;  NATURALES O REPRODUCIDAS POR SINTESIS (INCLUIDOS LO',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; SALES E HIDROXIDOS DE AMONIO CUATERNARIO; LECITINAS Y OTROS FOSFOAMINOLIPIDOS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; SULFONAMIDAS',
 'PRODUCTOS QUÍMICOS ORGÁNICOS; TIOCOMPUESTOS ORGANICOS',
 'PRODUCTOS\xa0DIVERSOS\xa0DE\xa0LAS\xa0INDUSTRIAS\xa0QUÍMICAS;\xa0PREPARACIONES\xa0ANTICONGELANTES\xa0Y\xa0LIQUIDOS\xa0PREPARADOS\xa0PARA\xa0DESCONGELAR',
 'PRODUCTOS\xa0QUÍMICOS\xa0INORGÁNICOS;\xa0COMPUESTOS\xa0INORGÁNICOS\xa0U\xa0ORGÁNICOS\xa0DE\xa0ACIDO\xa0SULFURICO;\xa0OLEUM',
 'PRODUCTOS\xa0QUÍMICOS\xa0INORGÁNICOS;\xa0COMPUESTOS\xa0INORGÁNICOS\xa0U\xa0ORGÁNICOS\xa0DE\xa0CARBUROS;\xa0\xa0AUNQUE\xa0NO\xa0SEAN\xa0DE\xa0CONSTITUCION\xa0QUIMICA\xa0DEFINIDA',
 'PRODUCTOS\xa0QUÍMICOS\xa0INORGÁNICOS;\xa0COMPUESTOS\xa0INORGÁNICOS\xa0U\xa0ORGÁNICOS\xa0DE\xa0FOSFINATOS\xa0(HIPOFOSFITOS);\xa0\xa0FOSFONATOS\xa0(FOSFITOS);\xa0\xa0FOSFATOS\xa0Y\xa0POLIFOSFATOS',
 'PRODUCTOS\xa0QUÍMICOS\xa0INORGÁNICOS;\xa0COMPUESTOS\xa0INORGÁNICOS\xa0U\xa0ORGÁNICOS\xa0DE\xa0FOSFUROS;\xa0AUNQUE\xa0NO\xa0SEAN\xa0DE\xa0CONSTITUCION\xa0QUIMICA\xa0DEFINIDA;\xa0EXCEPTO\xa0LOS\xa0FERROFO',
 'PRODUCTOS\xa0QUÍMICOS\xa0INORGÁNICOS;\xa0COMPUESTOS\xa0INORGÁNICOS\xa0U\xa0ORGÁNICOS\xa0DE\xa0METALES\xa0PRECIOSOS\xa0EN\xa0ESTADO\xa0COLOIDAL;\xa0COMPUESTOS\xa0INORGANICOS\xa0U\xa0ORGANICOS\xa0DE\xa0ME',
 'PRODUCTOS\xa0QUÍMICOS\xa0INORGÁNICOS;\xa0COMPUESTOS\xa0INORGÁNICOS\xa0U\xa0ORGÁNICOS\xa0DE\xa0SALES\xa0DE\xa0LOS\xa0ACIDOS\xa0OXOMETALICOS\xa0O\xa0PEROXOMETALICOS',
 'PRODUCTOS\xa0QUÍMICOS\xa0INORGÁNICOS;\xa0COMPUESTOS\xa0INORGÁNICOS\xa0U\xa0ORGÁNICOS\xa0DE\xa0SILICATOS;\xa0SILICATOS\xa0COMERCIALES\xa0DE\xa0LOS\xa0METALES\xa0ALCALINOS',
 'PRODUCTOS\xa0QUÍMICOS\xa0ORGÁNICOS;\xa0ACIDOS\xa0NUCLEICOS\xa0Y\xa0SUS\xa0SALES;\xa0LOS\xa0DEMAS\xa0COMPUESTOS\xa0HETEROCICLICOS',
 'PRODUCTOS\xa0QUÍMICOS\xa0ORGÁNICOS;\xa0ESTERES\xa0DE\xa0LOS\xa0DEMAS\xa0ACIDOS\xa0INORGANICOS\xa0(CON\xa0EXCLUSION\xa0DE\xa0LOS\xa0ESTERES\xa0DE\xa0HALOG',
 'PRODUCTOS\xa0QUÍMICOS\xa0ORGÁNICOS;\xa0ETERES;\xa0\xa0ETERES-ALCOHOLES;\xa0\xa0ETERES-FENOLES;\xa0\xa0ETERES-ALCOHOLES-FENOLES;\xa0\xa0PEROXI',
 'PRODUCTOS\xa0QUÍMICOS\xa0ORGÁNICOS;\xa0LOS\xa0DEMAS\xa0COMPUESTOS\xa0ORGANICOS',
 'PÓLVORAS Y EXPLOSIVOS; ARTÍCULOS DE PIROTECNIA; FÓSFOROS (CERILLAS); ARTICULOS PARA FUEGOS ARTIFICIALES;  COHETES DE SE?ALES O GRANIFUGOS Y SIMILAR',
 'PÓLVORAS Y EXPLOSIVOS; ARTÍCULOS DE PIROTECNIA; FÓSFOROS (CERILLAS); FERROCERIO Y DEMAS ALEACIONES PIROFORICAS EN CUALQUIER FORMA; ARTICULOS DE MATER',
 'PÓLVORAS Y EXPLOSIVOS; ARTÍCULOS DE PIROTECNIA; FÓSFOROS (CERILLAS); FOSFOROS (CERILLAS);  EXCEPTO LOS ARTICULOS DE PIROTECNIA DE LA PARTIDA 3604',
 'PÓLVORAS Y EXPLOSIVOS; ARTÍCULOS DE PIROTECNIA; FÓSFOROS (CERILLAS); MECHAS DE SEGURIDAD; CORDONES DETONANTES; CEBOS Y CAPSULAS FULMINANTES; INFLAM',
 'PÓLVORAS Y EXPLOSIVOS; ARTÍCULOS DE PIROTECNIA; FÓSFOROS (CERILLAS); POLVORAS',
 'RESIDUOS Y DESPERDICIOS DE LAS INDUSTRIAS ALIMENTARIAS; ALIMENTOS HARINA;  POLVO Y <PELLETS>;  DE CARNE;  DE DESPOJOS;  DE PESCADO O DE CRUSTACE',
 'RESIDUOS Y DESPERDICIOS DE LAS INDUSTRIAS ALIMENTARIAS; ALIMENTOS LIAS O HECES DE VINO; TARTARO BRUTO',
 'RESIDUOS Y DESPERDICIOS DE LAS INDUSTRIAS ALIMENTARIAS; ALIMENTOS PREPARACIONES DEL TIPO DE LAS UTILIZADAS PARA LA ALIMENTACION DE LOS ANIMALES',
 'RESIDUOS Y DESPERDICIOS DE LAS INDUSTRIAS ALIMENTARIAS; ALIMENTOS TORTAS Y DEMAS RESIDUOS SOLIDOS DE LA EXTRACCION DEL ACEITE DE CACAHUETE;  INC',
 'RESIDUOS Y DESPERDICIOS DE LAS INDUSTRIAS ALIMENTARIAS; ALIMENTOS TORTAS Y DEMAS RESIDUOS SOLIDOS DE LA EXTRACCION DEL ACEITE DE SOJA;  INCLUSO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1442 - 5.1 - PERCLORATO AMÓNICO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1453 - 5.1 - CLORITO CÁLCICO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1454 - 5.1 - NITRATO CÁLCICO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1459 - 5.1 - CLORATO Y CLORURO DE MAGNESIO; MEZCLA SÓLIDA DE',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1463 - 5.1 - TRIÓXIDO DE CROMO ANHIDRO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1479 - 5.1 - SÓLIDO COMBURENTE; N.E.P.',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1485 - 5.1 - CLORATO POTÁSICO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1498 - 5.1 - NITRATO SÓDICO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1500 - 5.1 - NITRITO SÓDICO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1503 - 5.1 - PERMANGANATO SÓDICO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1505 - 5.1 - PERSULFATO SÓDICO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1507 - 5.1 - NITRATO DE ESTRONCIO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1511 - 5.1 - UREA-AGUA OXIGENADA',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1513 - 5.1 - CLORATO DE CINC',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1514 - 5.1 - NITRATO DE CINC',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 1748 - 5.1 - HIPOCLORITO CÁLCICO SECO o HIPOCLORITO CÁLCICO EN MEZCLA SECA; con',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2014 - 5.1 - PERÓXIDO DE HIDRÓGENO EN SOLUCIÓN ACUOSA con un mínimo del 20% y un',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2015 - 5.1 - PERÓXIDO DE HIDRÓGENO ESTABILIZADO; o PERÓXIDO DE HIDRÓGENO EN SOLU',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2208 - 5.1 - HIPOCLORITO CÁLCICO EN MEZCLA SECA con más del 10% pero no más del',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2426 - 5.1 - NITRATO AMÓNICO LÍQUIDO (en solución concentrada caliente)',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2427 - 5.1 - CLORATO POTÁSICO EN SOLUCIÓN ACUOSA',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2428 - 5.1 - CLORATO SÓDICO EN SOLUCIÓN ACUOSA',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2429 - 5.1 - CLORATO CÁLCICO EN SOLUCIÓN ACUOSA',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2464 - 5.1 - NITRATO DE BERILIO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2465 - 5.1 - ÁCIDO DICLOROISO -',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2466 - 5.1 - SUPERÓXIDO POTÁSICO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2741 - 5.1 - HIPOCLORITO BÁRICO con más del 22% de cloro activo',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2880 - 5.1 - HIPOCLORITO CÁLCICO HIDRATADO o HIPOCLORITO CÁLCICO HIDRATADO EN ME',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 3214 - 5.1 - PERMANGANATOS INORGÁNICOS EN SOLUCIÓN ACUOSA; N.E.P.',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 3215 - 5.1 - PERSULFATOS INORGÁNICOS; N.E.P.',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 3356 - 5.1 - GENERADOR QUÍMICO DE OXÍGENO.',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 3375 - 5.1 - EMULSIÓN DE NITRATO DE AMONIO o SUSPENSIÓN o GEL; explosivos interm',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 3378 - 5.1 - CARBONATO SÓDICO PEROXIHIDRATADO',
 'SUSTANCIAS CORROSIVAS 1052 - 8 - FLUORURO DE HIDRÓGENO ANHIDRO',
 'SUSTANCIAS CORROSIVAS 1718 - 8 - FOSFATO ÁCIDO DE BUTILO',
 'SUSTANCIAS CORROSIVAS 1719 - 8 - LÍQUIDO ALCALINO CAÚSTICO N.E.P.',
 'SUSTANCIAS CORROSIVAS 1726 - 8 - CLORURO DE ALUMINIO ANHIDRO',
 'SUSTANCIAS CORROSIVAS 1736 - 8 - CLORURO DE BENZOILO',
 'SUSTANCIAS CORROSIVAS 1759 - 8 - SÓLIDO CORROSIVO; N.E.P.',
 'SUSTANCIAS CORROSIVAS 1760 - 8 - LÍQUIDO CORROSIVO; N.E.P.',
 'SUSTANCIAS CORROSIVAS 1764 - 8 - ÁCIDO DICLOROACÉTICO',
 'SUSTANCIAS CORROSIVAS 1774 - 8 - EXTINTORES DE INCENDIOS; CARGAS PARA; líquidos corrosivos',
 'SUSTANCIAS CORROSIVAS 1779 - 8 - ÁCIDO FÓRMICO con más de 85%; en masa; de ácido',
 'SUSTANCIAS CORROSIVAS 1789 - 8 - ÁCIDO CLORHÍDRICO',
 'SUSTANCIAS CORROSIVAS 1791 - 8 - HIPOCLORITOS EN SOLUCIÓN',
 'SUSTANCIAS CORROSIVAS 1799 - 8 - NONILTRICLOROSILANO',
 'SUSTANCIAS CORROSIVAS 1803 - 8 - ÁCIDO FENOLSULFÓNICO LÍQUIDO',
 'SUSTANCIAS CORROSIVAS 1805 - 8 - ÁCIDO FOSFÓRICO EN SOLUCIÓN',
 'SUSTANCIAS CORROSIVAS 1806 - 8 - PENTACLORURO DE FÓSFORO',
 'SUSTANCIAS CORROSIVAS 1811 - 8 - HIDROGENODIFLUORURO DE POTASIO; SÓLIDO',
 'SUSTANCIAS CORROSIVAS 1814 - 8 - HIDRÓXIDO POTÁSICO EN SOLUCIÓN',
 'SUSTANCIAS CORROSIVAS 1823 - 8 - HIDRÓXIDO SÓDICO SÓLIDO',
 'SUSTANCIAS CORROSIVAS 1824 - 8 - HIDRÓXIDO SÓDICO EN SOLUCIÓN',
 'SUSTANCIAS CORROSIVAS 1829 - 8 - TRIÓXIDO DE AZUFRE ESTABILIZADO',
 'SUSTANCIAS CORROSIVAS 1830 - 8 - ÁCIDO SULFÚRICO con más del 51% de ácido',
 'SUSTANCIAS CORROSIVAS 1903 - 8 - DESINFECTANTE LÍQUIDO CORROSIVO; N.E.P.',
 'SUSTANCIAS CORROSIVAS 1905 - 8 - ÁCIDO SELÉNICO',
 'SUSTANCIAS CORROSIVAS 1906 - 8 - LODOS ÁCIDOS',
 'SUSTANCIAS CORROSIVAS 1907 - 8 - CAL SODADA con más del 4% de hidróxido sódico',
 'SUSTANCIAS CORROSIVAS 1908 - 8 - CLORITOS EN SOLUCIÓN',
 'SUSTANCIAS CORROSIVAS 1910 - 8 - ÓXIDO CÁLCICO',
 'SUSTANCIAS CORROSIVAS 2029 - 8 - HIDRAZINA ANHIDRA',
 'SUSTANCIAS CORROSIVAS 2031 - 8 - ÁCIDO NÍTRICO; excepto el ácido nítrico fumante rojo; con más del 70%',
 'SUSTANCIAS CORROSIVAS 2054 - 8 - MORFOLINA',
 'SUSTANCIAS CORROSIVAS 2079 - 8 - DIETILENTRIAMINA',
 'SUSTANCIAS CORROSIVAS 2209 - 8 - FORMALDEHIDO EN SOLUCIÓN con un mínimo del 25% de formaldehído',
 'SUSTANCIAS CORROSIVAS 2214 - 8 - ANHÍDRIDO FTÁLICO con más del 0;05% de anhídrido maleico',
 'SUSTANCIAS CORROSIVAS 2331 - 8 - CLORURO DE CINC ANHIDRO',
 'SUSTANCIAS CORROSIVAS 2357 - 8 - CICLOHEXILAMINA',
 'SUSTANCIAS CORROSIVAS 2401 - 8 - PIPERIDINA',
 'SUSTANCIAS CORROSIVAS 2430 - 8 - ALQUILFENOLES SÓLIDOS; N.E.P. (incluidos los homólogos C2 a C12)',
 'SUSTANCIAS CORROSIVAS 2434 - 8 - DIBENCILDICLOROSILANO',
 'SUSTANCIAS CORROSIVAS 2435 - 8 - ETILFENILDICLOROSILANO',
 'SUSTANCIAS CORROSIVAS 2437 - 8 - METILFENILDICLOROSILANO',
 'SUSTANCIAS CORROSIVAS 2439 - 8 - HIDROGENODIFLUORURO DE SODIO',
 'SUSTANCIAS CORROSIVAS 2440 - 8 - CLORURO ESTÁNNICO PENTAHIDRATADO',
 'SUSTANCIAS CORROSIVAS 2442 - 8 - CLORURO DE TRICLOROACETILO',
 'SUSTANCIAS CORROSIVAS 2443 - 8 - OXITRICLORURO DE VANADIO',
 'SUSTANCIAS CORROSIVAS 2444 - 8 - TETRACLORURO DE VANADIO',
 'SUSTANCIAS CORROSIVAS 2475 - 8 - TRICLORURO DE VANADIO',
 'SUSTANCIAS CORROSIVAS 2491 - 8 - ETANOLAMINA o ETANOLAMINA EN SOLUCIÓN',
 'SUSTANCIAS CORROSIVAS 2509 - 8 - SULFATO ÁCIDO DE POTASIO',
 'SUSTANCIAS CORROSIVAS 2511 - 8 - ÁCIDO 2-CLOROPROPIÓNICO',
 'SUSTANCIAS CORROSIVAS 2513 - 8 - BROMURO DE BROMOACETILO',
 'SUSTANCIAS CORROSIVAS 2581 - 8 - CLORURO DE ALUMINIO EN SOLUCIÓN',
 'SUSTANCIAS CORROSIVAS 2582 - 8 - CLORURO FÉRRICO EN SOLUCIÓN',
 'SUSTANCIAS CORROSIVAS 2583 - 8 - ÁCIDOS ALQUILSULFÓNICOS SÓLIDOS o ÁCIDOS ARILSULFÓNICOS SÓLIDOS; con',
 'SUSTANCIAS CORROSIVAS 2584 - 8 - ÁCIDOS ALQUILSULFÓNICOS LÍQUIDOS o ÁCIDOS ARILSULFÓNICOS LÍQUIDOS; co',
 'SUSTANCIAS CORROSIVAS 2586 - 8 - ÁCIDOS ALQUILSULFÓNICOS LÍQUIDOS o ÁCIDOS ARILSULFÓNICOS LÍQUIDOS; co',
 'SUSTANCIAS CORROSIVAS 2672 - 8 - AMONIACO EN SOLUCIÓN acuosa de densidad relativa comprendida entre 0;',
 'SUSTANCIAS CORROSIVAS 2685 - 8 - N;N-DIETILETILENDIAMINA',
 'SUSTANCIAS CORROSIVAS 2735 - 8 - AMINAS LÍQUIDAS; CORROSIVAS; N.E.P. o POLIAMINAS LÍQUIDAS; CORROSIVAS',
 'SUSTANCIAS CORROSIVAS 2789 - 8 - ÁCIDO ACÉTICO GLACIAL o ÁCIDO ACÉTICO EN SOLUCIÓN con más del 80%; en',
 'SUSTANCIAS CORROSIVAS 2790 - 8 - ÁCIDO ACÉTICO EN SOLUCIÓN con un mínimo del 50% y un máximo del 80%;',
 'SUSTANCIAS CORROSIVAS 2794 - 8 - ACUMULADORES ELÉCTRICOS DE ELECTROLITO LÍQUIDO ÁCIDO',
 'SUSTANCIAS CORROSIVAS 2795 - 8 - ACUMULADORES ELÉCTRICOS DE ELECTROLITO LÍQUIDO ALCALINO',
 'SUSTANCIAS CORROSIVAS 2796 - 8 - ÁCIDO SULFÚRICO con un máximo del\xa051% de ácido o ELECTROLITO ÁCIDO PA',
 'SUSTANCIAS CORROSIVAS 2800 - 8 - ACUMULADORES ELÉCTRICOS NO DERRAMABLES DE ELECTROLITO LÍQUIDO',
 'SUSTANCIAS CORROSIVAS 2801 - 8 - COLORANTE LÍQUIDO CORROSIVO; N.E.P. o MATERIA INTERMEDIA PARA COLORAN',
 'SUSTANCIAS CORROSIVAS 2815 - 8 - N-AMINOETILPIPERAZINA',
 'SUSTANCIAS CORROSIVAS 2823 - 8 - ÁCIDO CROTÓNICO SÓLIDO',
 'SUSTANCIAS CORROSIVAS 2834 - 8 - ÁCIDO FOSFOROSO',
 'SUSTANCIAS CORROSIVAS 2837 - 8 - BISULFATOS EN SOLUCIÓN ACUOSA',
 'SUSTANCIAS CORROSIVAS 2905 - 8 - CLOROFENOLATOS SÓLIDOS o FENOLATOS SÓLIDOS',
 'SUSTANCIAS CORROSIVAS 2920 - 8 - LÍQUIDO CORROSIVO INFLAMABLE; N.E.P.',
 'SUSTANCIAS CORROSIVAS 2922 - 8 - LÍQUIDO CORROSIVO; TÓXICO; N.E.P.',
 'SUSTANCIAS CORROSIVAS 2967 - 8 - ÁCIDO SULFÁMICO',
 'SUSTANCIAS CORROSIVAS 2986 - 8 - CLOROSILANOS; CORROSIVOS; INFLAMABLES; N.E.P.',
 'SUSTANCIAS CORROSIVAS 2987 - 8 - CLOROSILANOS; CORROSIVOS; N.E.P.',
 'SUSTANCIAS CORROSIVAS 3028 - 8 - ACUMULADORES ELÉCTRICOS SECOS QUE CONTIENEN HIDRÓXIDO DE POTASIO SÓLI',
 'SUSTANCIAS CORROSIVAS 3066 - 8 - PINTURA (incluye pintura; laca; esmalte; colorante; goma laca; barniz',
 'SUSTANCIAS CORROSIVAS 3145 - 8 - ALQUILFENOLES LÍQUIDOS; N.E.P. (incluidos los homólogos C2 a C12)',
 'SUSTANCIAS CORROSIVAS 3145 - 8 - ALQUILFENOLES LÍQUIDOS; N.E.P. (incluidos los homólogos C2 a C12)SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2433 - 6.1 - CLORONITROTOLUENOS LÍQUIDOS',
 'SUSTANCIAS CORROSIVAS 3147 - 8 - COLORANTE SÓLIDO; CORROSIVO; N.E.P. o MATERIA INTERMEDIA PARA COLORAN',
 'SUSTANCIAS CORROSIVAS 3259 - 8 - AMINAS SÓLIDAS; CORROSIVAS; N.E.P.; o POLIAMINAS SÓLIDAS; CORROSIVAS;',
 'SUSTANCIAS CORROSIVAS 3260 - 8 - SÓLIDO CORROSIVO; ÁCIDO; INORGÁNICO; N.E.P.',
 'SUSTANCIAS CORROSIVAS 3262 - 8 - SÓLIDO CORROSIVO; BÁSICO; INORGÁNICO; N.E.P.',
 'SUSTANCIAS CORROSIVAS 3264 - 8 - LÍQUIDO CORROSIVO; ÁCIDO; INORGÁNICO; N.E.P.',
 'SUSTANCIAS CORROSIVAS 3265 - 8 - LÍQUIDO CORROSIVO; ÁCIDO; ORGÁNICO; N.E.P.',
 'SUSTANCIAS CORROSIVAS 3266 - 8 - LÍQUIDO CORROSIVO; BÁSICO; INORGÁNICO; N.E.P.',
 'SUSTANCIAS CORROSIVAS 3453 - 8 - ÁCIDO FOSFÓRICO SÓLIDO',
 'SUSTANCIAS CORROSIVAS 3456 - 8 - ÁCIDO NITROSILSULFÚRICO SÓLIDO',
 'SUSTANCIAS CORROSIVAS 3470 - 8 - PINTURAS CORROSIVAS; INFLAMABLES (incluidos pinturas; lacas; esmaltes',
 'SUSTANCIAS CORROSIVAS 3471 - 8 - HIDROGENODIFLUORUROS EN SOLUCIÓN; N.E.P.',
 'SUSTANCIAS CORROSIVAS 3498 - 8 - MONOCLORURO DE YODO LÍQUIDO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2468 - 5.1 - ÁCIDO TRICLORO-ISOCIANÚRICO SECO',
 'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2469 - 5.1 - BROMATO DE CINC'
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1051 - 6.1 - CIANURO DE HIDRÓGENO ANHIDRO ESTABILIZADO con menos del\xa03% de agua',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1098 - 6.1 - ALCOHOL ALÍLICO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1544 - 6.1 - ALCALOIDES SÓLIDOS; N.E.P.; o\xa0SALES DE ALCALOIDES; SÓLIDAS; N.E.P.',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1593 - 6.1 - DICLOROMETANO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1602 - 6.1 - COLORANTE LÍQUIDO; TÓXICO; N.E.P. o MATERIA INTERMEDIA PARA COLORAN',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1631 - 6.1 - BENZOATO DE MERCURIO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1637 - 6.1 - GLUCONATO DE MERCURIO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1649 - 6.1 - MEZCLA ANTIDETONANTE PARA COMBUSTIBLES DE MOTORES',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1657 - 6.1 - SALICILATO DE NICOTINA',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1670 - 6.1 - PERCLOROMETIL-MERCAPTANO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1690 - 6.1 - FLUORURO SÓDICO SÓLIDO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1702 - 6.1 - 1;1;2;2-TETRACLOROETANO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1851 - 6.1 - MEDICAMENTO LÍQUIDO; TÓXICO; N.E.P.',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1886 - 6.1 - CLORURO DE BENCILIDENO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1897 - 6.1 - TETRACLOROETILENO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1994 - 6.1 - HIERRO PENTACARBONILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2078 - 6.1 - DIISOCIANATO DE TOLUENO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2205 - 6.1 - ADIPONITRILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2285 - 6.1 - ISOCIANATOBENZOTRI-FLUORUROS',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2382 - 6.1 - DIMETILHIDRAZINA SIMÉTRICA',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2407 - 6.1 - CLOROFORMIATO DE ISOPROPILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2431 - 6.1 - ANISIDINAS',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2432 - 6.1 - N;N-DIETILANILINA',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2433 - 6.1 - CLORONITROTOLUENOS LÍQUIDOS',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2438 - 6.1 - CLORURO DE TRIMETILACETILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2446 - 6.1 - NITROCRESOLES SÓLIDOS',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2470 - 6.1 - FENILACETONITRILO LÍQUIDO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2471 - 6.1 - TETRÓXIDO DE OSMIO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2473 - 6.1 - ARSANILATO SÓDICO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2474 - 6.1 - TIOFOSGENO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2477 - 6.1 - ISOTIOCIANATO DE METILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2480 - 6.1 - ISOCIANATO DE METILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2481 - 6.1 - ISOCIANATO DE ETILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2482 - 6.1 - ISOCIANATO DE n-PROPILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2483 - 6.1 - ISOCIANATO DE ISOPROPILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2484 - 6.1 - ISOCIANATO DE terc-BUTILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2485 - 6.1 - ISOCIANATO DE n-BUTILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2486 - 6.1 - ISOCIANATO DE ISOBUTILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2574 - 6.1 - FOSFATO DE TRICRESILO con más del 3% de isómero orto',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2588 - 6.1 - PLAGUICIDA; SÓLIDO; TÓXICO; N.E.P.',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2611 - 6.1 - CLORHIDRINA PROPILÉNICA',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2629 - 6.1 - FLUOROACETATO DE SODIO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2644 - 6.1 - YODURO DE METILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2655 - 6.1 - FLUOROSILICATO DE POTASIO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2674 - 6.1 - FLUOROSILICATO DE SODIO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2757 - 6.1 - PLAGUICIDA A BASE DE CARBAMATO; SÓLIDO; TÓXICO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2810 - 6.1 - LÍQUIDO TÓXICO; ORGÁNICO; N.E.P.',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2811 - 6.1 - SÓLIDO TÓXICO; ORGÁNICO; N.E.P.',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2821 - 6.1 - FENOL EN SOLUCIÓN',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2854 - 6.1 - FLUOROSILICATO AMÓNICO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2874 - 6.1 - ALCOHOL FURFURÍLICO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2902 - 6.1 - PLAGUICIDA LÍQUIDO; TÓXICO; N.E.P.',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2903 - 6.1 - PLAGUICIDA LÍQUIDO; TÓXICO; INFLAMABLE; N.E.P.; de punto de inflama',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2927 - 6.1 - LÍQUIDO TÓXICO; CORROSIVO; ORGÁNICO; N.E.P.',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2931 - 6.1 - SULFATO DE VANADILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2936 - 6.1 - ÁCIDO TIOLÁCTICO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2937 - 6.1 - ALCOHOL alfa-METILBENCÍLICO LÍQUIDO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2941 - 6.1 - FLUORANILINAS',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 2966 - 6.1 - TIOGLICOL',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 3014 - 6.1 - PLAGUICIDA A BASE DE NITROFENOLES SUSTITUIDOS; LÍQUIDO; TÓXICO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 3018 - 6.1 - PLAGUICIDA A BASE DE ORGANOFÓSFORO; LÍQUIDO; TÓXICO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 3023 - 6.1 - 2-METIL-2-HEPTANOTIOL',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 3140 - 6.1 - ALCALOIDES LÍQUIDOS; N.E.P. o\xa0SALES DE ALCALOIDES LÍQUIDAS; N.E.P.',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 3143 - 6.1 - COLORANTE SÓLIDO; TÓXICO; N.E.P. o MATERIA INTERMEDIA PARA COLORANT',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 3249 - 6.1 - MEDICAMENTO SÓLIDO; TÓXICO; N.E.P.',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 3277 - 6.1 - CLOROFORMIATOS TÓXICOS; CORROSIVOS; N.E.P.',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 3302 - 6.1 - ACRILATO 2-DIMETILAMINOETÍLICO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 3352 - 6.1 - PLAGUICIDA PIRETROIDEO; LÍQUIDO; TÓXICO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 3430 - 6.1 - XILENOLES LÍQUIDOS',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 1841 - 9 - ALDEHIDATO AMÓNICO',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 2211 - 9 - POLÍMERO EN BOLITAS DILATABLES que desprenden vapores inflamables',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 2216 - 9 - HARINA DE PESCADO (DESECHOS DE PESCADO) ESTABILIZADA',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 2990 - 9 - APARATOS DE SALVAMENTO AUTOINFLABLES',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 3077 - 9 - SUSTANCIA SÓLIDA POTENCIALMENTE PELIGROSA PARA EL MEDIO AMBIENTE; N.E',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 3082 - 9 - SUSTANCIA LÍQUIDA POTENCIALMENTE PELIGROSA PARA EL MEDIO AMBIENTE; N.',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 3090 - 9 - BATERÍAS DE METAL LITIO (incluidas las baterías de aleación de litio)',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 3091 - 9 - BATERÍAS DE METAL LITIO INSTALADAS EN UN EQUIPO O BATERÍAS DE METAL L',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 3166 - 9 - MOTOR DE COMBUSTIÓN INTERNA o VEHÍCULO PROPULSADO POR GAS INFLAMABLE',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 3268 - 9 - INFLADORES DE BOLSAS NEUMÁTICAS o MÓDULOS DE BOLSAS NEUMÁTICAS o PRET',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 3314 - 9 - COMPUESTO PARA EL MOLDEADO DE PLÁSTICOS en forma de pasta; hoja o cue',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 3316 - 9 - EQUIPO QUÍMICO o BOTIQUÍN DE URGENCIA',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 3363 - 9 - MERCANCÍAS PELIGROSAS EN MAQUINARIA o MERCANCÍAS PELIGROSAS EN APARAT',
 'SUSTANCIAS Y OBJETOS PELIGROSOS VARIOS\xa0 3480 - 9 - BATERÍAS DE IÓN LITIO (incluidas las baterías poliméricas de ión liti',
 'SUSTANCIAS\xa0COMBURENTES\xa0Y\xa0PERÓXIDOS\xa0ORGÁNICOS\xa01748\xa0-\xa05.1\xa0-\xa0HIPOCLORITO\xa0CÁLCICO\xa0SECO\xa0o\xa0HIPOCLORITO\xa0CÁLCICO\xa0EN\xa0MEZCLA\xa0SECA;\xa0con',
 'SUSTANCIAS\xa0COMBURENTES\xa0Y\xa0PERÓXIDOS\xa0ORGÁNICOS\xa02427\xa0-\xa05.1\xa0-\xa0CLORATO\xa0POTÁSICO\xa0EN\xa0SOLUCIÓN\xa0ACUOSA',
 'SUSTANCIAS\xa0COMBURENTES\xa0Y\xa0PERÓXIDOS\xa0ORGÁNICOS\xa02428\xa0-\xa05.1\xa0-\xa0CLORATO\xa0SÓDICO\xa0EN\xa0SOLUCIÓN\xa0ACUOSA',
 'SUSTANCIAS\xa0COMBURENTES\xa0Y\xa0PERÓXIDOS\xa0ORGÁNICOS\xa02464\xa0-\xa05.1\xa0-\xa0NITRATO\xa0DE\xa0BERILIO',
 'SUSTANCIAS\xa0COMBURENTES\xa0Y\xa0PERÓXIDOS\xa0ORGÁNICOS\xa02465\xa0-\xa05.1\xa0-\xa0ÁCIDO\xa0DICLOROISO\xa0-',
 'SUSTANCIAS\xa0CORROSIVAS\xa01052\xa0-\xa08\xa0-\xa0FLUORURO\xa0DE\xa0HIDRÓGENO\xa0ANHIDRO',
 'SUSTANCIAS\xa0CORROSIVAS\xa01777\xa0-\xa08\xa0-\xa0ÁCIDO\xa0FLUOROSULFÓNICO',
 'SUSTANCIAS\xa0CORROSIVAS\xa01811\xa0-\xa08\xa0-\xa0HIDROGENODIFLUORURO\xa0DE\xa0POTASIO;\xa0SÓLIDO',
 'SUSTANCIAS\xa0CORROSIVAS\xa01823\xa0-\xa08\xa0-\xa0HIDRÓXIDO\xa0SÓDICO\xa0SÓLIDO',
 'SUSTANCIAS\xa0CORROSIVAS\xa02029\xa0-\xa08\xa0-\xa0HIDRAZINA\xa0ANHIDRA',
 'SUSTANCIAS\xa0CORROSIVAS\xa02031\xa0-\xa08\xa0-\xa0ÁCIDO\xa0NÍTRICO;\xa0excepto\xa0el\xa0ácido\xa0nítrico\xa0fumante\xa0rojo;\xa0con\xa0más\xa0del\xa070%',
 'SUSTANCIAS\xa0CORROSIVAS\xa02357\xa0-\xa08\xa0-\xa0CICLOHEXILAMINA',
 'SUSTANCIAS\xa0CORROSIVAS\xa02401\xa0-\xa08\xa0-\xa0PIPERIDINA',
 'SUSTANCIAS\xa0CORROSIVAS\xa02439\xa0-\xa08\xa0-\xa0HIDROGENODIFLUORURO\xa0DE\xa0SODIO',
 'SUSTANCIAS\xa0CORROSIVAS\xa02443\xa0-\xa08\xa0-\xa0OXITRICLORURO\xa0DE\xa0VANADIO',
 'SUSTANCIAS\xa0CORROSIVAS\xa02796\xa0-\xa08\xa0-\xa0ÁCIDO\xa0SULFÚRICO\xa0con\xa0un\xa0máximo\xa0del\xa051%\xa0de\xa0ácido\xa0o\xa0ELECTROLITO\xa0ÁCIDO\xa0PA',
 'SUSTANCIAS\xa0CORROSIVAS\xa03470\xa0-\xa08\xa0-\xa0PINTURAS\xa0CORROSIVAS;\xa0INFLAMABLES\xa0(incluidos\xa0pinturas;\xa0lacas;\xa0esmaltes',
 'SUSTANCIAS\xa0TÓXICAS\xa0Y\xa0SUSTANCIAS\xa0INFECCIOSAS\xa01649\xa0-\xa06.1\xa0-\xa0MEZCLA\xa0ANTIDETONANTE\xa0PARA\xa0COMBUSTIBLES\xa0DE\xa0MOTORES',
 'SUSTANCIAS\xa0TÓXICAS\xa0Y\xa0SUSTANCIAS\xa0INFECCIOSAS\xa01702\xa0-\xa06.1\xa0-\xa01;1;2;2-TETRACLOROETANO',
 'SUSTANCIAS\xa0TÓXICAS\xa0Y\xa0SUSTANCIAS\xa0INFECCIOSAS\xa02078\xa0-\xa06.1\xa0-\xa0DIISOCIANATO\xa0DE\xa0TOLUENO',
 'SUSTANCIAS\xa0TÓXICAS\xa0Y\xa0SUSTANCIAS\xa0INFECCIOSAS\xa02382\xa0-\xa06.1\xa0-\xa0DIMETILHIDRAZINA\xa0SIMÉTRICA',
 'SUSTANCIAS\xa0TÓXICAS\xa0Y\xa0SUSTANCIAS\xa0INFECCIOSAS\xa02438\xa0-\xa06.1\xa0-\xa0CLORURO\xa0DE\xa0TRIMETILACETILO',
 'SUSTANCIAS\xa0TÓXICAS\xa0Y\xa0SUSTANCIAS\xa0INFECCIOSAS\xa02482\xa0-\xa06.1\xa0-\xa0ISOCIANATO\xa0DE\xa0n-PROPILO',
 'SUSTANCIAS\xa0TÓXICAS\xa0Y\xa0SUSTANCIAS\xa0INFECCIOSAS\xa02486\xa0-\xa06.1\xa0-\xa0ISOCIANATO\xa0DE\xa0ISOBUTILO',
 'SUSTANCIAS\xa0TÓXICAS\xa0Y\xa0SUSTANCIAS\xa0INFECCIOSAS\xa02674\xa0-\xa06.1\xa0-\xa0FLUOROSILICATO\xa0DE\xa0SODIO',
 'SUSTANCIAS\xa0TÓXICAS\xa0Y\xa0SUSTANCIAS\xa0INFECCIOSAS\xa03302\xa0-\xa06.1\xa0-\xa0ACRILATO\xa02-DIMETILAMINOETÍLICO',
 'SUSTANCIAS\xa0TÓXICAS\xa0Y\xa0SUSTANCIAS\xa0INFECCIOSAS\xa03430\xa0-\xa06.1\xa0-\xa0XILENOLES\xa0LÍQUIDOS',
 'SUSTANCIAS\xa0Y\xa0OBJETOS\xa0PELIGROSOS\xa0VARIOS\xa0\xa01841\xa0-\xa09\xa0-\xa0ALDEHIDATO\xa0AMÓNICO',
 'SUSTANCIAS\xa0Y\xa0OBJETOS\xa0PELIGROSOS\xa0VARIOS\xa0\xa02211\xa0-\xa09\xa0-\xa0POLÍMERO\xa0EN\xa0BOLITAS\xa0DILATABLES\xa0que\xa0desprenden\xa0vapores\xa0inflamables',
 'SUSTANCIAS\xa0Y\xa0OBJETOS\xa0PELIGROSOS\xa0VARIOS\xa0\xa03090\xa0-\xa09\xa0-\xa0BATERÍAS\xa0DE\xa0METAL\xa0LITIO\xa0(incluidas\xa0las\xa0baterías\xa0de\xa0aleación\xa0de\xa0litio)',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1295 - 4.3 - TRICLOROSILANO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1325 - 4.1 - SÓLIDO INFLAMABLE ORGÁNICO; N.E.P.',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1331 - 4.1 - FÓSFOROS DISTINTOS DE LOS DE SEGURIDAD',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1332 - 4.1 - METALDEHIDO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1346 - 4.1 - SILICIO EN POLVO; AMORFO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1350 - 4.1 - AZUFRE',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1357 - 4.1 - NITRATO DE UREA HUMIDIFICADO con un mínimo del 20%; en masa; de agu',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1362 - 4.2 - CARBÓN ACTIVADO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1373 - 4.2 - FIBRAS o TEJIDOS DE ORIGEN ANIMAL o VEGETAL o SINTÉTICOS; N.E.P.; i',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1378 - 4.2 - CATALIZADOR DE METAL HUMEDECIDO con un exceso visible de líquido',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1379 - 4.2 - PAPEL TRATADO CON ACEITES NO SATURADOS; incompletamente seco (inclu',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1384 - 4.2 - DITIONITO SÓDICO (HIDROSULFITO SÓDICO)',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1385 - 4.2 - SULFURO SÓDICO ANHIDRO o SULFURO SÓDICO con menos del 30% de agua d',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1387 - 4.2 - DESECHOS DE LANA; HÚMEDOS',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1389 - 4.3 - METALES ALCALINOS; AMALGAMA LÍQUIDA DE;',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1390 - 4.3 - AMIDAS DE METALES ALCALINOS',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1398 - 4.3 - ALUMINIOSILICIO EN POLVO; NO RECUBIERTO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1400 - 4.3 - BARIO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1401 - 4.3 - CALCIO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1402 - 4.3 - CARBURO CÁLCICO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1403 - 4.3 - CIANAMIDA CÁLCICA con más del 0;1% de carburo de calcio',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1418 - 4.3 - MAGNESIO EN POLVO o ALEACIONES DE MAGNESIO EN POLVO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1422 - 4.3 - POTASIO Y SODIO; ALEACIONES LÍQUIDAS DE',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1428 - 4.3 - SODIO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1431 - 4.2 - METILATO SÓDICO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1855 - 4.2 - CALCIO PIROFÓRICO o CALCIO; ALEACIONES PIROFÓRICAS DE',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 1944 - 4.1 - FÓSFOROS DE SEGURIDAD (en estuches; cartones o cajas)',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2000 - 4.1 - CELULOIDE en bloques; barras; rollos; hojas; tubos; etc.; excepto l',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2002 - 4.2 - CELULOIDE; DESECHOS DE',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2006 - 4.2 - PLÁSTICOS A BASE DE NITROCELULOSA QUE EXPERIMENTAN CALENTAMIENTO ES',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2008 - 4.2 - CIRCONIO EN POLVO; SECO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2009 - 4.2 - CIRCONIO SECO; en láminas; tiras o alambre',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2011 - 4.3 - FOSFURO MAGNÉSICO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2213 - 4.1 - PARAFORMALDEHIDO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2257 - 4.3 - POTASIO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2441 - 4.2 - TRICLORURO DE TITANIO PIROFÓRICO o TRICLORURO DE TITANIO PIROFÓRICO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2447 - 4.2 - FÓSFORO BLANCO FUNDIDO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2448 - 4.1 - AZUFRE FUNDIDO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2463 - 4.3 - HIDRURO ALUMÍNICO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2546 - 4.2 - TITANIO EN POLVO SECO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2555 - 4.1 - NITROCELULOSA CON un mínimo del 25%; en masa; de AGUA',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2557 - 4.1 - NITROCELULOSA; con un máximo del\xa012;6%; en\xa0masa seca; de nitrógeno;',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2717 - 4.1 - ALCANFOR sintético',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2793 - 4.2 - VIRUTAS; TORNEADURAS o RASPADURAS DE METALES FERROSOS en una forma',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2845 - 4.2 - LÍQUIDO PIROFÓRICO ORGÁNICO; N.E.P.',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 2858 - 4.1 - CIRCONIO SECO; en forma de alambre enrollado; de láminas metálicas',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3088 - 4.2 - SÓLIDO ORGÁNICO QUE EXPERIMENTA CALENTAMIENTO ESPONTÁNEO; N.E.P.',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3170 - 4.3 - SUBPRODUCTOS DE LA FUNDICIÓN DEL ALUMINIO o SUBPRODUCTOS DE LA REFU',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3174 - 4.2 - DISULFURO DE TITANIO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3181 - 4.1 - SALES METÁLICAS DE COMPUESTOS ORGÁNICOS; INFLAMABLES; N.E.P.',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3189 - 4.2 - POLVO METÁLICO QUE EXPERIMENTA CALENTAMIENTO ESPONTÁNEO; N.E.P.',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3208 - 4.3 - SUSTANCIA METÁLICA QUE REACCIONA CON EL AGUA; N.E.P.',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3209 - 4.3 - SUSTANCIA METÁLICA QUE REACCIONA CON EL AGUA Y QUE EXPERIMENTA CALE',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3241 - 4.1 - 2-BROMO-2-NITROPROPANO-1;3-DIOL',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3242 - 4.1 - AZODICARBONAMIDA',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3255 - 4.2 - HIPOCLORITO DE terc-BUTILO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3270 - 4.1 - FILTROS DE MEMBRANAS NITROCELULÓSICAS; con un máximo del 12;6% de n',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3292 - 4.3 - BATERÍAS QUE CONTIENEN SODIO o ELEMENTOS DE BATERÍA QUE CONTIENEN S',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3313 - 4.2 - PIGMENTOS ORGÁNICOS QUE EXPERIMENTAN UN CALENTAMIENTO ESPONTÁNEO',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3317 - 4.1 - 2-AMINO-4;6-DINITROFENOL; HUMIDIFICADO con una proporción de agua;',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3401 - 4.3 -  METALES ALCALINOS; AMALGAMA SÓLIDA DE',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3402 - 4.3 - METALES ALCALINOTÉRREOS; AMALGAMA SÓLIDA DE',
 'SÓLIDOS INFLAMABLES; SUSTANCIAS QUE PUEDEN EXPERIMENTAR COMBUSTIÓN ESPONTÁNEA; S 3402 - 4.3 - METALES ALCALINOTÉRREOS; AMALGAMA SÓLIDA DEGASES 1077 - 2.1 - PROPILENO',
 'SÓLIDOS\xa0INFLAMABLES;\xa0SUSTANCIAS\xa0QUE\xa0PUEDEN\xa0EXPERIMENTAR\xa0COMBUSTIÓN\xa0ESPONTÁNEA;\xa0S\xa02006\xa0-\xa04.2\xa0-\xa0PLÁSTICOS\xa0A\xa0BASE\xa0DE\xa0NITROCELULOSA\xa0QUE\xa0EXPERIMENTAN\xa0CALENTAMIENTO\xa0ES',
 'SÓLIDOS\xa0INFLAMABLES;\xa0SUSTANCIAS\xa0QUE\xa0PUEDEN\xa0EXPERIMENTAR\xa0COMBUSTIÓN\xa0ESPONTÁNEA;\xa0S\xa03208\xa0-\xa04.3\xa0-\xa0SUSTANCIA\xa0METÁLICA\xa0QUE\xa0REACCIONA\xa0CON\xa0EL\xa0AGUA;\xa0N.E.P.',
 'SÓLIDOS\xa0INFLAMABLES;\xa0SUSTANCIAS\xa0QUE\xa0PUEDEN\xa0EXPERIMENTAR\xa0COMBUSTIÓN\xa0ESPONTÁNEA;\xa0S\xa03255\xa0-\xa04.2\xa0-\xa0HIPOCLORITO\xa0DE\xa0terc-BUTILO',
 'SUSTANCIAS TÓXICAS Y SUSTANCIAS INFECCIOSAS 1051 - 6.1 - CIANURO DE HIDRÓGENO ANHIDRO ESTABILIZADO con menos del\xa03% de agua',
       'SUSTANCIAS COMBURENTES Y PERÓXIDOS ORGÁNICOS 2469 - 5.1 - BROMATO DE CINC']

quimicos = sorted(set(quimicos))

In [18]:
#Textiles 
textiles = ['ALFOMBRAS Y DEMÁS REVESTIMIENTOS PARA EL SUELO; DE MATERIAS TEXTILES; ALFOMBRAS DE NUDO DE MATERIAS TEXTILES;  INCLUSO CONFECCIONADAS',
 'ALFOMBRAS Y DEMÁS REVESTIMIENTOS PARA EL SUELO; DE MATERIAS TEXTILES; ALFOMBRAS Y DEMAS REVESTIMIENTOS PARA EL SUELO;  DE FIELTRO;  EXCEPTO LOS DE M',
 'ALFOMBRAS Y DEMÁS REVESTIMIENTOS PARA EL SUELO; DE MATERIAS TEXTILES; ALFOMBRAS Y DEMAS REVESTIMIENTOS PARA EL SUELO;  DE MATERIAS TEXTILES TEJIDAS',
 'ALFOMBRAS Y DEMÁS REVESTIMIENTOS PARA EL SUELO; DE MATERIAS TEXTILES; ALFOMBRAS Y DEMAS REVESTIMIENTOS PARA EL SUELO;  DE MATERIAS TEXTILES;  CON ME',
 'ALFOMBRAS Y DEMÁS REVESTIMIENTOS PARA EL SUELO; DE MATERIAS TEXTILES; LAS DEMAS ALFOMBRAS Y REVESTIMIENTOS PARA EL SUELO;  DE MATERIAS TEXTILES;  IN',
 'ALFOMBRAS\xa0Y\xa0DEMÁS\xa0REVESTIMIENTOS\xa0PARA\xa0EL\xa0SUELO;\xa0DE\xa0MATERIAS\xa0TEXTILES;\xa0ALFOMBRAS\xa0DE\xa0NUDO\xa0DE\xa0MATERIAS\xa0TEXTILES;\xa0\xa0INCLUSO\xa0CONFECCIONADAS',
 'ALFOMBRAS\xa0Y\xa0DEMÁS\xa0REVESTIMIENTOS\xa0PARA\xa0EL\xa0SUELO;\xa0DE\xa0MATERIAS\xa0TEXTILES;\xa0LAS\xa0DEMAS\xa0ALFOMBRAS\xa0Y\xa0REVESTIMIENTOS\xa0PARA\xa0EL\xa0SUELO;\xa0\xa0DE\xa0MATERIAS\xa0TEXTILES;\xa0\xa0IN',
 'ALGODÓN; ALGODON CARDADO O PEINADO',
 'ALGODÓN; ALGODON SIN CARDAR NI PEINAR',
 'ALGODÓN; DESPERDICIOS DE ALGODON (INCLUIDOS LOS DESPERDICIOS DE HILADOS Y LAS HILACHAS)',
 'ALGODÓN; HILADOS DE ALGODON (EXCEPTO EL HILO DE COSER) ACONDICIONADO PARA LA VENTA AL POR',
 'ALGODÓN; HILADOS DE ALGODON (EXCEPTO EL HILO DE COSER) CON UN CONTENIDO DE ALGODON;  E',
 'ALGODÓN; HILADOS DE ALGODON (EXCEPTO EL HILO DE COSER) CON UN CONTENIDO DE ALGODON;  IN',
 'ALGODÓN; HILO DE COSER DE ALGODON;  INCLUSO ACONDICIONADO PARA LA VENTA AL POR MENOR',
 'ALGODÓN; LOS DEMAS TEJIDOS DE ALGODON',
 'ALGODÓN; TEJIDOS DE ALGODON CON UN CONTENIDO DE ALGODON SUPERIOR O IGUAL AL 85 % EN PES',
 'ALGODÓN; TEJIDOS DE ALGODON MEZCLADO EXCLUSIVA O PRINCIPALMENTE CON FIBRAS SINTETICAS',
 'ALGODÓN;\xa0ALGODON\xa0SIN\xa0CARDAR\xa0NI\xa0PEINAR',
 'TEJIDOS IMPREGNADOS; RECUBIERTOS; REVESTIDOS O ESTRATIFICADOS; ARTÍCULOS REVESTIMIENTOS DE MATERIAS TEXTILES PARA PAREDES',
 'GUATA; FIELTRO Y TELAS SIN TEJER; HILADOS ESPECIALES; CORDELES; CUERDAS Y ARTICULOS DE HILADOS;  TIRAS O FORMAS SIMILARES DE LAS PARTIDAS 5404 O 5405;',
 'GUATA; FIELTRO Y TELAS SIN TEJER; HILADOS ESPECIALES; CORDELES; CUERDAS Y CORDELES;  CUERDAS Y CORDAJES;  TRENZADOS O NO;  INCLUSO IMPREGNADOS;  RECUBIE',
 'GUATA; FIELTRO Y TELAS SIN TEJER; HILADOS ESPECIALES; CORDELES; CUERDAS Y FIELTRO;  INCLUSO IMPREGNADO;  RECUBIERTO;  REVESTIDO O ESTRATIFICADO',
 'GUATA; FIELTRO Y TELAS SIN TEJER; HILADOS ESPECIALES; CORDELES; CUERDAS Y GUATA DE MATERIAS TEXTILES Y ARTICULOS DE ESTA GUATA; FIBRAS TEXTILES DE LONGI',
 'GUATA; FIELTRO Y TELAS SIN TEJER; HILADOS ESPECIALES; CORDELES; CUERDAS Y HILADOS ENTORCHADOS;  TIRAS Y FORMAS SIMILARES DE LAS PARTIDAS 5404 O 5405;  E',
 'GUATA; FIELTRO Y TELAS SIN TEJER; HILADOS ESPECIALES; CORDELES; CUERDAS Y HILADOS METALICOS E HILADOS METALIZADOS;  INCLUSO ENTORCHADOS;  CONSTITUIDOS P',
 'GUATA; FIELTRO Y TELAS SIN TEJER; HILADOS ESPECIALES; CORDELES; CUERDAS Y HILOS Y CUERDAS DE CAUCHO;  REVESTIDOS DE TEXTILES; HILADOS TEXTILES;  TIRAS',
 'GUATA; FIELTRO Y TELAS SIN TEJER; HILADOS ESPECIALES; CORDELES; CUERDAS Y REDES DE MALLAS ANUDADAS;  EN PA?OS O EN PIEZAS;  FABRICADAS CON CORDELES;  CU',
 'GUATA; FIELTRO Y TELAS SIN TEJER; HILADOS ESPECIALES; CORDELES; CUERDAS Y TELAS SIN TEJER;  INCLUSO IMPREGNADAS;  RECUBIERTAS;  REVESTIDAS O ESTRATIFICA',
 'GUATA;\xa0FIELTRO\xa0Y\xa0TELAS\xa0SIN\xa0TEJER;\xa0HILADOS\xa0ESPECIALES;\xa0CORDELES;\xa0CUERDAS\xa0Y\xa0CORDELES;\xa0\xa0CUERDAS\xa0Y\xa0CORDAJES;\xa0\xa0TRENZADOS\xa0O\xa0NO;\xa0\xa0INCLUSO\xa0IMPREGNADOS;\xa0\xa0RECUBIE',
'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; GUANTES;  MITONES Y MANOPLAS;  DE PUNTO',
'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; LOS DEMAS COMPLEMENTOS ( ACCESORIOS ) DE VESTIR CONFECCIONADOS;  DE PUNTO; PAR',
'TEJIDOS DE PUNTO LOS DEMÁS TEJIDOS DE PUNTO',
'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; TEJIDOS DE FIBRAS ARTIFICIALES DISCONTINUAS',
'FIBRAS SINTÉTICAS O ARTIFICIALES DISCONTINUAS; TEJIDOS DE FIBRAS SINTETICAS DISCONTINUAS CON UN CONTENIDO DE ESTAS FIBRAS INF',
'PELETERÍA Y CONFECCIONES DE PELETERÍA; PELETERÍA ARTIFICIAL O FICTICIA; PRENDAS;  COMPLEMENTOS DE VESTIR Y DEMAS ARTICULOS DE PELETERIA',
 'TEJIDOS DE PUNTO TEJIDOS DE PUNTO DE ANCHURA INFERIOR O IGUAL A 30 CM, EXCEPTO LOS DE LAS PARTIDAS 6001 Ó 6002',
 'TEJIDOS DE PUNTO TEJIDOS DE PUNTO DE ANCHURA SUPERIOR A 30 CM, CON UN CONTENIDO DE HILADOS DE ELASTÓMEROS O DE HILOS DE CAUCHO SUPERIOR O IGUAL AL 5% EN PESO, EXCEPTO LOS DE LA PARTIDA 6001',
 'TEJIDOS DE PUNTO TEJIDOS DE PUNTO POR URDIMBRE (INCLUIDOS LOS OBTENIDOS EN TELARES DE PASAMANERÍA), EXCEPTO LOS DE LAS PARTIDAS 6001 A 6004',
 'TEJIDOS DE PUNTO; TEJIDOS DE PUNTO DE ANCHURA INFERIOR O IGUAL A 30 CM, CON UN CONTENIDO DE HILOS DE CAUCHO SUPERIOR O IGUAL AL 5 PORCIENTO EN PESO, EXCEPTO LOS DE LA PARTIDA 6001',
 'TEJIDOS DE PUNTO; TERCIOPELO;  FELPA (INCLUIDOS LOS TEJIDOS DE PUNTO <DE PELO LARGO>) Y TEJIDOS',
 'TEJIDOS ESPECIALES; SUPERFICIES TEXTILES CON PELO INSERTADO; ENCAJES; BORDADOS EN PIEZAS;  TIRAS O MOTIVOS',
 'TEJIDOS ESPECIALES; SUPERFICIES TEXTILES CON PELO INSERTADO; ENCAJES; CINTAS;  EXCEPTO LOS ARTICULOS DE LA PARTIDA 5807; CINTAS SIN TRAMA;  DE HILAD',
 'TEJIDOS ESPECIALES; SUPERFICIES TEXTILES CON PELO INSERTADO; ENCAJES; ETIQUETAS;  ESCUDOS Y ARTICULOS SIMILARES;  DE MATERIAS TEXTILES;  EN PIEZA;',
 'TEJIDOS ESPECIALES; SUPERFICIES TEXTILES CON PELO INSERTADO; ENCAJES; PRODUCTOS TEXTILES ACOLCHADOS EN PIEZA;  CONSTITUIDOS POR UNA O VARIAS CAPAS D',
 'TEJIDOS ESPECIALES; SUPERFICIES TEXTILES CON PELO INSERTADO; ENCAJES; TAPICERIA TEJIDA A MANO (GOBELINOS;  FLANDES;  AUBUSSON;  BEAUVAIS Y SIMILARES',
 'TEJIDOS ESPECIALES; SUPERFICIES TEXTILES CON PELO INSERTADO; ENCAJES; TEJIDOS CON BUCLES DEL TIPO PARA TOALLAS;  EXCEPTO LOS PRODUCTOS DE LA PARTID',
 'TEJIDOS ESPECIALES; SUPERFICIES TEXTILES CON PELO INSERTADO; ENCAJES; TEJIDOS DE GASA DE VUELTA;  EXCEPTO LOS PRODUCTOS DE LA PARTIDA 5806',
 'TEJIDOS ESPECIALES; SUPERFICIES TEXTILES CON PELO INSERTADO; ENCAJES; TEJIDOS DE HILOS DE METAL Y TEJIDOS DE HILADOS METALICOS O DE HILADOS TEXTILE',
 'TEJIDOS ESPECIALES; SUPERFICIES TEXTILES CON PELO INSERTADO; ENCAJES; TERCIOPELO Y FELPA EXCEPTO LOS DE PUNTO;  Y TEJIDOS DE CHENILLA;  EXCEPTO LOS',
 'TEJIDOS ESPECIALES; SUPERFICIES TEXTILES CON PELO INSERTADO; ENCAJES; TRENZAS EN PIEZA; ARTICULOS DE PASAMANERIA Y ARTICULOS ORNAMENTALES ANALOGOS',
 'TEJIDOS ESPECIALES; SUPERFICIES TEXTILES CON PELO INSERTADO; ENCAJES; TUL;  TUL-BOBINOT Y TEJIDOS DE MALLAS ANUDADAS; ENCAJES EN PIEZAS;  TIRAS O MO',
 'TEJIDOS IMPREGNADOS; RECUBIERTOS; REVESTIDOS O ESTRATIFICADOS; ARTÍCULOS CORREAS TRANSPORTADORAS O DE TRANSMISION;  DE MATERIAS TEXTILES;  INCLUSO IMPR',
 'TEJIDOS IMPREGNADOS; RECUBIERTOS; REVESTIDOS O ESTRATIFICADOS; ARTÍCULOS LAS DEMAS TELAS IMPREGNADAS;  RECUBIERTAS O REVESTIDAS; LIENZOS PINTADOS PARA',
 'TEJIDOS IMPREGNADOS; RECUBIERTOS; REVESTIDOS O ESTRATIFICADOS; ARTÍCULOS LINOLEO;  INCLUSO CORTADO; REVESTIMIENTOS PARA EL SUELO FORMADOS POR UN RECUBR',
 'TEJIDOS IMPREGNADOS; RECUBIERTOS; REVESTIDOS O ESTRATIFICADOS; ARTÍCULOS MANGUERAS PARA BOMBAS Y TUBOS SIMILARES;  DE MATERIAS TEXTILES;  INCLUSO CON A',
 'TEJIDOS IMPREGNADOS; RECUBIERTOS; REVESTIDOS O ESTRATIFICADOS; ARTÍCULOS MECHAS DE MATERIAS TEXTILES TEJIDAS;  TRENZADAS O DE PUNTO;  PARA LAMPARAS;  H',
 'TEJIDOS IMPREGNADOS; RECUBIERTOS; REVESTIDOS O ESTRATIFICADOS; ARTÍCULOS NAPAS TRAMADAS PARA NEUMATICOS FABRICADAS CON HILOS DE ALTA TENACIDAD DE NAILO',
 'TEJIDOS IMPREGNADOS; RECUBIERTOS; REVESTIDOS O ESTRATIFICADOS; ARTÍCULOS PRODUCTOS Y ARTICULOS TEXTILES PARA USOS TECNICOS MENCIONADOS EN LA NOTA 7 DE ES',
 'TEJIDOS IMPREGNADOS; RECUBIERTOS; REVESTIDOS O ESTRATIFICADOS; ARTÍCULOS REVESTIMIENTOS DE MATERIAS TEXTILES PARA PAREDES',
 'TEJIDOS IMPREGNADOS; RECUBIERTOS; REVESTIDOS O ESTRATIFICADOS; ARTÍCULOS TELAS CAUCHUTADAS;  EXCEPTO LAS DE LA PARTIDA 5902',
 'TEJIDOS IMPREGNADOS; RECUBIERTOS; REVESTIDOS O ESTRATIFICADOS; ARTÍCULOS TELAS IMPREGNADASS;  RECUBIERTAS;  REVESTIDAS O ESTRATIFICADAS CON PLASTICO;',
 'TEJIDOS IMPREGNADOS; RECUBIERTOS; REVESTIDOS O ESTRATIFICADOS; ARTÍCULOS TELAS RECUBIERTAS DE COLA O MATERIAS AMILACEAS;  DEL TIPO DE LOS UTILIZADOS PA',
 'TEJIDOS\xa0DE\xa0PUNTO\xa0TEJIDOS\xa0DE\xa0PUNTO\xa0DE\xa0ANCHURA\xa0INFERIOR\xa0O\xa0IGUAL\xa0A\xa030\xa0CM,\xa0EXCEPTO\xa0LOS\xa0DE\xa0LAS\xa0PARTIDAS\xa06001\xa0Ó\xa06002',
 'TEJIDOS\xa0ESPECIALES;\xa0SUPERFICIES\xa0TEXTILES\xa0CON\xa0PELO\xa0INSERTADO;\xa0ENCAJES;\xa0BORDADOS\xa0EN\xa0PIEZAS;\xa0\xa0TIRAS\xa0O\xa0MOTIVOS',
 'TEJIDOS\xa0IMPREGNADOS;\xa0RECUBIERTOS;\xa0REVESTIDOS\xa0O\xa0ESTRATIFICADOS;\xa0ARTÍCULOS\xa0CORREAS\xa0TRANSPORTADORAS\xa0O\xa0DE\xa0TRANSMISION;\xa0\xa0DE\xa0MATERIAS\xa0TEXTILES;\xa0\xa0INCLUSO\xa0IMPR',
 'TEJIDOS\xa0IMPREGNADOS;\xa0RECUBIERTOS;\xa0REVESTIDOS\xa0O\xa0ESTRATIFICADOS;\xa0ARTÍCULOS\xa0PRODUCTOS\xa0Y\xa0ARTICULOS\xa0TEXTILES\xa0PARA\xa0USOS\xa0TECNICOS\xa0MENCIONADOS\xa0EN\xa0LA\xa0NOTA\xa07\xa0DE\xa0ES',
 'TEJIDOS\xa0IMPREGNADOS;\xa0RECUBIERTOS;\xa0REVESTIDOS\xa0O\xa0ESTRATIFICADOS;\xa0ARTÍCULOS\xa0TELAS\xa0RECUBIERTAS\xa0DE\xa0COLA\xa0O\xa0MATERIAS\xa0AMILACEAS;\xa0\xa0DEL\xa0TIPO\xa0DE\xa0LOS\xa0UTILIZADOS\xa0PA',
  'SEDA; CAPULLOS DE SEDA APTOS PARA EL DEVANADO',
 'SEDA; DESPERDICIOS DE SEDA (INCLUIDOS LOS CAPULLOS DE SEDA NO APTOS PARA EL DEVANADO',
 'SEDA; HILADOS DE DESPERDICIOS DE SEDA SIN ACONDICIONAR PARA LA VENTA AL POR MENOR',
 'SEDA; HILADOS DE SEDA (EXCEPTO LOS HILADOS DE DESPERDICIOS DE SEDA) SIN ACONDICIONAR P',
 'SEDA; HILADOS DE SEDA O DE DESPERDICIOS DE SEDA;  ACONDICIONADOS PARA LA VENTA AL PO',
 'SEDA; SEDA CRUDA SIN TORCER',
 'SEDA; TEJIDOS DE SEDA O DE DESPERDICIOS DE SEDA',
 'SEDA;\xa0DESPERDICIOS\xa0DE\xa0SEDA\xa0(INCLUIDOS\xa0LOS\xa0CAPULLOS\xa0DE\xa0SEDA\xa0NO\xa0APTOS\xa0PARA\xa0EL\xa0DEVANADO',
  'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; ABRIGOS;  CHAQUETONES;  CAPAS;  ANORAKS;  CAZADORAS Y ARTICULOS SIMILARES;  D',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; CALZAS;  <PANTIMEDIAS>;  LEOTARDOS;  MEDIAS;  CALCETINES Y DEMAS ARTICULOS DE',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; CALZONCILLOS ( SLIPS );  CAMISONES;  PIJAMAS;  ALBORNOCES;  BATAS Y ARTICULOS',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; CAMISAS DE PUNTO PARA HOMBRES O NI?OS',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; CAMISAS;  BLUSAS;  BLUSAS CAMISERAS;  DE PUNTO;  PARA MUJERES O NI?AS',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; COMBINACIONES;  ENAGUAS;  BRAGAS ( BOMBACHAS  CALZONES);  ( INCLUSO LAS QUE N',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; CONJUNTOS DE ABRIGO PARA ENTRENAMIENTO O DEPORTE;  MONOS ( OVEROLES ) Y CONJUN',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; LAS DEMAS PRENDAS DE VESTIR;  DE PUNTO',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; LOS DEMAS COMPLEMENTOS ( ACCESORIOS ) DE VESTIR CONFECCIONADOS;  DE PUNTO; PAR',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; PRENDAS CONFECCIONADAS CON TEJIDOS DE PUNTO DE LAS PARTIDAS 5903;  5906 O 5907',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; PRENDAS Y COMPLEMENTOS ( ACCESORIOS ) DE VESTIR;  DE PUNTO;  PARA BEBES',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; SUETERES( JERSEIS );  <PULLOVERS>;  <CARDIGANS>;  CHALECOS Y ARTICULOS SIMILAR',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; T-SHIRTS Y CAMISETAS INTERIORES DE PUNTO',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; TRAJES ( AMBOS O TERNOS );  CONJUNTOS;  CHAQUETAS ( SACOS );  PANTALONES LARGO',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; DE PUNTO; TRAJES SASTRE;  CONJUNTOS;  CHAQUETAS ( SACOS );  VESTIDOS;  FALDAS;  FALDAS P',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; ABRIGOS;  CHAQUETONES;  CAPAS;  ANORAKS;  CAZADORAS Y ARTICULOS SIMILARES;  PA',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; CAMISAS PARA HOMBRES O NI?OS',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; CAMISAS;  BLUSAS Y BLUSAS CAMISERAS;  PARA MUJERES O NI?AS',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; CAMISETAS INTERIORES;  CALZONCILLOS;  SLIPS CAMISONES;  PIJAMAS;  ALBORNOCES',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; CAMISETAS INTERIORES;  COMBINACIONES;  ENAGUAS;  BRAGAS; (BOMBACHAS;  CALZONES',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; CHALES;  PA?UELOS DE CUELLO;   BUFANDAS;  MANTILLAS;  VELOS Y ARTICULOS SIMILA',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; CONJUNTOS DE ABRIGO PARA ENTRENAMIENTO O DEPORTE;  MONOS ( OVEROLES ) Y CONJUN',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; CORBATAS Y LAZOS SIMILARES',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; GUANTES;  MITONES Y MANOPLAS',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; LOS DEMAS COMPLEMENTOS (ACCESORIOS) DE VESTIR; PARTES DE PRENDAS O DE COMPLEME',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; PA?UELOS DE BOLSILLO',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; PRENDAS DE VESTIR CONFECCIONADAS CON PRODUCTOS DE LAS PARTIDAS 5602;  5603;  5',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; PRENDAS Y COMPLEMENTOS DE VESTIR;  PARA BEBES',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; SOSTENES; (CORPI?OS);  FAJAS;  CORSES;  TIRANTES; (TIRADORES);  LIGAS Y ARTICU',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; TRAJES (AMBOS O TERNOS);  CONJUNTOS;  CHAQUETAS;  (SACOS) PANTALONES;  PANTALO',
 'PRENDAS Y COMPLEMENTOS DE VESTIR; EXCEPTO LOS DE PUNTO; TRAJES SASTRE;  CONJUNTOS;  CHAQUETAS; (SACOS) VESTIDOS;  FALDAS;  FALDAS PANT',
 'PRENDAS\xa0Y\xa0COMPLEMENTOS\xa0DE\xa0VESTIR;\xa0DE\xa0PUNTO;\xa0ABRIGOS;\xa0\xa0CHAQUETONES;\xa0\xa0CAPAS;\xa0\xa0ANORAKS;\xa0\xa0CAZADORAS\xa0Y\xa0ARTICULOS\xa0SIMILARES;\xa0\xa0D',
 'PRENDAS\xa0Y\xa0COMPLEMENTOS\xa0DE\xa0VESTIR;\xa0DE\xa0PUNTO;\xa0CALZAS;\xa0\xa0<PANTIMEDIAS>;\xa0\xa0LEOTARDOS;\xa0\xa0MEDIAS;\xa0\xa0CALCETINES\xa0Y\xa0DEMAS\xa0ARTICULOS\xa0DE',
 'PIELES (EXCEPTO LA PELETERÍA) Y CUEROS PIELES (EXCEPTO LA PELETERÍA) Y CUEROS  CUEROS PREPARADOS DESPUÉS DEL CURTIDO O DEL SECADO Y CUEROS Y PIELES APERGAMINADOS, DE OVINO, DEPILADOS, INCLUSO DIVIDIDO',
 'PIELES (EXCEPTO LA PELETERÍA) Y CUEROS; CUERO ARTIFICIAL O REGENERADO A BASE DE CUERO O DE FIBRAS DE CUERO;  EN PLANCH',
 'PIELES (EXCEPTO LA PELETERÍA) Y CUEROS; CUEROS Y PIELES AGAMUZADOS (INCLUIDO EL AGAMUZADO COMBINADO AL ACEITE)',
 'PIELES (EXCEPTO LA PELETERÍA) Y CUEROS; CUEROS Y PIELES BARNIZADOS O REVESTIDOS; CUEROS Y PIELES METALIZADOS',
 'PIELES (EXCEPTO LA PELETERÍA) Y CUEROS; CUEROS Y PIELES;  DE BOVINO Y DE EQUINO;  DEPILADOS;  PREPARADOS;  EXCEPTO LO',
 'PIELES (EXCEPTO LA PELETERÍA) Y CUEROS; CUEROS Y PIELES;  EN BRUTO;  DE BOVINO O DE EQUINO (FRESCOS O SALADOS;  SECOS',
 'PIELES (EXCEPTO LA PELETERÍA) Y CUEROS; LOS DEMAS CUEROS Y PIELES;  EN BRUTO (FRESCOS O SALADOS;  SECOS;  ENCALADOS;',
 'PIELES (EXCEPTO LA PELETERÍA) Y CUEROS; PIELES DEPILADAS DE CAPRINO;  PREPARADAS;  EXCEPTO LAS DE LAS PARTIDAS 4108 O',
 'PIELES (EXCEPTO LA PELETERÍA) Y CUEROS; PIELES DEPILADAS DE LOS DEMAS ANIMALES Y PIELES DE ANIMALES SIN PELO;  PREPARA',
 'PIELES (EXCEPTO LA PELETERÍA) Y CUEROS; PIELES DEPILADAS DE OVINO;  PREPARADAS;  EXCEPTO LAS DE LAS PARTIDAS 4108 O 41',
 'PIELES (EXCEPTO LA PELETERÍA) Y CUEROS; PIELES EN BRUTO DE OVINO (FRESCAS O SALADAS;  SECAS;  ENCALADAS;  PIQUELADAS',
 'PIELES (EXCEPTO LA PELETERÍA) Y CUEROS; RECORTES Y DEMAS DESPERDICIOS DE CUERO O DE PIELES;  PREPARADOS;  O DE CUERO A',
 'PIELES\xa0(EXCEPTO\xa0LA\xa0PELETERÍA)\xa0Y\xa0CUEROS;\xa0CUEROS\xa0Y\xa0PIELES;\xa0\xa0DE\xa0BOVINO\xa0Y\xa0DE\xa0EQUINO;\xa0\xa0DEPILADOS;\xa0\xa0PREPARADOS;\xa0\xa0EXCEPTO\xa0LO',
  'PELETERÍA Y CONFECCIONES DE PELETERÍA; PELETERÍA ARTIFICIAL O FICTICIA; PELETERIA ARTIFICIAL O FACTICIA Y ARTICULOS DE PELETERIA ARTIFICIAL O FACTICIA',
 'PELETERÍA Y CONFECCIONES DE PELETERÍA; PELETERÍA ARTIFICIAL O FICTICIA; PELETERIA CURTIDA O ADOBADA (INCLUIDAS LAS CABEZAS;  COLAS;  PATAS Y TROZOS;',
 'PELETERÍA Y CONFECCIONES DE PELETERÍA; PELETERÍA ARTIFICIAL O FICTICIA; PELETERIA EN BRUTO (INCLUIDAS LAS CABEZAS;  COLAS;  PATAS Y TROZOS UTILIZABLE',
 'PELETERÍA\xa0Y\xa0CONFECCIONES\xa0DE\xa0PELETERÍA;\xa0PELETERÍA\xa0ARTIFICIAL\xa0O\xa0FICTICIA;\xa0PELETERIA\xa0ARTIFICIAL\xa0O\xa0FACTICIA\xa0Y\xa0ARTICULOS\xa0DE\xa0PELETERIA\xa0ARTIFICIAL\xa0O\xa0FACTICIA',
 'PELETERÍA\xa0Y\xa0CONFECCIONES\xa0DE\xa0PELETERÍA;\xa0PELETERÍA\xa0ARTIFICIAL\xa0O\xa0FICTICIA;\xa0PELETERIA\xa0EN\xa0BRUTO\xa0(INCLUIDAS\xa0LAS\xa0CABEZAS;\xa0\xa0COLAS;\xa0\xa0PATAS\xa0Y\xa0TROZOS\xa0UTILIZABLE',
 'PELETERÍA\xa0Y\xa0CONFECCIONES\xa0DE\xa0PELETERÍA;\xa0PELETERÍA\xa0ARTIFICIAL\xa0O\xa0FICTICIA;\xa0PRENDAS;\xa0\xa0COMPLEMENTOS\xa0DE\xa0VESTIR\xa0Y\xa0DEMAS\xa0ARTICULOS\xa0DE\xa0PELETERIA',
 'LANA Y PELO FINO U ORDINARIO; HILADOS Y TEJIDOS DE CRIN; DESPERDICIOS DE LANA O DE PELO FINO U ORDINARIO;  INCLUIDOS LOS DESPERDICIOS D',
 'LANA Y PELO FINO U ORDINARIO; HILADOS Y TEJIDOS DE CRIN; HILACHAS DE LANA O DE PELO FINO U ORDINARIO',
 'LANA Y PELO FINO U ORDINARIO; HILADOS Y TEJIDOS DE CRIN; HILADOS DE LANA CARDADA SIN ACONDICIONAR PARA LA VENTA AL POR MENOR',
 'LANA Y PELO FINO U ORDINARIO; HILADOS Y TEJIDOS DE CRIN; HILADOS DE LANA O DE PELO FINO;  ACONDICIONADOS PARA LA VENTA AL POR MENOR',
 'LANA Y PELO FINO U ORDINARIO; HILADOS Y TEJIDOS DE CRIN; HILADOS DE LANA PEINADA SIN ACONDICIONAR PARA LA VENTA AL POR MENOR',
 'LANA Y PELO FINO U ORDINARIO; HILADOS Y TEJIDOS DE CRIN; HILADOS DE PELO FINO CARDADO O PEINADO SIN ACONDICIONAR PARA LA VENTA AL POR MEN',
 'LANA Y PELO FINO U ORDINARIO; HILADOS Y TEJIDOS DE CRIN; HILADOS DE PELO ORDINARIO O DE CRIN (INCLUIDOS LOS HILADOS DE CRIN ENTORCHADOS',
 'LANA Y PELO FINO U ORDINARIO; HILADOS Y TEJIDOS DE CRIN; LANA SIN CARDAR NI PEINAR',
 'LANA Y PELO FINO U ORDINARIO; HILADOS Y TEJIDOS DE CRIN; LANA Y PELO FINO U ORDINARIO;  CARDADOS O PEINADOS (INCLUIDA LA LANA PEINADA',
 'LANA Y PELO FINO U ORDINARIO; HILADOS Y TEJIDOS DE CRIN; PELO FINO U ORDINARIO;  SIN CARDAR NI PEINAR',
 'LANA Y PELO FINO U ORDINARIO; HILADOS Y TEJIDOS DE CRIN; TEJIDOS DE LANA CARDADA O DE PELO FINO CARDADO',
 'LANA Y PELO FINO U ORDINARIO; HILADOS Y TEJIDOS DE CRIN; TEJIDOS DE LANA PEINADA O DE PELO FINO PEINADO',
 'LAS DEMÁS FIBRAS TEXTILES VEGETALES; HILADOS DE PAPEL Y TEJIDOS DE HILADOS CA?AMO (CANNABIS SATIVA L;) EN BRUTO O TRABAJADO;  PERO SIN HILAR; ESTOPAS Y D',
 'LAS DEMÁS FIBRAS TEXTILES VEGETALES; HILADOS DE PAPEL Y TEJIDOS DE HILADOS COCO;  ABACA (CA?AMO DE MANILA O MUSA TEXTILIS NEE);  RAMIO Y DEMAS FIBRAS TEX',
 'LAS DEMÁS FIBRAS TEXTILES VEGETALES; HILADOS DE PAPEL Y TEJIDOS DE HILADOS HILADOS DE LAS DEMAS FIBRAS VEGETALES; HILADOS DE PAPEL',
 'LAS DEMÁS FIBRAS TEXTILES VEGETALES; HILADOS DE PAPEL Y TEJIDOS DE HILADOS HILADOS DE LINO',
 'LAS DEMÁS FIBRAS TEXTILES VEGETALES; HILADOS DE PAPEL Y TEJIDOS DE HILADOS HILADOS DE YUTE O DEMAS FIBRAS TEXTILES DEL LIBER DE LA PARTIDA 5303',
 'LAS DEMÁS FIBRAS TEXTILES VEGETALES; HILADOS DE PAPEL Y TEJIDOS DE HILADOS LINO EN BRUTO O TRABAJADO;  PERO SIN HILAR; ESTOPAS Y DESPERDICIOS DE LINO (IN',
 'LAS DEMÁS FIBRAS TEXTILES VEGETALES; HILADOS DE PAPEL Y TEJIDOS DE HILADOS SISAL Y DEMAS FIBRAS TEXTILES DEL GENERO AGAVE;  EN BRUTO O TRABAJADOS;  PERO',
 'LAS DEMÁS FIBRAS TEXTILES VEGETALES; HILADOS DE PAPEL Y TEJIDOS DE HILADOS TEJIDOS DE LAS DEMAS FIBRAS TEXTILES VEGETALES; TEJIDOS DE HILADOS DE PAPEL',
 'LAS DEMÁS FIBRAS TEXTILES VEGETALES; HILADOS DE PAPEL Y TEJIDOS DE HILADOS TEJIDOS DE LINO',
 'LAS DEMÁS FIBRAS TEXTILES VEGETALES; HILADOS DE PAPEL Y TEJIDOS DE HILADOS TEJIDOS DE YUTE O DEMAS FIBRAS TEXTILES DEL LIBER DE LA PARTIDA 5303',
 'LAS DEMÁS FIBRAS TEXTILES VEGETALES; HILADOS DE PAPEL Y TEJIDOS DE HILADOS YUTE Y DEMAS FIBRAS TEXTILES DEL LIBER (CON EXCLUSION DEL LINO;  CA?AMO Y RAMI',
 'LAS\xa0DEMÁS\xa0FIBRAS\xa0TEXTILES\xa0VEGETALES;\xa0HILADOS\xa0DE\xa0PAPEL\xa0Y\xa0TEJIDOS\xa0DE\xa0HILADOS\xa0HILADOS\xa0DE\xa0LINO',
 'LAS\xa0DEMÁS\xa0FIBRAS\xa0TEXTILES\xa0VEGETALES;\xa0HILADOS\xa0DE\xa0PAPEL\xa0Y\xa0TEJIDOS\xa0DE\xa0HILADOS\xa0TEJIDOS\xa0DE\xa0LINO',
 'LOS DEMÁS ARTÍCULOS TEXTILES CONFECCIONADOS; CONJUNTOS O SURTIDOS; ARTICULOS DE PRENDERIA',
 'LOS DEMÁS ARTÍCULOS TEXTILES CONFECCIONADOS; CONJUNTOS O SURTIDOS; JUEGOS CONSTITUIDOS POR PIEZAS DE TEJIDO E HILADOS;  INCLUSO CON ACCESORIOS;',
 'LOS DEMÁS ARTÍCULOS TEXTILES CONFECCIONADOS; CONJUNTOS O SURTIDOS; LOS DEMAS ARTICULOS CONFECCIONADOS;  INCLUIDOS LOS PATRONES PARA PRENDAS DE VE',
 'LOS DEMÁS ARTÍCULOS TEXTILES CONFECCIONADOS; CONJUNTOS O SURTIDOS; LOS DEMAS ARTICULOS DE TAPICERIA;  EXCEPTO LOS DE LA PARTIDA 9404',
 'LOS DEMÁS ARTÍCULOS TEXTILES CONFECCIONADOS; CONJUNTOS O SURTIDOS; MANTAS',
 'LOS DEMÁS ARTÍCULOS TEXTILES CONFECCIONADOS; CONJUNTOS O SURTIDOS; ROPA DE CAMA;  DE MESA;   TOCADOR O COCINA',
 'LOS DEMÁS ARTÍCULOS TEXTILES CONFECCIONADOS; CONJUNTOS O SURTIDOS; SACOS  (BOLSAS) Y TALEGAS;  PARA ENVASAR',
 'LOS DEMÁS ARTÍCULOS TEXTILES CONFECCIONADOS; CONJUNTOS O SURTIDOS; TOLDOS DE CUALQUIER CLASE; TIENDAS (CARPAS);  VELAS PARA EMBARCACIONES;  DESLI',
 'LOS DEMÁS ARTÍCULOS TEXTILES CONFECCIONADOS; CONJUNTOS O SURTIDOS; TRAPOS;  CORDELES;  CUERDAS Y CORDAJES;  DE MATERIAS TEXTILES;  EN DESPERDICIO',
 'LOS DEMÁS ARTÍCULOS TEXTILES CONFECCIONADOS; CONJUNTOS O SURTIDOS; VISILLOS Y CORTINAS; GUARDAMALLETAS Y RODAPIES DE CAMA',
 'CALZADO; POLAINAS; BOTINES Y ARTÍCULOS ANÁLOGOS; PARTES DE ESTOS ARTÍCULOS; CALZADO CON SUELA DE CAUCHO;  PLASTICO;  CUERO NATURAL;  REGENERADO Y PARTE SU',
 'CALZADO; POLAINAS; BOTINES Y ARTÍCULOS ANÁLOGOS; PARTES DE ESTOS ARTÍCULOS; CALZADO IMPERMEABLE CON SUELA Y PARTE SUPERIOR DE CAUCHO O DE PLASTICO;  CUYA',
 'CALZADO; POLAINAS; BOTINES Y ARTÍCULOS ANÁLOGOS; PARTES DE ESTOS ARTÍCULOS; LOS DEMAS CALZADOS',
 'CALZADO; POLAINAS; BOTINES Y ARTÍCULOS ANÁLOGOS; PARTES DE ESTOS ARTÍCULOS; LOS DEMAS CALZADOS CON SUELA Y PARTE SUPERIOR DE CAUCHO O DE PLASTICO',
 'CALZADO; POLAINAS; BOTINES Y ARTÍCULOS ANÁLOGOS; PARTES DE ESTOS ARTÍCULOS; PARTES DE CALZADO ( INCLUSO LAS PARTES SUPERIORES FIJADAS A LAS PALMILLAS DIST',
 'CALZADO;\xa0POLAINAS;\xa0BOTINES\xa0Y\xa0ARTÍCULOS\xa0ANÁLOGOS;\xa0PARTES\xa0DE\xa0ESTOS\xa0ARTÍCULOS;\xa0PARTES\xa0DE\xa0CALZADO\xa0(\xa0INCLUSO\xa0LAS\xa0PARTES\xa0SUPERIORES\xa0FIJADAS\xa0A\xa0LAS\xa0PALMILLAS\xa0DIST'

 ]

textiles = sorted(set(textiles))

In [19]:
#Otros
otros = ['ANIMALES VIVOS CABALLOS,  ASNOS,  MULOS Y BURDEGANOS,   VIVOS',
 'ANIMALES VIVOS; ANIMALES VIVOS DE LA ESPECIE BOVINA',
 'ANIMALES VIVOS; ANIMALES VIVOS DE LA ESPECIE PORCINA',
 'ANIMALES VIVOS; GALLOS; GALLINAS; PATOS; GANSOS; PAVOS Y PINTADAS DE LAS ESPECIES DOMESTICAS',
 'ANIMALES VIVOS; LOS DEMAS ANIMALES VIVOS',
 'ANIMALES\xa0VIVOS;\xa0ANIMALES\xa0VIVOS\xa0DE\xa0LA\xa0ESPECIE\xa0PORCINA',
 'OTROS MENAJES',
 'OTROS MUDANZAS',
 'ANIMALES VIVOS; LOS DEMAS ANIMALES VIVOS',
 'PLANTAS VIVAS Y PRODUCTOS DE LA FLORICULTURA; LAS DEMAS PLANTAS VIVAS(INCLUIDAS SUS RAICES) ESQUEJES E INJERTOS;  BLANCO DE',
 'PLANTAS VIVAS Y PRODUCTOS DE LA FLORICULTURA; FLORES Y CAPULLOS; CORTADOS PARA RAMOS O ADORNOS; FRESCOS; SECOS; BLANQUEADOS',
 'TABACO Y SUCEDÁNEOS DEL TABACO ELABORADOS; CIGARROS O PUROS (INCLUSO DESPUNTADOS);  PURITOS Y CIGARRILLOS;  DE TABACO O D',
 'TABACO Y SUCEDÁNEOS DEL TABACO ELABORADOS; LOS DEMAS TABACOS Y SUCEDANEOS DEL TABACO;  ELABORADOS; TABACO HOMOGENEIZADO',
 'TABACO Y SUCEDÁNEOS DEL TABACO ELABORADOS; TABACO EN RAMA O SIN ELABORAR; DESPERDICIOS DE TABACO',
 'TABACO\xa0Y\xa0SUCEDÁNEOS\xa0DEL\xa0TABACO\xa0ELABORADOS;\xa0LOS\xa0DEMAS\xa0TABACOS\xa0Y\xa0SUCEDANEOS\xa0DEL\xa0TABACO;\xa0\xa0ELABORADOS;\xa0TABACO\xa0HOMOGENEIZADO',
 'TABACO\xa0Y\xa0SUCEDÁNEOS\xa0DEL\xa0TABACO\xa0ELABORADOS;\xa0TABACO\xa0EN\xa0RAMA\xa0O\xa0SIN\xa0ELABORAR;\xa0DESPERDICIOS\xa0DE\xa0TABACO',
]
otros = sorted(set(otros))


In [20]:
categories = [alimentos,bebidas,construccion,combustibles,manufacturas,mineria,papel_carton,plasticos,paqueteo,quimicos,textiles,otros]
categories_names = ['Alimentos','Bebidas','Construccion','Combustibles','Manufacturas','Mineria','Papel y carton','Plásticos y polímeros','Paqueteo','Químicos','Textiles','Otros']


In [21]:
category_dict = {}
for category in categories:
  for product in category:
    category_dict[product] = categories_names[categories.index(category)]

category_dict['VARIOS PRODUCTOS VARIOS'] = 'VARIOS PRODUCTOS VARIOS'
category_dict['VARIOS CONTENEDOR VACIO'] = 'CONTENEDOR VACIO'

In [22]:
df_rndc['category'] = df_rndc['product'].map(category_dict)
df_rndc['category'] = df_rndc['category'].str.upper()

#Merge RNDC

In [23]:
#Drop some columns that aren't useful in this case
df_rndc =  df_rndc.drop( columns = ['city_cod','paid_vale','cod_product','driver','category','plaque','product','cod_product'])
df_rndc = df_rndc.drop_duplicates()
df_rndc.head()

,weekday,year,month,date,hour,operation_code,city,a_remittances,setting,corridor,corridor_2,setting_grouped
0,Sabado,2019,4,2019-04-13,14,True,CAPARRAPI CUNDINAMARCA,1,2,CALLE 80,CALLE 13,C2P
1,Martes,2019,4,2019-04-16,15,True,EL ROSAL CUNDINAMARCA,3,2,CALLE 80,CALLE 13,C2P
2,Lunes,2019,5,2019-05-20,8,True,GUADUAS CUNDINAMARCA,1,3S3,CALLE 80,CALLE 13,TRACTOCAMIONES
3,Jueves,2019,1,2019-01-31,21,True,PEREIRA RISARALDA,1,2,AUTO SUR,CALLE 13,C2P
4,Lunes,2019,5,2019-05-27,8,True,GUADUAS CUNDINAMARCA,1,3S3,CALLE 80,CALLE 13,TRACTOCAMIONES


In [24]:
#Translating days of the week
dias = ['Sabado', 'Martes', 'Lunes', 'Jueves', 'Viernes', 'Miercoles','Domingo']
days = ['Saturday','Tuesday','Monday','Thursday','Friday','Wednesday','Sunday']
df_rndc['weekday'] = df_rndc['weekday'].replace(dias,days)

In [25]:
#Filtering by corridor
df_rndc_80 = df_rndc[df_rndc['corridor'] == 'CALLE 80']
df_rndc_13 = df_rndc[df_rndc['corridor'] == 'CALLE 13']

In [27]:
#Grouping by keys

#df_rndc_setting = df_rndc[['date','hour','month','weekday','setting_grouped','operation_code']].groupby(['date','hour','month','weekday','setting_grouped']).count().reset_index()
#df_rndc_setting.columns = ['date', 'hour','month','weekday', 'setting', 'count_by_tipology']

df_rndc_setting_80 = df_rndc_80[['date','hour','month','weekday','setting_grouped','operation_code']].groupby(['date','hour','month','weekday','setting_grouped']).count().reset_index()
df_rndc_setting_80.columns = ['date', 'hour','month','weekday', 'setting', 'count_by_tipology']

df_rndc_setting_13 = df_rndc_80[['date','hour','month','weekday','setting_grouped','operation_code']].groupby(['date','hour','month','weekday','setting_grouped']).count().reset_index()
df_rndc_setting_13.columns = ['date', 'hour','month','weekday', 'setting', 'count_by_tipology']

In [28]:
#Creating a new column for equivalent trucks for the model

#df_rndc_setting_pivoted = df_rndc_setting.pivot(index=['date','hour','month','weekday'], columns='setting', values='count_by_tipology').reset_index().fillna(0)
#df_rndc_setting_pivoted = df_rndc_setting_pivoted.set_index(['date','hour','month','weekday'])
#df_rndc_setting_pivoted['camiones_equiv'] = df_rndc_setting_pivoted['C2G'] *2 + df_rndc_setting_pivoted['C2P'] *1 +df_rndc_setting_pivoted['TRACTOCAMIONES'] *3
#df_rndc_2merge = df_rndc_setting_pivoted

#Calle 80
df_rndc_setting_pivoted_80 = df_rndc_setting_80.pivot(index=['date','hour','month','weekday'], columns='setting', values='count_by_tipology').reset_index().fillna(0)
df_rndc_setting_pivoted_80 = df_rndc_setting_pivoted_80.set_index(['date','hour','month','weekday'])
df_rndc_setting_pivoted_80['camiones_equiv'] = df_rndc_setting_pivoted_80['C2G'] *2 + df_rndc_setting_pivoted_80['C2P'] *1 +df_rndc_setting_pivoted_80['TRACTOCAMIONES'] *3
df_rndc_2merge_80 = df_rndc_setting_pivoted_80

#Calle 13
df_rndc_setting_pivoted_13 = df_rndc_setting_13.pivot(index=['date','hour','month','weekday'], columns='setting', values='count_by_tipology').reset_index().fillna(0)
df_rndc_setting_pivoted_13 = df_rndc_setting_pivoted_13.set_index(['date','hour','month','weekday'])
df_rndc_setting_pivoted_13['camiones_equiv'] = df_rndc_setting_pivoted_13['C2G'] *2 + df_rndc_setting_pivoted_13['C2P'] *1 +df_rndc_setting_pivoted_13['TRACTOCAMIONES'] *3
df_rndc_2merge_13 = df_rndc_setting_pivoted_13


In [29]:
#proportion_by_setting = df_rndc_setting_pivoted[['C2G',	'C2P'	,'TRACTOCAMIONES']].reset_index().groupby('hour').mean()
proportion_by_setting_80 = df_rndc_setting_pivoted_80[['C2G',	'C2P'	,'TRACTOCAMIONES']].reset_index().groupby('hour').mean().drop(['month'],axis = 1)
proportion_by_setting_13 = df_rndc_setting_pivoted_13[['C2G',	'C2P'	,'TRACTOCAMIONES']].reset_index().groupby('hour').mean().drop(['month'],axis = 1)

setting,C2G,C2P,TRACTOCAMIONES
hour,,,
0,0.076433,1.477707,1.945860
1,0.210826,6.683761,3.082621
2,0.049853,2.806452,1.483871
3,0.095101,4.639769,1.613833
4,0.130556,4.127778,2.494444


In [30]:
#Dataframe to merge from RNDC for calle 13
df_rndc_2merge_13.head()

setting                        C2G  C2P  TRACTOCAMIONES  camiones_equiv
date       hour month weekday                                          
2019-01-01 0    1     Tuesday  0.0  0.0             1.0             3.0
           1    1     Tuesday  0.0  0.0             1.0             3.0
           2    1     Tuesday  0.0  0.0             1.0             3.0
           5    1     Tuesday  0.0  1.0             0.0             1.0
           6    1     Tuesday  0.0  4.0             0.0             4.0

In [31]:
#Dataframe to merge from RNDC for calle 80
df_rndc_2merge_80.head()

setting                        C2G  C2P  TRACTOCAMIONES  camiones_equiv
date       hour month weekday                                          
2019-01-01 0    1     Tuesday  0.0  0.0             1.0             3.0
           1    1     Tuesday  0.0  0.0             1.0             3.0
           2    1     Tuesday  0.0  0.0             1.0             3.0
           5    1     Tuesday  0.0  1.0             0.0             1.0
           6    1     Tuesday  0.0  4.0             0.0             4.0

# Load Bitcarrier

In [33]:
#Loading Bitcarrier
df = pd.read_sql(''' SELECT * FROM carguebogota.bitcarr ''', conn)

In [34]:
#Data cleaning
df['name_from'] = df['name_from'].replace('KR50,CL13', 'CL13')
df['name_from'] = df['name_from'].replace('CL13,KR50', 'CL13')

df['name_to'] = df['name_to'].replace('CL5A;KR47', 'KR50;KR47')
df['name_to'] = df['name_to'].replace('CL17;KR56', 'KR50;KR56')
df['name_to'] = df['name_to'].replace('KR62;CL5A', 'KR62;KR50')

#Defining direction
West_East = ['KR97;KR90', 'KR123;KR106', 'CL17;KR129', 'KR106;KR97','KR128;KR123','KR128;AV.CARACAS','KR27;KR22','KR22;AV.CARACAS',
              'KR39;KR33','KR33;KR27','KR47;KR39','KR56;KR47','KR65;KR56','KR68;KR65','KR68D;KR68','KR90;KR68D','KR62;KR50','KR50;KR47','KR119;AV.CARACAS',
              'KR24;CARACAS', 'KR69H;KR58','KR76;KR71','KR81;KR76','KR89A;KR81', 'KR58;KR24', 'KR112A;KR102','KR119;KR112A','KR71;KR69H;',
              'KR102;KR96','KR96;KR89A','KR90;KR68','KR128;KR106']
East_West = ['KR90;KR97','KR97;KR106','KR106;KR123','KR129;CL17','KR123;KR128','AV.CARACAS;KR128','KR50;KR56','KR22;KR27','AV.CARACAS;KR22',
              'KR33;KR39','KR39;KR47','KR27;KR33','KR47;KR56','KR56;KR65','KR65;KR68','KR68;KR68D','KR68D;KR90','AV.CARACAS;KR119','CARACAS;KR24',
              'KR69H;KR71','KR76;KR81','KR81;KR89A','KR24;KR58','KR58;KR69H','KR112A;KR119','KR71;KR76' ,'KR102;KR112A''KR96;KR102','KR89A;KR96',
              'KR68;KR90','KR106;KR128']
df['orientation'] = 'west_east'
df['orientation'] = df['orientation'].where(df['name_to'].isin(West_East), 'east_west')

#Change day language 
dias = ['Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo','Lunes']
days = ['Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','Monday']
df['weekday'] = df['weekday'].replace(dias,days)





In [35]:
#Direction of the corredor
#0: In Bogota
#1: Out Bogota

df['direction'] = df['orientation'] == 'east_west'
df['direction'] = df['direction'].replace([False,True],[0,1])
df.head()

,year,hour,date,distance,average_speed,quarter_hour,weekday,month,name_from,name_to,orientation,direction
0,2018,4,2018-08-12,13471.0,48.120485,04:30:00,Sunday,8,CL13,AV.CARACAS;KR128,east_west,1
1,2018,4,2018-08-12,13470.0,50.777186,04:30:00,Sunday,8,CL13,KR128;AV.CARACAS,west_east,0
2,2018,22,2018-10-11,1618.0,37.936268,22:00:00,Thursday,10,DIAG16,KR123;KR106,west_east,0
3,2018,22,2018-10-11,4205.0,33.795980,22:00:00,Thursday,10,DIAG16,KR129;CL17,east_west,1
4,2018,4,2018-08-12,3241.0,61.701417,04:30:00,Sunday,8,CL13,KR68;KR90,east_west,1


In [36]:
#Split df into CL13 and CL80
df_bitcar_13 = df[df['name_from'].isin(['DIAG16', 'CL17', 'CL13'])]
df_bitcar_80 = df[df['name_from'] == 'CL80']
#df_bitcarrier = pd.concat([df_CL13, df_CL80])
#df_bitcarrier = df_bitcarrier[df_bitcarrier['year'] == 2019]
df_bitcar_13 = df_bitcar_13[df_bitcar_13['year'] == 2019]
df_bitcar_80 = df_bitcar_80[df_bitcar_80['year'] == 2019]

df_bitcar_80.head()

,year,hour,date,distance,average_speed,quarter_hour,weekday,month,name_from,name_to,orientation,direction
512503,2019,1,2019-11-11,9642.0,36.565237,01:15:00,Monday,11,CL80,AV.CARACAS;KR119,east_west,1
512504,2019,1,2019-11-11,9642.0,37.434540,01:30:00,Monday,11,CL80,AV.CARACAS;KR119,east_west,1
512505,2019,1,2019-11-11,9642.0,38.354916,01:45:00,Monday,11,CL80,AV.CARACAS;KR119,east_west,1
512506,2019,2,2019-11-11,9642.0,40.118677,02:00:00,Monday,11,CL80,AV.CARACAS;KR119,east_west,1
512507,2019,2,2019-11-11,9642.0,27.112913,02:15:00,Monday,11,CL80,AV.CARACAS;KR119,east_west,1


In [37]:
#Average speed for the last segment of the corridor

#df_bitcarrier_tramo = df_bitcarrier[(df_bitcarrier['name_to'] == 'KR128;KR123') |(df_bitcarrier['name_to'] == 'KR123;KR128')]
df_bitcar_80_tramo = df_bitcar_80[(df_bitcar_80['name_to'] == 'KR119;KR112A') |(df_bitcar_80['name_to'] == 'KR112A;KR119')]
df_bitcar_13_tramo = df_bitcar_13[(df_bitcar_13['name_to'] == 'KR128;KR123') |(df_bitcar_13['name_to'] == 'KR123;KR128')]

#df_bitcarrier_tramo_speed = df_bitcarrier_tramo[['date','hour','month','weekday','average_speed']].groupby(['date','hour','month','weekday']).mean()
df_bitcar_tramo_speed_80 = df_bitcar_80_tramo[['date','hour','month','weekday','average_speed','direction']].groupby(['date','hour','month','weekday','direction']).mean()
df_bitcar_tramo_speed_13 = df_bitcar_13_tramo[['date','hour','month','weekday','average_speed','direction']].groupby(['date','hour','month','weekday','direction']).mean()


In [38]:
df_bitcar_tramo_speed_80.head()

average_speed
date       hour month weekday direction               
2019-01-01 0    1     Tuesday 0              42.849641
                              1              73.297378
           1    1     Tuesday 0              60.399577
                              1              57.598020
           2    1     Tuesday 0              57.317943

In [39]:
#df_bitcarrier_speed = df_bitcarrier[['date','hour','month','weekday','average_speed']].groupby(['date','hour','month','weekday']).mean()
df_bitcar_speed_80 = df_bitcar_80[['date','hour','month','weekday','direction','average_speed']].groupby(['date','hour','month','weekday','direction']).mean()
df_bitcar_speed_13 = df_bitcar_13[['date','hour','month','weekday','direction','average_speed']].groupby(['date','hour','month','weekday','direction']).mean()

In [40]:
#df_bitcarrier_2merge = df_bitcarrier_speed.merge(df_bitcarrier_tramo_speed,left_index = True,right_index=True)
df_bitcar_2merge_80 = df_bitcar_speed_80.merge(df_bitcar_tramo_speed_80,left_index = True,right_index=True)
df_bitcar_2merge_13 = df_bitcar_speed_13.merge(df_bitcar_tramo_speed_13,left_index = True,right_index=True)

#df_bitcarrier_2merge.columns = ['average_speed_total', 'average_speed_tramo']
df_bitcar_2merge_80.columns = ['average_speed_total', 'average_speed_tramo']
df_bitcar_2merge_13.columns = ['average_speed_total', 'average_speed_tramo']

df_bitcar_2merge_80 = df_bitcar_2merge_80.reset_index().set_index(['date','hour','month','weekday'])
df_bitcar_2merge_13 = df_bitcar_2merge_13.reset_index().set_index(['date','hour','month','weekday'])


In [41]:
#Dataframe to merge from BITCARRIER for calle 80
df_bitcar_2merge_80.head()

direction  ...  average_speed_tramo
date       hour month weekday             ...                     
2019-01-01 0    1     Tuesday          0  ...            42.849641
                      Tuesday          1  ...            73.297378
           1    1     Tuesday          0  ...            60.399577
                      Tuesday          1  ...            57.598020
           2    1     Tuesday          0  ...            57.317943

[5 rows x 3 columns]

In [42]:
#Dataframe to merge from BITCARRIER for calle 13
df_bitcar_2merge_13.head()

direction  ...  average_speed_tramo
date       hour month weekday             ...                     
2019-01-01 0    1     Tuesday          0  ...            39.452538
                      Tuesday          1  ...            48.416683
           1    1     Tuesday          0  ...            40.022312
                      Tuesday          1  ...            43.134039
           2    1     Tuesday          0  ...            39.019569

[5 rows x 3 columns]

#Load Accidents

In [43]:
accidents = pd.read_sql('''SELECT * from public.accidentes_13_80''', conn)

In [44]:
df_accidents19 = accidents[accidents['year'] == '2019']
df_accidents19['direction'] = df_accidents19['sentido'] == 'east_west'
df_accidents19['direction'] = df_accidents19['direction'].replace([False,True],[0,1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [45]:
df_accidents19 = df_accidents19.drop(['codigo_siniestro','hora','hora_1','gravedad','clase','choque_con','objeto_fijo','direccion','total_muertos','total_heridos',
                    'localidad','diseno_lugar','corredor','direcc_1','ciudad','pais','ciclista','conductor','motociclista','pasajero','peaton',
                    'total_condicion','herido','ileso','muerto','sin informacion_x','total_estado','femenino','masculino','total_sexo','automovil',
                    'bicicleta','bus','buseta','camioneta','campero','cuatrimoto','microbus','motocarro', 'motocicleta',	'motociclo'	,'tractocamion',
                    'volqueta',	'total_clase'	,'diplomatico',	'oficial'	,'particular'	,'publico',	'total_servicio','first_codc',	'codigo_causa',
                    'descripcion',	'direcc_2',	'direcc_3',	'direcc_copl','sentido','year_month',	'year' ,'dia','camion-furgon','lat','lon','accidents_road_1_cl80'], axis = 1)

In [46]:
df_accidents_13 = df_accidents19[df_accidents19['accidents_road_1_cl13'] == True]
df_accidents_13['accidents_road_1_cl13'] = df_accidents_13['accidents_road_1_cl13'].replace(True,1)
df_accidents_13.columns = ['date', 'month', 'weekday', 'hour', 'cargo_accident','accident', 'direction']
df_accidents_13['corridor'] = 'CL13'
df_accidents_13['accident'] = df_accidents_13['accident'].astype(int)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [47]:
df_accidents_13.head()

,date,month,weekday,hour,cargo_accident,accident,direction,corridor
1625,2019-01-23,1,Wednesday,8,1,1,0,CL13
1627,2019-03-02,3,Saturday,19,0,1,1,CL13
1628,2019-12-02,12,Monday,14,0,1,1,CL13
1629,2019-02-19,2,Tuesday,7,1,1,1,CL13
1630,2019-02-20,2,Wednesday,9,1,1,1,CL13


In [48]:
#Dataframe to merge from accidents for calle 13. Calle 80 doesn't have accidents for the last segment of the road
df_accidents_2merge = df_accidents_13.set_index(['date','hour','month','weekday'])

# Load precipitation

In [49]:
df_precipitation_og = pd.read_sql('''SELECT * from public.precipitacion_2019_2020''', conn)

In [50]:
df_precipitation = df_precipitation_og.copy() 

In [51]:
df_precipitation['hour'] = pd.to_datetime(df_precipitation['fecha'], format='%d/%m/%Y %H:%M').dt.hour
df_precipitation['weekday'] = pd.to_datetime(df_precipitation['fecha'], format='%d/%m/%Y %H:%M').dt.weekday
df_precipitation['month'] = pd.to_datetime(df_precipitation['fecha'], format='%d/%m/%Y %H:%M').dt.month
df_precipitation['fecha'] = pd.to_datetime(df_precipitation['fecha'], format='%d/%m/%Y %H:%M').dt.date
dias = np.arange(0,7,1)
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
df_precipitation['weekday'] = df_precipitation['weekday'].replace(dias,days)

In [52]:
df_precipitation_19 = df_precipitation[df_precipitation['fecha'] < datetime.strptime('2020-01-01', '%Y-%m-%d').date()]
df_precipitation_19.shape

(8760, 15)

In [53]:
df_precipitation_19['valor'] = df_precipitation_19['valor'].astype(float)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [54]:
df_precipitation_19.columns = ['codigoestacion', 'nombreestacion', 'latitud', 'longitud', 'altitud',
       'municipio', 'idparametro', 'etiqueta', 'date', 'precipitation', 'grado',
       'nivelaprobacion', 'hour', 'weekday', 'month']

In [55]:
df_precipitacion_2merge = df_precipitation_19[['date','hour','month','weekday','precipitation']].groupby(['date','hour','month','weekday']).mean()
df_precipitacion_2merge.head()

precipitation
date       hour month weekday               
2019-01-01 0    1     Tuesday            0.0
           1    1     Tuesday            0.0
           2    1     Tuesday            0.0
           3    1     Tuesday            0.0
           4    1     Tuesday            0.0

#Unique View

Merging RNDC, BITCARRIER, accidents and precipitation

In [56]:

#df_merged = df_rndc_2merge.merge(df_bitcarrier_2merge, left_index = True,right_index=True)
#df_pre_vista_unica = df_merged.join(df_accidents_2merge).fillna(0)
#df_vista_unica = df_pre_vista_unica.join(df_precipitacion_2merge)


df_merged_13 = df_rndc_2merge_13.merge(df_bitcar_2merge_13, left_index = True,right_index=True)
df_merged_80 = df_rndc_2merge_80.merge(df_bitcar_2merge_80, left_index = True,right_index=True)

df_merged_13.columns = ['C2G', 'C2P', 'TRACTOCAMIONES', 'camiones_equiv', 'direction_x','average_speed_total', 'average_speed_tramo']

df_pre_vista_unica_13 = df_merged_13.join(df_accidents_2merge).fillna(0)
df_pre_vista_unica_13['corridor'] = 'CL13'


df_vista_unica_13 = df_pre_vista_unica_13.merge(df_precipitacion_2merge,left_index = True,right_index=True)
df_vista_unica_13 = df_vista_unica_13.reset_index()
df_vista_unica_13.columns = ['date', 'hour', 'month', 'weekday', 'C2G', 'C2P', 'TRACTOCAMIONES',
       'camiones_equiv', 'direction', 'average_speed_total',
       'average_speed_tramo', 'cargo_accident', 'accident', 'direction_acc',
       'corridor', 'precipitation']
df_vista_unica_80 = df_merged_80.join(df_precipitacion_2merge)
df_vista_unica_80['corridor'] = 'CL80'
df_vista_unica_80 = df_vista_unica_80.reset_index()

In [57]:
#Concat calle 13 and calle 80

df_vista_unica = pd.concat([df_vista_unica_13,df_vista_unica_80]).fillna(0).set_index(['date', 'hour', 'month', 'weekday'])
df_vista_unica[['C2G','cargo_accident','C2P','TRACTOCAMIONES','cargo_accident','accident','direction']] = df_vista_unica[['C2G','cargo_accident','C2P','TRACTOCAMIONES','cargo_accident','accident','direction']].astype(int)
df_vista_unica.columns = ['C2G', 'C2P', 'TRACTOCAMIONES', 'camiones_equiv', 'direction',
       'average_speed_total', 'average_speed_tramo', 'cargo_accident',
       'accident', 'direction_acc', 'corridor', 'precipitation']
df_vista_unica.head()

C2G  C2P  ...  corridor  precipitation
date       hour month weekday            ...                         
2019-01-01 0    1     Tuesday    0    0  ...      CL13            0.0
                      Tuesday    0    0  ...      CL13            0.0
           1    1     Tuesday    0    0  ...      CL13            0.0
                      Tuesday    0    0  ...      CL13            0.0
           2    1     Tuesday    0    0  ...      CL13            0.0

[5 rows x 12 columns]